### Import Data

#### - Import Data

In [ ]:
# -- Import libraries for HTTP requests, authentication, and URL handling
import requests                           # -- For making HTTP requests to the DHIS2 API
from requests.auth import HTTPBasicAuth   # -- For handling basic authentication in HTTP requests
from urllib.parse import quote            # -- For URL encoding special characters
from getpass import getpass               # -- For securely collecting the passkey (hidden input)
import dataframe_image as dfi
import textwrap
import socket                             # -- For handling DNS resolution errors
from urllib3.exceptions import NameResolutionError
from requests.exceptions import HTTPError, RequestException  # -- For handling HTTP and request-related exceptions
import time                               # -- For using time.sleep in retry logic

# -- Import data manipulation and numerical libraries
import pandas as pd                       # -- For data manipulation and creating dfs
import numpy as np                        # -- For data manipulation and numerical operations

# -- Import Jupyter Notebook display and interactive utilities
from IPython.display import clear_output  # -- For clearing screen display in Jupyter 
from IPython.display import display, HTML       # -- For displaying dfs in Jupyter Notebooks
import ipywidgets as widgets              # -- For creating interactive buttons and widgets
from functools import partial             # -- For cleaner argument binding in function calls

# -- Import file, system, and regular expression utilities
import os                                 # -- For operating system interactions (e.g., file paths)
import sys                                # -- For system-specific parameters and functions
import re                                 # -- For working with regular expressions

# -- Import Excel file manipulation and styling tools
from openpyxl import load_workbook        # -- For working with Excel files
from openpyxl.styles import Font, Alignment  # -- For styling and aligning Excel cells

# -- Import document and image processing libraries
from docx import Document                # -- For creating and editing Word documents
from docx.shared import Pt, RGBColor, Inches  # -- For Word document styling (e.g., font size, color, dimensions)
from PIL import Image                    # -- For image processing

#### - IHVN DHIS2 API

In [ ]:
# -- Define the main function to fetch and process DHIS2 data
def fetch_and_process_DHIS2_data(username, password, start_period, end_period, named_urls=None):
    """
    Fetch and process DHIS2 data from named URLs with a user-specified period range, returning a dictionary of processed dfs.
    
    Args:
        username (str): DHIS2 username for authentication
        password (str): DHIS2 password for authentication
        start_period (str): Start period in YYYYMM format (e.g., '202501')
        end_period (str): End period in YYYYMM format (e.g., '202503')
        named_urls (dict, optional): Dictionary where keys are names (e.g., Report Rate Facility) and values are DHIS2 API URLs.
                                    If None, a default set of URLs is used.
    
    Returns:
        dict: Dictionary where keys are URL names and values are processed dfs
    """

    # -- Step 4: Define separator line
    separator_line = '-' * 43                                                  # -- Create a separator line of 43 dashes

    # -- Step 1: Define default DHIS2 URLs if none are provided
    if named_urls is None:                                                    # -- Check if named_urls is not provided
        named_urls = {
            "Report Rate Facility": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3AKH62ia35VIZ%3Bum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false",
            "Report Rate LGA": "https://ihvn.dhistance.com/api/analytics.json?dimension=pe%3A202501&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-lmSTo2yxNsA&dimension=dx%3AZ7E9RxXmwxG.REPORTING_RATE%3BVmGwLcfPS2N.REPORTING_RATE%3BYFnIy7lATQL.REPORTING_RATE%3BNkuV7xoThHV.REPORTING_RATE%3BHwfLR3npibF.REPORTING_RATE%3BvN9rk5ChByM.REPORTING_RATE%3BoxUN7AXSF8r.REPORTING_RATE&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "AGYW MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AQ1KKjeS4seJ%3BzhvcKIWKvEX%3BpgsZWQLbTvw%3BdPefeXOI0MT%3Bb62rwfvBP13%3Bechn4uCHBhF%3BeoSKe92wBYa%3BIYJciZgP6Yt%3BbbSqZH1OAxk%3By5mYZFQbMe6%3BSSa2P2O1keL%3BrtjZkt3ImND%3BDhw4lcmA8i5%3BAMs19im8mm7%3BTNTPcRPC3jV%3BWWaSbjutZof%3BcspEoTIBnOB%3BQ1ntMoY7ZrP%3BjQ51vKvy1SN&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "ART MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AEVJnWv5UQ2I%3BaDFx7U0OSNp%3BuTAVBA24Qgg%3BVNvZaoEcS8M%3BdJpKA2CL66w%3BwUlUHsYzh80%3BmaTBR3htwav%3BN4skK4jJVnm%3BcMpe2pMLJed%3BmJ3Af4Qg0wV%3BxFsrvhyu0Wx%3BCVIR5mDSrr0%3BqO3FSAwqg15%3BE8J56tfMIEa%3BBiqkhMdFIwy%3BfVpRSB8jy9Q%3BudXxeZhT8Fd%3BE20mRpvl5jK%3Bgcg9I4dagWN%3BfBJzc5QIP1b%3BgDuNCzc5liq%3BngCJ4UZOCme%3BEtg6BPVX548%3BiTJ2VvKOWHG%3BrJB5XXrF5zx%3BFRMmrIYSRfz%3BPhUxFwPj2US%3BKtMzH6OTxXL%3Bu0W1SpovSd3%3Bxz9C4uZwMuB%3BLqdahCtUzSX%3Bqb3YzC5X9Lo%3BOU60086uSKx%3Bs2d4Hk231P9%3BmnbibCKaDNb%3BlEdQdPdK5KL&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "ART MSF_tb screening": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AaDFx7U0OSNp&dimension=sbKiaUuaHpX%3AmyDWsEb0XMg%3BnsnGDTrtxbw&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "HTS MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AsxXB2RokZrt%3BhxYcq0LyXh6%3Bi3I5wU8vm0U%3BwSLnU5ihgb3%3BVz0ZjXCnFtX%3BK7Bkwdae7X1%3BRBxwgKGZYpv%3BjNodLWC4U4d%3Bq6bey1tg06I%3BAZSZngrMzj6%3BpS0Cjik4WEH%3BBrVqJd9MHSA%3BXKu24OQK64R%3BpgiqhOhD95N%3BYL87jqnIzQA%3BmN5TqnUQRgq%3BEvB0bzvxMq0%3BgWVJbVYOT1A%3Bd8lMQuQpLPe%3BT6f8BA7M9Q0%3BSVpx1fVD3bh%3BpJH2bCApdTe%3BuGUbRjQvCeN%3BE3bZgIIG5qQ%3Bd8pnVkpyGlU%3BDn4PJOJJASf%3BQV3wq0WsLXe%3Bs9ksVidbnRG%3BOFj0H56KbKp%3Bh9SMVFhNbBN%3BrJD9ER08iT4%3BJLOnLNuLOva%3BZztEVzPBwwl%3BX6wyczqYmJN%3Bvhhrj8rTq83%3ByR5JEOs2t7q%3BCnSrjlY5Siw%3BUMPLBlGYjWM%3BiE5kXuUJoMC%3BuMqKjodBP76%3Bu9gp0VFbnHh%3BzJuaAQbUlq8%3BQXNyy0dbIpd%3BomaL0XteWO8%3BKtzvesDYABJ%3BHdTQzlvMtjU%3Bqznyc2H90Ay%3Bdtri8UEwZFV%3BFB1EZfVSmYi%3BXzghRlcLMqB%3BOk6fJzfTk75%3BomujB6405jI%3BenPRkAVrHKi%3BE6wddhqJVIN%3BjWajcYCyiDE%3BLSQuUgoTH7o%3Bilp3JWW1qt5%3BURgQ4d3y8WF%3Bxc40kwzTBVG%3BmCD2Qhbd6CY%3BnQtTVOdEMqV%3BKQAEkHXQe7N%3BxBr9Sgyk4cO%3BlI7YzdC7wEd%3BegRRIwWxnJs%3BPPxR0HhCKQR%3BkmBNFDE8duJ%3BGTUzO3HGLWA%3Bh8HFl5EeHHl%3BuXwAyGT9eqW%3BDahOUj6bRk0%3BT8G2KNNZ4eI%3BD7ygy6yCHFs%3BuSQJcqAEvHg%3BU74jSwLCoA1%3BZheiLjTrqRZ%3BjlFslOk3bkU%3BOxBKckVqp29%3BJWHc7D4J132%3Bq1XlBEcB1PE%3BNb95zNUKf29%3BevlfYhoKrjI%3BU8xSE6OneYl%3BF2JEmiJt3Yl%3BPZWrrb7VyCj%3BXxpFcHK1S89%3BsICfdv3or5G%3Bg5HvgkCKSSU%3Bh7XmbTNyTUi%3BUi7DiLwSgqm%3BHIHQHXPOGKl%3Bl3aXhanFimZ%3BSl6d3hqzq2C%3BpIsmPa1GjFs%3BOwrvPMKq0pQ%3BV7hcDYvuPMY%3BDWb8URoPRym%3BL7l30ySaQDy%3BnYrdLtwDlV0%3BifE9LKaLqUm%3BuMkZMIHVVjV%3BcCLFgRUkIww%3BHUWYU2ruloN%3BoXsckqTpzhN%3Bm4CzDuc50Jn%3BLPyxdv2eBEj%3BZLARI5LYBOL%3BPYL5GdQGPfI%3BmQZ4z94jzak%3BRkwT0w0mPNj%3BEC5iN37lZ61%3BS0QI1IcESjq%3BnR3ZklnEuBy%3BgxZKoTwyLnn%3BIp75Jb7o8Au%3BVPMZR303TWP%3BjqP0tN3v5hJ%3BuoBeVy413Mj%3BPn1GooQL0I4%3BEh4GHGcZV1U%3BJr0gixBBrvT&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "HTS MSF_hivst approach": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AT8G2KNNZ4eI&dimension=tBdRxXi3Dxr%3AALL_ITEMS&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "NSP MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AxesSKTdzhPF%3BuIx4sChOeMC%3BkJSLUKrBKez%3BJylie7CPg63%3BTV5DhhOgF7s%3BL0X8cbiEfrC%3BoSDqDrfRpb3%3BIQYpNyC6lF2%3BlOVVI8B4Ag6%3BRxQv56EIuWs%3BqyV24hEIgM5%3BObJbRGQ3QPI%3BptgY5CK3GUc%3BC16TIH6Zxju%3BKY2gb90cvTu%3BO3g2Lq9fpUU%3Bkk3hEztWa48%3BxvfqoSSOIOL%3BcLSlAzBug6Y%3BayTk1t8sjFN%3BjY2SLdSlMug%3Bv3sfwf9O1R9%3BxrHzg7SORIt&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ABWLsXE490Yw%3BAtzuY8wFIaP%3BmjoPYWLowE6%3BhWkEco9hG0R%3BgybVHn9Y3SQ%3BkNEHSaIxOja%3BSWsoIgeQBKd%3Bym1WjkXu8ol%3BHDVcSJrN68V%3BIY3zDXL1t0b%3BCrv4RYmAJot%3Bn1MZAkrNWlM%3BGDFB54Uuepj%3BPaXizDbvXcE%3BhnqKzrB2hX0%3BZv0Go6hoDxZ%3BZOzgBnzWLOK%3Bbj8jc0HujI2%3BVlhKdJD8Mav%3BqO3KHhkVM38%3BkSHVwcO9iYZ%3BWnEhg2SiMjF%3BFR9ibOgyNKg%3BbyiZy4Xo7Rx%3BvpEeS23uOzB%3BzoSht7Xk3OI%3BUMFtzIDX9nE%3BxkQqd3C5vd8%3BCAO8uuth3q6%3BRATzsmd7JfL%3BDPFGLfkzZTF%3BZZEXn1RzLwL%3BuMATgKm1ZEV%3ByShXhGc00TY%3Bdc0p7yc1jPy%3BJsyAmKQMqK1%3BlIJwQ20CgPE%3BbxeHOqH0s2V%3BOum5ofYWexC%3BadqiOXDSRbU%3BHFq8NihMhgw%3BcibdW4TLOJ5%3BMJHhWCCOy1q%3BPQHBZmMVWE3%3Bw9NSJbXp5Ub%3BCyrYJsUOe7Q%3BER9cFdvluIM%3BUP3mRp6Zr69%3Bgl5kyYu85zX%3BrIY8zpZJxtv%3BaPtwtDwhOeL%3BuQEZL5j8ZHS%3BSpuoRhODIuZ%3Bvu5TP1CuesM%3BeZ4kUaPlQWC%3BSVDAcLT6IKS%3BFLt9JLhBWlK%3Bt9NJ4JhsXVl%3BnOpah0JpIzI%3BDuFJekp1ymh%3BTJ5BREHjZnu%3BWzAmqotjxV7%3BwRRoicp2S4Y%3Bzq6Tvgk5GFl%3BLMrI0jXhjP4%3BTDLVNVgHUvf%3BakVDx9ew0y8%3ByDAbvAlPR69%3Bi6rCuewXarq%3BzKYuhIKhMKM%3BGQ4kUNGMf7i%3Bv2e5LaziiBr%3BsLWmlnDQzgt%3BrKzvhBgkpe5%3BpyUW87YFbY5%3BssvxYtnGXpp%3BlY6wzD8718l%3BeLZuFwiv4rQ%3BDRFLRwv7f8F%3BLgbHzzQILZY%3BWt8jhQxqq9L%3BTdqRbqoS3Uj%3BcfUpkarx8og%3BWbRBSkKaTE8%3BEvZz6jNSRMX%3BYGZr2K5Y0Yq%3BJGj4bfhtzCk%3BkKSgWbeTAL1%3Bo9zPtZI4c9T%3BcmkwuM6ZOaN%3BcGPBuUrZ5Oi%3BN4WysYNBRFk%3Bd0izXgJhn2P%3BxUYwPZH7ulJ%3BykZcKOVoJeH%3Bj3ycCFq6vzR%3BK2lEmtE4xjz%3Bw4jlyeHkVTD%3BDbj5r3zevDF%3BZYmUM2TtV80%3BYfvO0pN0SVl%3BwbplLMxCVSo%3BEJ31P0DyQaY&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "KP Prev MSF": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3ArJFps9PGgYI%3BpFrm4E6yE0i%3Bwld8ChYjUUS%3Bhnqs8zD2RBz%3BFdogfvZ9Es5%3BUWnjd2hDMpZ%3BmM3TZYEkMaF%3BK93mvkTXpbl%3BtIZvjax5TD5%3BN05zqacXFsD%3BcVHdvMKuwse%3BBFSM6D4KWg4%3ByHBgGkS0dPr%3BEw3ZRxdYY7g%3BihRnS748lOC%3BlIu2vmR2zHU%3BfbxvYGgVsrK%3BTdATue0b5th%3BFBBKTzFqBVR%3BF27CfSgCmSk%3BdecdNlpZy4B%3BRVuw3Ny3ysw%3BhN5ZT3zziqA%3BKngVqiOqAdp%3Bdfznt6rhrYg%3BbxK7Uh32HMK%3BduzVXts1QUh%3BXpDOj1a7ZdC%3BogXBJLCw1Qe%3BAKQTKPoUHt6%3BrClFr1FdYVY%3BdTWbSB6HZg6%3BGZo5a5CwzV7%3BdIElcrfEgSB%3BP8oCCg2mUcG%3BZc7dmnNEQwj%3BmWQsKiSyv7J%3BQVOL72RTOZR%3Bri9iXOK3wxX%3Bpqp8Tyvsdc1%3BNat3LtmEWXD%3BPm78aQ5UVh8%3BWaI8nE16Ab0%3BfHlNG1CMqYX%3BGwxEq7jrcSc%3BiqqfsLcdWiK%3BeF7Yi0cLTXP%3BVnDrAREzUE2%3BSSxprjJB8vd%3BL5pAPbPCrNF%3BPrdKtyHsOE8%3BLNQtCiSCfnC%3Bb3JIlAZUjNJ%3BeK3YlBsHGkB%3BTz9WrAjxVY9%3BMkl91koS5cn%3BdAQlguHfLF8%3BBdgtqOw1P33%3BhZzpfxOQo4b%3BJBoik13RWdP%3BhAVQ3sTq3nJ%3BGKiJWrIeEJu%3BqqFvQly7I9h%3BJtOlLSk4yTF%3BUfMiT3LdXYc%3BdLj7PgakBi5%3BUaOdcjrcI59%3BssQZNWweFrL%3BhHkzZLsAqmI%3BkYAhnVk8fYp%3BkTIGrSkqbrG%3Bl4ZD3aaxH7j%3BWAZm2HXm3xI%3BSECmJH5Lfer%3BKHnHX7Rb29d%3BVE0Z777wXB2%3Bzgqte2mbKxT%3BDbavrmvoWPG%3BJgDiY0dv2RX%3BS8WFXWTFvai%3BBBwzjs2JQ1Z%3BxRJrWUdpAxm%3BYS8QAyBakNp%3BJfPSsLKaeDV%3BkmsvoBtZ4oa&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT MSF_sdp": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AN4WysYNBRFk&dimension=brKpOJgkKa0%3AOLcx26MiJia%3BQ9RcEpV6443%3BEGHRBByvqFu&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT MSF_sdp_pos": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AykZcKOVoJeH&dimension=brKpOJgkKa0%3AOLcx26MiJia%3BQ9RcEpV6443%3BEGHRBByvqFu&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT MSF_sd<72_in-outside": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3Av2e5LaziiBr&dimension=FtBUOVZVrC6%3AE8fr3yVA0mn%3BS0QLh0UvPbm&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID",
            "PMTCT MSF_sd>72_in-outside": "https://ihvn.dhistance.com/api/analytics.json?dimension=dx%3AGQ4kUNGMf7i&dimension=FtBUOVZVrC6%3AE8fr3yVA0mn%3BS0QLh0UvPbm&dimension=pe%3A202501%3B202502%3B202503%3B202504&dimension=ou%3Aum5TFmcsSi8%3BLEVEL-UIlRiekzsf6&showHierarchy=true&hierarchyMeta=true&includeMetadataDetails=true&includeNumDen=true&skipRounding=false&completedOnly=false&outputIdScheme=UID"
        }                                                                    # -- Initialize dictionary for URLs

    # -- Step 2: Extract dx IDs from named_urls
    all_dx_ids = set()                                                        # -- Initialize empty set to store unique dx IDs
    for url in named_urls.values():                                           # -- Iterate over each URL in named_urls
        dx_matches = re.findall(r'dx%3A([^&]+)', url)                         # -- Extract dx parameter from URL using regex
        for match in dx_matches:                                              # -- Process each dx match found
            ids = [id.split('.')[0] for id in match.split('%3B')]             # -- Split by %3B and remove .REPORTING_RATE suffixes
            all_dx_ids.update(ids)                                            # -- Add cleaned IDs to the set

    # -- Step 3: Fetch only relevant data element descriptions
    base_url = 'https://ihvn.dhistance.com/api/dataElements'                  # -- Set base URL for data elements endpoint
    dx_filter = f"id:in:[{','.join(all_dx_ids)}]"                             # -- Create filter string for specific dx IDs
    params = {                                                                # -- Define query parameters for the API request
        'fields': 'id,name,description',                                      # -- Request id, name, and description fields
        'filter': dx_filter,                                                  # -- Apply filter for specific IDs
        'paging': 'false'                                                     # -- Disable pagination to get all results
    }
    # Retry parameters
    max_retries = 3                                                           # -- Set maximum retry attempts
    retry_delay = 5                                                           # -- Set delay between retries in seconds
    dataelement_to_description = {}                                           # -- Initialize dictionary for data element descriptions
    for attempt in range(1, max_retries + 1):                                 # -- Loop through retry attempts
        try:
            socket.create_connection(("8.8.8.8", 53), timeout=5)              # -- Test DNS resolution with Google's DNS
            response = requests.get(
                base_url,
                auth=HTTPBasicAuth(username, password),
                params=params,
                timeout=30                                                    # -- Add timeout to prevent hanging
            )
            response.raise_for_status()                                       # -- Raise exception for HTTP errors
            data_elements = response.json().get('dataElements', [])           # -- Extract data elements from response
            print(f"Fetched {len(data_elements)} data elements")              # -- Log number of fetched data elements
            dataelement_to_description = {
                de['id']: de.get('description', de['name']) for de in data_elements
            }                                                                 # -- Map data element IDs to descriptions or names
            break                                                             # -- Exit retry loop on success
        except HTTPError as e:                                                # -- Handle HTTP errors
            if response.status_code == 401:                                   # -- Check for unauthorized error
                print(separator_line)                                         # -- Print separator line
                print("⦸ Error: Invalid IHVN DHIS2 login credentials")        # -- Notify user of invalid credentials
                print(separator_line)                                         # -- Print separator line
                return {}                                                     # -- Return empty dict to halt execution
            else:
                print(separator_line)                                         # -- Print separator line
                print(f"⦸ HTTP Error (Attempt {attempt}/{max_retries}): HTTP status code {response.status_code} - {response.reason}")
                                                                              # -- Log HTTP error with status and reason
            if attempt == max_retries:                                        # -- Check if max retries reached
                print(separator_line)                                         # -- Print separator line
                print("⦸ Error: Max retries reached. Unable to fetch data elements.")  # -- Notify user of failure
                print(separator_line)                                         # -- Print separator line
                return {}                                                     # -- Return empty dict
            time.sleep(retry_delay)                                           # -- Wait before retrying
        except (ConnectionError, NameResolutionError) as e:                   # -- Handle network/DNS errors
            print(separator_line)                                             # -- Print separator line
            print(f"⦸ Network Error (Attempt {attempt}/{max_retries}): Unable to resolve 'ihvn.dhistance.com'. Check domain or network connection.") # -- Notify user of network/DNS issue
            print(separator_line)                                             # -- Print separator line
            if attempt == max_retries:                                        # -- Check if max retries reached
                print(separator_line)                                         # -- Print separator line
                print("⦸ Error: Max retries reached. Please verify the domain name or check your internet connection.") # -- Suggest checking domain/network
                print(separator_line)                                         # -- Print separator line
                return {}                                                     # -- Return empty dict
            print(f"Retrying in {retry_delay} seconds...")                    # -- Log retry attempt
            time.sleep(retry_delay)                                           # -- Wait before retrying
        except RequestException as e:                                         # -- Handle other request errors
            print(separator_line)                                             # -- Print separator line
            print(f"⦸ Request Error (Attempt {attempt}/{max_retries}): A general error occurred while trying to fetch data elements.") # -- Log general request error
            if attempt == max_retries:                                        # -- Check if max retries reached
                print(separator_line)                                         # -- Print separator line
                print("⦸ Error: Max retries reached. Unable to fetch data elements.") # -- Notify user of failure
                print(separator_line)                                         # -- Print separator line
                return {}                                                     # -- Return empty dict
            time.sleep(retry_delay)                                           # -- Wait before retrying

    # -- Step 5: Validate and parse the start and end periods
    # -- Step 5.1: Validate start and end periods
    for period in [start_period, end_period]:                                 # -- Check both start and end periods
        if not (isinstance(period, str) and len(period) == 6 and period.isdigit() and 1 <= int(period[4:]) <= 12): # -- Validate format and month range
            print(separator_line)                                             # -- Print separator line
            print("⦸ Error: Invalid period date format")                     # -- Print error message
            print(separator_line)                                             # -- Print separator line
            return {}                                                         # -- Return empty dict to halt execution

    # -- Step 5.2: Parse the start and end periods
    start_year = int(start_period[:4])                                        # -- Extract year from start period
    start_month = int(start_period[4:])                                       # -- Extract month from start period
    end_year = int(end_period[:4])                                            # -- Extract year from end period
    end_month = int(end_period[4:])                                           # -- Extract month from end period

    # -- Step 6: Format the period range for display
    month_names = {                                                           # -- Define month names for display
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
        7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    }
    start_display = f"{month_names[start_month]}{str(start_year)[-2:]}"       # -- Format start period (e.g., Jan-25)
    end_display = f"{month_names[end_month]}{str(end_year)[-2:]}"             # -- Format end period (e.g., Mar-25)

    # -- Step 7: Print separator line to highlight successful setup
    print('Data processed and stored as:')                                    # -- Print processing start message
    print(separator_line)                                                     # -- Print separator line

    # -- Step 8: Generate a list of periods between start and end
    periods = []                                                              # -- Initialize empty list for periods
    current_year, current_month = start_year, start_month                     # -- Set starting point for period generation
    while (current_year < end_year) or (current_year == end_year and current_month <= end_month): # -- Loop until end period is reached
        periods.append(f"{current_year}{current_month:02d}")                  # -- Add period in YYYYMM format
        current_month += 1                                                    # -- Increment month
        if current_month > 12:                                                # -- If month exceeds 12
            current_month = 1                                                 # -- Reset to January
            current_year += 1                                                 # -- Increment year

    # -- Step 9: Encode the periods for URL use
    period_string = "%3B".join(periods)                                       # -- Join periods with URL-encoded semicolon
    period_param = f"dimension=pe%3A{period_string}"                          # -- Format period parameter for API URL

    # -- Step 10: Initialize data storage and counters
    processed_data = {}                                                       # -- Initialize dict to store processed DataFrames
    success_count = 0                                                         # -- Counter for successful URL processes
    total_urls = len(named_urls)                                              # -- Total number of URLs to process

    # -- Step 11: Define cluster mapping for LGAs
    cluster = {                                                               # -- Define mapping of LGAs to clusters
        # -- Cluster: Aguata
        "an Aguata": "Aguata",
        "an Anaocha": "Aguata",
        "an Orumba North": "Aguata",
        "an Orumba South": "Aguata",
        # -- Cluster: Awka
        "an Awka North": "Awka",
        "an Awka South": "Awka",
        "an Dunukofia": "Awka",
        "an Idemili North": "Awka",
        "an Idemili South": "Awka",
        "an Njikoka": "Awka",
        # -- Cluster: Nnewi
        "an Ekwusigo": "Nnewi",
        "an Ihiala": "Nnewi",
        "an Nnewi North": "Nnewi",
        "an Nnewi South": "Nnewi",
        # -- Cluster: Omambala
        "an Anambra East": "Omambala",
        "an Anambra West": "Omambala",
        "an Ayamelum": "Omambala",
        "an Oyi": "Omambala",
        # -- Cluster: Onitsha
        "an Ogbaru": "Onitsha",
        "an Onitsha North": "Onitsha",
        "an Onitsha South": "Onitsha"
    }

    # -- Step 12: Process each named URL
    for url_name, url in named_urls.items():                                  # -- Iterate over each name-URL pair
        # -- Step 12.1: Update URL with new period range
        if "dimension=pe%3A" in url:                                          # -- Check if URL has a period dimension
            start_idx = url.find("dimension=pe%3A")                           # -- Find start of period parameter
            end_idx = url.find("&", start_idx) if url.find("&", start_idx) != -1 else len(url) # -- Find end of period parameter
            url = url[:start_idx] + period_param + url[end_idx:]              # -- Replace old period with new one
        else:                                                                 # -- If no period dimension exists
            url = url + "&" + period_param if "?" in url else url + "?" + period_param # -- Append period parameter

        # -- Step 12.2: Fetch data from DHIS2 API
        try:
            response = requests.get(url, auth=HTTPBasicAuth(username, password)) # -- Send GET request with authentication
            response.raise_for_status()                                       # -- Raise exception if request fails
            data = response.json()                                            # -- Parse response into JSON
        except RequestException as e:                                         # -- Catch request-related exceptions
            print(f"⦸ Error: No signal to get '{url_name}' data")            # -- Print error message
            continue                                                          # -- Skip to next URL

        # -- Step 12.3: Extract table structure from JSON
        headers = [header['name'] for header in data.get('headers', [])]      # -- Get column names from headers
        df = pd.DataFrame(data.get('rows', []), columns=headers)              # -- Create DataFrame from rows
        df = df.rename(columns={'dx': 'dataElement', 'ou': 'orgUnit', 'pe': 'period'}) # -- Standardize column names

        # -- Step 12.4: Extract metadata from the JSON response
        meta = data.get('metaData', {})                                       # -- Get metadata section
        ou_hierarchy = meta.get('ouHierarchy', {})                            # -- Get organizational unit hierarchy
        items = meta.get('items', {})                                         # -- Get item mappings (IDs to names)

        # -- Ensure dimension names are available for special URLs
        if url_name == 'ART MSF_tb screening':                                # -- Check for ART MSF_tb screening URL
            dimension_id = 'sbKiaUuaHpX'                                      # -- Set dimension ID
        elif url_name == 'HTS MSF_hivst approach':                            # -- Check for HTS MSF_hivst approach URL
            dimension_id = 'tBdRxXi3Dxr'                                      # -- Set dimension ID
        elif url_name == 'PMTCT MSF_sdp':                                     # -- Check for PMTCT MSF_sdp URL
            dimension_id = 'brKpOJgkKa0'                                      # -- Set dimension ID
        elif url_name == 'PMTCT MSF_sdp_pos':                                 # -- Check for PMTCT MSF_sdp_pos URL
            dimension_id = 'brKpOJgkKa0'                                      # -- Set dimension ID 
        elif url_name == 'PMTCT MSF_sd<72_in-outside':                        # -- Check for PMTCT MSF_sd<72_in-outside URL
            dimension_id = 'FtBUOVZVrC6'                                      # -- Set dimension ID
        elif url_name == 'PMTCT MSF_sd>72_in-outside':                        # -- Check for PMTCT MSF_sd>72_in-outside URL
            dimension_id = 'FtBUOVZVrC6'                                      # -- Set dimension ID  
        else:
            dimension_id = None                                               # -- No dimension ID for other URLs

        if dimension_id:                                                      # -- If dimension ID is set
            dimension_values = set(df[dimension_id].unique())                 # -- Get unique dimension values
            missing_dimensions = [dv for dv in dimension_values if dv not in items] # -- Find missing dimension values
            if missing_dimensions:                                            # -- If there are missing dimensions
                dimension_url = 'https://ihvn.dhistance.com/api/categoryOptions'   # -- Set URL for category options
                dimension_filter = f"id:in:[{','.join(missing_dimensions)}]"   # -- Create filter for missing dimensions
                dimension_params = {
                    'fields': 'id,name',
                    'filter': dimension_filter,
                    'paging': 'false'
                }                                                             # -- Define query parameters
                try:
                    dimension_response = requests.get(
                        dimension_url,
                        auth=HTTPBasicAuth(username, password),
                        params=dimension_params
                    )                                                         # -- Fetch dimension names
                    dimension_response.raise_for_status()                     # -- Raise exception for HTTP errors
                    dimensions = dimension_response.json().get('categoryOptions', [])  # -- Extract category options
                    for dim in dimensions:
                        items[dim['id']] = {'name': dim['name']}              # -- Add to items
                except RequestException as e:                                 # -- Handle request errors
                    print(f"⦸ Warning: Failed to fetch dimension names for {url_name}: {e}") # -- Log warning

        # -- Step 12.5: Create organizational unit mappings based on URL name
        if url_name == "Report Rate LGA":                                     # -- Special case for Report Rate LGA
            orgunit_to_level = {ou: ou for ou in df['orgUnit'].unique()}      # -- Map orgUnit to itself
        else:                                                                 # -- For other URLs
            orgunit_to_level = {
                ou: ou_hierarchy.get(ou, '').split('/')[1] if '/' in ou_hierarchy.get(ou, '') else ou
                for ou in df['orgUnit'].unique()
            }                                                                 # -- Extract second level from hierarchy

        # -- Step 12.6: Create name mappings for organizational units and data elements
        level_to_name = {
            org_id: items[org_id]['name']
            for org_id in set(orgunit_to_level.values()) if org_id in items
        }                                                                 # -- Map orgUnit level IDs to names
        orgunit_to_name = {
            ou: items[ou]['name']
            for ou in df['orgUnit'].unique() if ou in items
        }                                                                 # -- Map orgUnit IDs to names
        dataelement_to_name = {
            de: items[de]['name']
            for de in df['dataElement'].unique() if de in items
        }                                                                 # -- Map dataElement IDs to names
        if url_name in ['ART MSF_tb screening', 'HTS MSF_hivst approach', 'PMTCT MSF_sdp', 
                        'PMTCT MSF_sdp_pos', 'PMTCT MSF_sd<72_in-outside', 'PMTCT MSF_sd>72_in-outside']:  # -- Check for special URLs
            column_map = {
                'ART MSF_tb screening': 'sbKiaUuaHpX',
                'HTS MSF_hivst approach': 'tBdRxXi3Dxr',
                'PMTCT MSF_sdp': 'brKpOJgkKa0',
                'PMTCT MSF_sdp_pos': 'brKpOJgkKa0',
                'PMTCT MSF_sd<72_in-outside': 'FtBUOVZVrC6',
                'PMTCT MSF_sd>72_in-outside': 'FtBUOVZVrC6'
            }                                                               # -- Define column mapping for URL names
            dimension_values = df[column_map[url_name]].unique()            # -- Get unique dimension values from selected column

        # -- Step 12.7: Pivot the df to reshape the data
        if url_name == 'HTS MSF_hivst approach':                              # -- Check for HTS MSF_hivst approach
            if 'tBdRxXi3Dxr' not in df.columns:                               # -- Verify dimension column exists
                print(f"⦸ Warning: Dimension tBdRxXi3Dxr missing for {url_name}") # -- Log warning
                continue                                                      # -- Skip to next URL
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],
                columns='tBdRxXi3Dxr',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        elif url_name == 'ART MSF_tb screening':                              # -- Check for ART MSF_tb screening
            if 'sbKiaUuaHpX' not in df.columns:                              # -- Verify dimension column exists
                print(f"⦸ Warning: Dimension sbKiaUuaHpX missing for {url_name}") # -- Log warning
                continue                                                      # -- Skip to next URL
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],
                columns='sbKiaUuaHpX',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        elif url_name == 'PMTCT MSF_sdp':                                     # -- Check for PMTCT MSF_sdp
            if 'brKpOJgkKa0' not in df.columns:                              # -- Verify dimension column exists
                print(f"⦸ Warning: Dimension brKpOJgkKa0 missing for {url_name}") # -- Log warning
                continue                                                      # -- Skip to next URL 
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],
                columns='brKpOJgkKa0',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        elif url_name == 'PMTCT MSF_sdp_pos':                                     # -- Check for PMTCT MSF_sdp
            if 'brKpOJgkKa0' not in df.columns:                              # -- Verify dimension column exists
                print(f"⦸ Warning: Dimension brKpOJgkKa0 missing for {url_name}") # -- Log warning
                continue                                                      # -- Skip to next URL 
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],
                columns='brKpOJgkKa0',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        elif url_name == 'PMTCT MSF_sd<72_in-outside':                        # -- Check for PMTCT MSF_sd<72_in-outside
            if 'FtBUOVZVrC6' not in df.columns:                              # -- Verify dimension column exists
                print(f"⦸ Warning: Dimension FtBUOVZVrC6 missing for {url_name}") # -- Log warning
                continue                                                      # -- Skip to next URL 
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],        
                columns='FtBUOVZVrC6',
                values='value'  
            ).reset_index()                                                   # -- Pivot DataFrame
        elif url_name == 'PMTCT MSF_sd>72_in-outside':                        # -- Check for PMTCT MSF_sd>72_in-outside
            if 'FtBUOVZVrC6' not in df.columns:                              # -- Verify dimension column exists    
                print(f"⦸ Warning: Dimension FtBUOVZVrC6 missing for {url_name}")   
                continue                                                      # -- Skip to next URL
            pivoted_df = df.pivot(  
                index=['period', 'orgUnit'],
                columns='FtBUOVZVrC6',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        else:
            pivoted_df = df.pivot(
                index=['period', 'orgUnit'],
                columns='dataElement',
                values='value'
            ).reset_index()                                                   # -- Pivot DataFrame
        pivoted_df.columns.name = None                                        # -- Clear column index name

        # -- Step 12.8: Format the 'period' column to 'Mon-YY' (e.g., Jan-24)
        pivoted_df['period'] = pd.to_datetime(pivoted_df['period'], format='%Y%m').dt.strftime('%b-%y') # -- Convert YYYYMM to Mon-YY

        # -- Step 12.9: Add organizational hierarchy information to the pivoted df
        pivoted_df['orgunitlevel'] = pivoted_df['orgUnit'].map(orgunit_to_level) # -- Add orgUnit level
        pivoted_df['LGA'] = pivoted_df['orgunitlevel'].map(level_to_name)     # -- Add LGA name
        pivoted_df['orgUnit'] = pivoted_df['orgUnit'].map(orgunit_to_name)    # -- Replace orgUnit ID with name
        pivoted_df['Cluster'] = pivoted_df['LGA'].map(cluster)                # -- Add cluster mapping

        # -- Step 12.10: Rename columns with dimension or data element names
        if url_name in ["Report Rate Facility", 'Report_Rate_LGA']:           # -- Check if URL is a report rate type
            rename_dict = {
                **{col: dataelement_to_name.get(col, col) for col in pivoted_df.columns if col in dataelement_to_name},
                'period': 'ReportPeriod',
                'orgUnit': 'FacilityName'
            }                                                                 # -- Create renaming dictionary
        elif url_name in ['ART MSF_tb screening', 'HTS MSF_hivst approach', 'PMTCT MSF_sdp', 
                          'PMTCT MSF_sdp_pos', 'PMTCT MSF_sd<72_in-outside', 'PMTCT MSF_sd>72_in-outside']:  # -- Check for special URLs
            rename_dict = {
                **{col: items.get(col, {}).get('name', col) for col in pivoted_df.columns if col in items},
                'period': 'ReportPeriod',
                'orgUnit': 'FacilityName'
            }                                                                 # -- Use items for dimension names
        else:
            rename_dict = {
                **{col: dataelement_to_description.get(col, col) for col in pivoted_df.columns if col in dataelement_to_description},
                'period': 'ReportPeriod',
                'orgUnit': 'FacilityName'
            }                                                                 # -- Use descriptions for data elements
        pivoted_df.rename(columns=rename_dict, inplace=True)                  # -- Apply renaming

        # -- Step 12.11: Shorten FacilityName
        pivoted_df['FacilityName'] = pivoted_df['FacilityName'].apply(
            lambda x: x[:34] + '...' if isinstance(x, str) and len(x) > 33 else x
        )                                                            # -- Truncate FacilityName to 34 chars if >33

        # -- Step 12.12: Handle NaN values
        if url_name in ["Report Rate Facility", 'Report_Rate_LGA']:           # -- Check if URL is a report rate type
            pivoted_df.fillna('', inplace=True)                               # -- Replace NaN with empty string
        else:                                                                 # -- For other URLs
            pivoted_df.fillna(0, inplace=True)                                # -- Replace NaN with 0

        # -- Step 12.13: Finalize DataFrame
        pivoted_df = pivoted_df.reset_index(drop=True)                        # -- Reset index and drop it
        processed_data[url_name] = pivoted_df                                 # -- Store processed DataFrame
        success_count += 1                                                    # -- Increment success counter
        print(f"- {url_name}")                                                # -- Print URL name as processed

    # -- Step 13: Display processing summary
    global report_period_display                                              # -- Declare global variable
    report_period_display = (
        f"✔️ Data fetched successfully!\nReport extracts: ({success_count}/{total_urls})\n"
        f"Extraction period: {start_display} - {end_display}\nWorkbook variables & functions loaded"
    )                                                                     # -- Format summary message
    if processed_data:                                                    # -- Check if any data was processed
        print(separator_line)                                             # -- Print separator line
        print(report_period_display)                                      # -- Print success message
        print(separator_line)                                             # -- Print separator line
    else:                                                                 # -- If no data was processed
        print(separator_line)                                             # -- Print separator line
        print(f"⦸ Failed:\nReport extracts: (0/{total_urls})\nIHVN DHIS2 login credentials invalid") # -- Print failure message
        print(separator_line)                                             # -- Print separator line

    # -- Step 14: Return processed data
    return processed_data                                                 # -- Return dictionary of processed DataFrames

#### - Function: Get data

In [ ]:
# -- Global variable to store DHIS2_data
def fetch_dhis2_data_interactive_jupyter_mode():
    """
    Interactive function to collect user inputs and fetch/process DHIS2 data in a Jupyter notebook.
    
    Args:
        None
    
    Returns:
        dict: DHIS2_data fetched from the DHIS2 server (accessible globally after submission)
    """
    global DHIS2_data, load_variables, load_functions  # -- Declare global varible and function to modify it within handlers
    
    # -- Step 1: Define constants
    separator_line = '-' * 43  # -- Define a static separator line of 43 dashes for formatting
    output = widgets.Output()  # -- Output area for displaying results

    # -- Step 2: Create widgets for collecting credentials
    username_input = widgets.Text(description="Username:", placeholder="Enter IHVN DHIS2 username")
    password_input = widgets.Password(description="Passkey:", placeholder="Enter IHVN DHIS2 password")
    submit_credentials = widgets.Button(description="Submit")
    credentials_box = widgets.VBox([username_input, password_input, submit_credentials])

    # -- Step 3: Create widgets for collecting periods
    start_period_input = widgets.Text(description="Start Period:", placeholder="Enter report period (YYYYMM, e.g., 202501)")
    end_period_input = widgets.Text(description="End Period:", placeholder="Enter report period (YYYYMM, e.g., 202512)")
    submit_periods = widgets.Button(description="Submit")
    periods_box = widgets.VBox([start_period_input, end_period_input, submit_periods])

    # -- Step 4: Store collected inputs
    credentials = [None, None]  # -- To store username and password
    periods = [None, None]      # -- To store start_period and end_period

    # -- Step 5: Define formatting functions
    def format_credentials(username, password):
        """Format login credentials for display with masked password."""
        username_line = f"{'Username: ':<{43 - len(username)}}{username}"
        password_line = f"{'Passkey: ':<{43 - len(password)}}{'*' * len(password)}"
        return f"{username_line}\n{password_line}"

    def format_report_period(start_period, end_period):
        """Format report period dates for display."""
        start_period_line = f"{'Period Start Date: ':<{43 - len(start_period)}}{start_period}"
        end_period_line = f"{'Period End Date: ':<{43 - len(end_period)}}{end_period}"
        return f"{start_period_line}\n{end_period_line}"

    def display_information(credentials_display, report_display):
        """Display all collected information in a formatted way."""
        with output:
            clear_output()
            print("Enter IHVN DHIS2 login credentials:")
            print(separator_line)
            print(credentials_display)
            print(separator_line)
            print()
            print('Enter report period (YYYYMM, e.g., 202501):')
            print(separator_line)
            print(report_display)
            print(separator_line)
            print()

    # -- Step 6: Define button handlers
    def on_submit_credentials(b):
        """Handle submission of credentials."""
        credentials[0] = username_input.value
        credentials[1] = password_input.value
        with output:
            clear_output()
            print("Enter IHVN DHIS2 login credentials:")
            print(separator_line)
            print(format_credentials(credentials[0], credentials[1]))
            print(separator_line)
            print()
            print('Enter report period (YYYYMM, e.g., 202501):')
            print(separator_line)
            display(periods_box)

    def on_submit_periods(b):
        """Handle submission of periods and fetch data."""
        global DHIS2_data
        periods[0] = start_period_input.value
        periods[1] = end_period_input.value

        # Display formatted info
        credentials_display = format_credentials(credentials[0], credentials[1])
        report_display = format_report_period(periods[0], periods[1])
        display_information(credentials_display, report_display)

        # -- Fetch and process DHIS2 data (placeholder)
        with output:
            DHIS2_data = fetch_and_process_DHIS2_data(credentials[0], credentials[1], periods[0], periods[1])
            if DHIS2_data:
                load_variables()
                load_functions()

            # -- Optional LGA Filter Widget Setup
            if "Report Rate LGA" in DHIS2_data and "LGA" in DHIS2_data["Report Rate LGA"].columns:
                available_lgas = sorted(DHIS2_data["Report Rate LGA"]["LGA"].dropna().unique())

                lga_filter_widget = widgets.SelectMultiple(
                    options=available_lgas,
                    description="Select LGA:",
                    #layout=widgets.Layout(width='300px', height='150px'),
                    rows=6
                )

                apply_filter_button = widgets.Button(description="Apply report level")

                def on_apply_filter_clicked(b):
                    selected_lgas = list(lga_filter_widget.value)
                    with output:
                        if not selected_lgas:
                            print("✔️ State level data ready")
                        else:
                            for key, df in DHIS2_data.items():
                                if isinstance(df, pd.DataFrame) and "LGA" in df.columns:
                                    DHIS2_data[key] = df[df["LGA"].isin(selected_lgas)].copy()
                            print(f"✔️ LGA level data ready for {selected_lgas}")

                apply_filter_button.on_click(on_apply_filter_clicked)

                print(f"\nOptional: Select report level - LGA")
                print(separator_line)
                display(widgets.VBox([
                    lga_filter_widget,
                    apply_filter_button
                ]))
                print(separator_line)
                
    # -- Step 7: Link buttons to handlers
    submit_credentials.on_click(on_submit_credentials)
    submit_periods.on_click(on_submit_periods)

    # -- Step 8: Display the initial interface
    with output:
        print("Enter IHVN DHIS2 login credentials:")
        print(separator_line)
        display(credentials_box)
    display(output)

#### - Functions: Variables

In [ ]:
def load_variables():
    """
    Defines variables for the notebook and assigns them as global.
    """
    
    # Declare all variables as global
    global file_path, report_name, report_name_rate, report_name_outlier, report_name_period
    global report_name_period_name, report_period_name_folder, sub_folder_image_file, sub_folder_doc_file
    global sub_folder2_image_file_report_rate, sub_folder2_image_file_msf_outlier
    global doc_file_report_rate_xlsx, doc_file_msf_outlier_docx, doc_file_msf_outlier_xlsx
    global highlight_red_list, MSF_hierarchy, MSF_report_rate_columns    

    try:
        # -- Step 1: Define main report export path
        try:
            file_path = r'C:\Users\HP\Desktop\ANSO\CQI\python\report\dhis2\msf\ihvn'
        except Exception as e:
            print(f"⦸ Error defining file_path: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 2: Create report name, dynamic period, and joined report period name
        try:
            report_name = "anso msf report"
            report_name_rate = "anso msf report rate"
            report_name_outlier = "anso msf outlier"
            report_name_period = DHIS2_data["Report Rate Facility"].ReportPeriod.iloc[0]
            report_name_period_name = f"{report_name_period} {report_name}"
        except Exception as e:
            print(f"⦸ Error defining report names or accessing DHIS2_data: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 3: Create report period folder
        try:
            report_period_name_folder = os.path.join(file_path, f"{report_name_period_name}")
        except Exception as e:
            print(f"⦸ Error creating report_period_name_folder: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 4: Define folders for storing reports
        try:
            sub_folder_image_file = os.path.join(report_period_name_folder, "image file")
            sub_folder_doc_file = os.path.join(report_period_name_folder, "document file")
        except Exception as e:
            print(f"⦸ Error defining sub_folder_image_file or sub_folder_doc_file: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 4.1: Add subfolders for specific report types
        try:
            sub_folder2_image_file_report_rate = os.path.join(sub_folder_image_file, f"{report_name_rate}")
            sub_folder2_image_file_msf_outlier = os.path.join(sub_folder_image_file, f"{report_name_outlier}")
        except Exception as e:
            print(f"⦸ Error defining sub_folder2_image_file_report_rate or sub_folder2_image_file_msf_outlier: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 5: Create folders if they do not exist
        try:
            os.makedirs(sub_folder_image_file, exist_ok=True)
            os.makedirs(sub_folder_doc_file, exist_ok=True)
            os.makedirs(sub_folder2_image_file_report_rate, exist_ok=True)
            os.makedirs(sub_folder2_image_file_msf_outlier, exist_ok=True)
        except Exception as e:
            print(f"⦸ Error creating directories: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 6: Define report document file paths
        try:
            doc_file_report_rate_xlsx = os.path.join(sub_folder_doc_file, f"{report_name_period} {report_name_rate}.xlsx")
            doc_file_msf_outlier_docx = os.path.join(sub_folder_doc_file, f"{report_name_period_name}.docx")
            doc_file_msf_outlier_xlsx = os.path.join(sub_folder_doc_file, f"{report_name_period_name}.xlsx")
        except Exception as e:
            print(f"⦸ Error defining document file paths: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 7: Define list of words and phrases to keep after HTML cleaning
        try:
            highlight_red_list = [
                'Community', 'Walk-In', 'Community & Walk-In', 'subset of 4',
                'Self, Spouse, Sexual Partner, Children, Social Network, Others',
                'FSW, MSM, PWID, TG, Others', 'Testing frequency', 'Outreach',
                'Outreach-Pregnant', 'Outreach-Others', 'Excluding community testing',
                'Excluding previously known', 'IPV', 'ANC', 'L&D', '<72hrs PP',
                '<72 hrs', '>72 hrs - < 6 months', '>6 - 12 months',
                'ANC, L&D, <72hrs Post Partum', 'Facility', 'Outside Facility',
                'Within and outside the facility', 'within 72 hrs of birth',
                'between >72 hrs - <2 months of birth', 'All regimens', 'Regimen Lines',
                'MMD', 'DSD', 'excludes ART transfer-in', 'ART Addendum-2'
            ]
        except Exception as e:
            print(f"⦸ Error defining highlight_red_list: {str(e)}")
            raise  # Re-raise to trigger top-level except

        # -- Step 8: Define MSF hierarchy
        try:
            MSF_hierarchy = ['ReportPeriod', 'Cluster', 'LGA', 'FacilityName']
        except Exception as e:
            print(f"⦸ Error defining MSF_hierarchy: {str(e)}")
            raise  # Re-raise to trigger top-level except

    except Exception as e:
        print(f"⦸ Error loading variables: {str(e)}")
        # Optionally assign fallback values to globals, but here we just return
        return

#### - Function: Validations & Exports

In [ ]:
def load_functions():
    """
    Defines and assigns global functions for styling dfs and exporting them to image, Excel, and Word formats.
    """
    try:
        # -- Step 1: Declare global functions and variables
        global outlier_red_report_rate, outlier_green_report_rate, outlier_red, outlier_yellow, outlier_red_LT0, outlier_yellow_LT0, outlier_red_GT0, outlier_yellow_GT0
        global export_df_to_doc_image_excel, filter_gap_and_check_empty_df, prepare_and_convert_df, wrap_column_headers, wrap_column_headers2, widget_display_df
        global Pre_HTS_MSF_positive, Pre_MSF_positives_all

        def wrap_column_headers(df, max_width=20):                       # -- Wrap DataFrame column headers
            """
            Wraps DataFrame column headers exceeding max_width characters for better display.

            Args:
                df (pd.DataFrame): Input DataFrame.
                max_width (int): Maximum character width before wrapping. Defaults to 26.

            Returns:
                pd.DataFrame: DataFrame with wrapped column headers.
            """
            try:
                wrapped_columns = []                                     # -- Initialize list for wrapped column names
                for col in df.columns:                                   # -- Iterate over column names
                    if len(str(col)) > max_width:                        # -- Check if column name exceeds max width
                        wrapped = '\n'.join(textwrap.wrap(col, max_width, break_long_words=True)) # -- Wrap long column name
                        wrapped_columns.append(wrapped)                  # -- Add wrapped name to list
                    else:
                        wrapped_columns.append(col)                      # -- Keep short column name as is
                df.columns = wrapped_columns                             # -- Update DataFrame columns
                return df                                                # -- Return modified DataFrame
            except Exception as e:
                print(f"⦸ Error in wrap_column_headers: {str(e)}")      # -- Print error message
                return df                                                # -- Return original DataFrame on error

        # -- Function: Wrap list of column names
        def wrap_column_headers2(columns, max_width=20):                 # -- Wrap list of column names
            """
            Wraps a list of column names exceeding max_width characters.

            Args:
                columns (list): List of column names.
                max_width (int): Maximum character width before wrapping. Defaults to 26.

            Returns:
                list: List of wrapped column names.
            """
            try:
                wrapped_columns = []                                     # -- Initialize list for wrapped column names
                for col in columns:                                      # -- Iterate over column names
                    if len(str(col)) > max_width:                        # -- Check if column name exceeds max width
                        wrapped = '\n'.join(textwrap.wrap(col, max_width, break_long_words=True))
                                                                # -- Wrap long column name
                        wrapped_columns.append(wrapped)                  # -- Add wrapped name to list
                    else:
                        wrapped_columns.append(col)                      # -- Keep short column name as is
                return wrapped_columns                                   # -- Return wrapped column names
            except Exception as e:
                print(f"⦸ Error in wrap_column_headers2: {str(e)}")     # -- Print error message
                return columns                                           # -- Return original columns on error

        # -- Step 2: Define outlier_red_report_rate function
        # -- Function: Style cells with light coral for values less than 100
        def outlier_red_report_rate(val):                                 # -- Define function to style cells red
            """
            Styles cells with a light coral background and bold font for values less than 100.
            Applies a border for consistent formatting.
    
            Args:
                val: Value to evaluate (int, float, or string).
    
            Returns:
                str: CSS style string if condition met.
            """
            try:                                                        # -- Begin try block for error handling
                # -- Step 2.1: Define conditions for styling
                condition = (                                           # -- Combine all conditions for red styling
                ((isinstance(val, (int, float)) and val < 100) or       # -- Check if numeric and less than 100
                 (isinstance(val, object) and val != '100' and val != ''))  # -- Check if string, not '100', and not empty
                or                                                      # -- OR condition for additional cases
                (not (isinstance(val, (int, float)) and val < 100) and  # -- Check if not numeric less than 100
                 (isinstance(val, (int, float)) and val != 0))          # -- AND numeric not equal to 0
            )  # -- Check if numeric and less than 100
                # -- Step 2.2: Apply styling if condition is met
                if condition:                                           # -- Evaluate combined condition
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'  # -- Return red styling for matching values
                return None                                             # -- Return None if no styling applies
            except Exception as e:                                      # -- Catch exceptions in outlier_red_report_rate definition
                print(f"⦸ Error defining outlier_red_report_rate: {str(e)}")        # -- Print error message
                return None                                             # -- Return None on error
        
        # -- Step 3: Define outlier_green_report_rate function
        # -- Function: Style cells with light green for values equal to 100
        try:
            def outlier_green_report_rate(val):
                """
                Styles cells with a light green background and bold font for values equal to 100.
                Applies a border for consistent formatting.
                
                Args:
                    val: Value to evaluate (int, float, or string).
                
                Returns:
                    str: CSS style string if condition met, None otherwise.
                """
                if (isinstance(val, (int, float)) and val == 100) or (isinstance(val, object) and val == '100' and val != ''):
                    return 'background-color: lightgreen; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_green_report_rate: {str(e)}") # -- Print error message for outlier_green_report_rate
            return None                                                     # -- Fallback to a no-op function
        
        # -- Function: Style cells with light red for values equal is not 0
        try:
            def outlier_red(val):
                """
                Styles cells with a light red background and bold font for values is not 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val != 0):
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_red: {str(e)}")               # -- Print error message for outlier_red
            return None                                                     # -- Fallback to a no-op function
        
        # -- Function: Style cells with light yellow for values is not 0
        try:
            def outlier_yellow(val):
                """
                Styles cells with a light yellow background and bold font for values is not 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val != 0):
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_yellow: {str(e)}")               # -- Print error message for outlier_yellow
            return None                                                        # -- Fallback to a no-op function
        
        # -- Function: Style cells with light red for values is less than 0
        try:
            def outlier_red_LT0(val):
                """
                Styles cells with a light red background and bold font for values less than 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val < 0):
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_red_LT0: {str(e)}")               # -- Print error message for outlier_red_LT0
            return None                                                         # -- Fallback to a no-op function
        
        # -- Function: Style cells with light yellow for values is less than 0
        try:
            def outlier_yellow_LT0(val):
                """
                Styles cells with a light yellow background and bold font for values less than 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val < 0):
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_yellow_LT0: {str(e)}")               # -- Print error message for outlier_yellow_LT0
            return None                                                            # -- Fallback to a no-op function
        
        # -- Function: Style cells with light red for values is greater than 0
        try:
            def outlier_red_GT0(val):
                """
                Styles cells with a light red background and bold font for values greater than 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val > 0):
                    return 'background-color: lightcoral; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_red_GT0: {str(e)}")               # -- Print error message for outlier_red_GT0
            return None                                                         # -- Fallback to a no-op function
        
        # -- Function: Style cells with light yellow for values is greater than 0
        try:
            def outlier_yellow_GT0(val):
                """
                Styles cells with a light yellow background and bold font for values gretar than 0. Applies a border for consistent formatting.
                """
                if (isinstance(val, (int, float)) and val > 0):
                    return 'background-color: #fff59d; font-weight: normal; border-bottom: 0.01px solid #f3f3f3;'
                return None
        except Exception as e:
            print(f"⦸ Error defining outlier_yellow_GT0: {str(e)}")               # -- Print error message for outlier_yellow_GT0
            return None                                                            # -- Fallback to a no-op function

        # -- Step 4: Define export_df_to_doc_image_excel function
        try:
            def export_df_to_doc_image_excel(
                report_name=None,
                df_style=None,
                img_file_name=None,
                img_file_path=None,
                doc_description=None,
                doc_indicators_to_italicize=None,
                doc_indicators_to_underline=None,
                doc_file_path=doc_file_msf_outlier_docx,
                xlm_file_path=None,
                xlm_sheet_name=None,
                highlight_red_list=highlight_red_list
            ):
                """
                Exports a df to an image, Excel file, and Word document with optional description for the document.
                (Docstring truncated for brevity; see original for full details.)
                """
                try:
                    # -- Step 4.1: Process image export
                    if all([df_style is not None, img_file_name is not None, img_file_path is not None]):  # -- Check if all image params are provided
                        # Apply basic styling that Matplotlib can interpret
                        styled_df = df_style.set_properties(**{
                            'text-align': 'right',
                            'font-size': '12pt',  
                            #'border': '1px solid black'
                        })
                        image_path = os.path.join(img_file_path, img_file_name)
                        
                        dfi.export(
                            styled_df, 
                            image_path, 
                            table_conversion='matplotlib',  # -- Use matplotlib for rendering
                            max_rows=-1,                    # -- Render all rows (no truncation)
                            max_cols=-1,                    # -- Render all columns
                            fontsize=12,                    # -- Reduce font size to fit more rows
                            dpi=300                         # -- Increase DPI for better quality
                        )

                    # -- Step 4.2: Process Excel export
                    if all([df_style is not None, xlm_file_path is not None, xlm_sheet_name is not None]):  # -- Check if all Excel params are provided
                        xlm_sheet_name = xlm_sheet_name[:31]            # -- Truncate sheet name to 31 characters
                        mode = 'a' if os.path.exists(xlm_file_path) else 'w'  # -- Set mode: append if file exists, write if not
                        with pd.ExcelWriter(xlm_file_path, engine='openpyxl', mode=mode, if_sheet_exists='replace' if mode == 'a' else None) as writer:  # -- Open Excel writer
                            df_style.to_excel(writer, sheet_name=xlm_sheet_name, index=False)  # -- Write styled DataFrame to Excel

                        wb = load_workbook(xlm_file_path)               # -- Load workbook for further formatting
                        ws = wb[xlm_sheet_name]                         # -- Select worksheet by truncated name

                        for cell in ws[1]:                              # -- Process header row for formatting
                            if cell.value and isinstance(cell.value, str):  # -- Check if cell has string value
                                protected_map = {}                      # -- Initialize map for protected phrases
                                for phrase in highlight_red_list or []: # -- Iterate over phrases to highlight
                                    token = f"@@PROTECT_{abs(hash(phrase))}@@"  # -- Create unique token
                                    protected_map[token] = phrase       # -- Map token to phrase
                                    cell.value = cell.value.replace(phrase, token)  # -- Replace phrase with token
                                cell.value = re.sub(r"<.*?>", "", cell.value)  # -- Remove HTML tags
                                for token, phrase in protected_map.items():  # -- Restore protected phrases
                                    cell.value = cell.value.replace(token, phrase)  # -- Replace token with phrase
                                cell.value = cell.value.strip()         # -- Strip whitespace

                        font_style = Font(name='Calibri', size=8)       # -- Define font style for cells
                        header_font = Font(name='Calibri', size=8, bold=True)  # -- Define font style for headers
                        header_alignment = Alignment(horizontal="left", vertical="bottom", wrap_text=True)  # -- Define header alignment

                        for row in ws.iter_rows():                      # -- Apply font style to all cells
                            for cell in row:
                                cell.font = font_style

                        for cell in ws[1]:                              # -- Apply header formatting
                            cell.alignment = header_alignment
                            cell.font = header_font

                        for col in ws.iter_cols(min_col=1, max_col=4):  # -- Adjust column widths
                            max_length = max((len(str(cell.value)) if cell.value else 0) for cell in col)  # -- Find max length
                            ws.column_dimensions[col[0].column_letter].width = max_length  # -- Set column width

                        ws.auto_filter.ref = ws.dimensions              # -- Enable auto-filter for sheet
                        wb.save(xlm_file_path)                          # -- Save workbook

                    # -- Step 4.3: Create or append to Word document
                    if all([doc_file_path is not None, image_path is not None, doc_indicators_to_italicize is not None, doc_indicators_to_underline is not None]):  # -- Check if all doc params are provided
                        if os.path.exists(doc_file_path):               # -- Check if document exists
                            doc = Document(doc_file_path)               # -- Load existing document
                        else:
                            doc = Document()                            # -- Create new document

                        style = doc.styles['Normal']                    # -- Set default style
                        style.font.name = 'Calibri'                     # -- Set font to Calibri
                        style.font.size = Pt(9.5)                       # -- Set font size

                        for section in doc.sections:                    # -- Configure section margins
                            section.left_margin = Inches(0.5)
                            section.right_margin = Inches(0.5)
                            section.top_margin = Inches(1)
                            section.bottom_margin = Inches(1)

                        if doc_description:                             # -- Add description if provided
                            title_paragraph = doc.add_heading(report_name, level=2)  # -- Add report name as heading
                            title_run = title_paragraph.runs[0]         # -- Get title run
                            title_run.font.size = Pt(10)                # -- Set title font size
                            title_run.font.color.rgb = RGBColor(0, 0, 0)  # -- Set title color

                            paragraph = doc.add_paragraph()             # -- Add paragraph for description
                            paragraph.paragraph_format.space_after = Pt(0)  # -- Remove space after paragraph

                            phrases_to_bold = [                         # -- Define phrases to bold
                                "REPORT ONLY 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY HEI ARVs FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID SAMPLE COLLECTION FOR 2025 LIVE BIRTHS BY PPW",
                                "REPORT ONLY EID PCR RESULTS FOR 2025 LIVE BIRTHS BY PPW",
                                "Report Name:", "should not be greater than",
                                "should not be lesser than", "should not be equal to",
                                "should be greater than", "should be lesser than",
                                "should be equal to", "plus", "Note", "OR"
                            ]
                            all_phrases = phrases_to_bold + (doc_indicators_to_italicize or []) + (doc_indicators_to_underline or [])  # -- Combine all phrases
                            pattern = r'|'.join(re.escape(phrase) for phrase in all_phrases)  # -- Create regex pattern
                            matches = list(re.finditer(pattern, doc_description))  # -- Find matches in description

                            last_index = 0                              # -- Track last processed index
                            for match in matches:                       # -- Process each match
                                start, end = match.start(), match.end() # -- Get match boundaries
                                paragraph.add_run(doc_description[last_index:start])  # -- Add text before match
                                run = paragraph.add_run(doc_description[start:end])  # -- Add matched text
                                if match.group(0) in phrases_to_bold:   # -- Apply bold if in bold list
                                    run.bold = True
                                if match.group(0) in doc_indicators_to_italicize:  # -- Apply italic if in italicize list
                                    run.italic = True
                                if match.group(0) in doc_indicators_to_underline:  # -- Apply underline if in underline list
                                    run.underline = True
                                last_index = end                            # -- Update last index

                            paragraph.add_run(doc_description[last_index:])  # -- Add remaining text

                        doc.add_picture(image_path, width=Inches(7))    # -- Add image to document

                        section = doc.sections[-1]                      # -- Get last section for footer
                        footer = section.footer.paragraphs[0]           # -- Access footer paragraph
                        footer.text = "This is an auto-generated report. Ensure all data is reviewed before any update is made."  # -- Set footer text
                        footer.runs[0].font.size = Pt(7.5)             # -- Set footer font size
                        footer.runs[0].font.color.rgb = RGBColor(100, 100, 100)  # -- Set footer color

                        doc.save(doc_file_path)                         # -- Save document

                    # -- Step 4.4: Generate and print success messages
                    if all([report_name, img_file_name, img_file_path, xlm_file_path, xlm_sheet_name]):  # -- Check if all params for success message are provided
                        img_file_path_name = os.path.basename(img_file_path)  # -- Get base image path name
                        xlm_file_path_name = os.path.basename(xlm_file_path)  # -- Get base Excel path name
                        image_success_print = rf"IMG: '{img_file_name}' in {img_file_path_name}"  # -- Format image success message
                        excel_success_print = rf"XLS: '{xlm_sheet_name}' in {xlm_file_path_name}"  # -- Format Excel success message
                        
                        messages = [image_success_print, excel_success_print]  # -- Initialize success messages list
                        if doc_description:                             # -- Check if document description exists
                            doc_file_path_name = os.path.basename(doc_file_path)  # -- Get base document path name
                            doc_success_print = rf"DOC: '{report_name}' in {doc_file_path_name}"  # -- Format document success message
                            messages.append(doc_success_print)          # -- Add document message to list

                        separator_line = '-' * max(len(msg) for msg in messages)  # -- Create separator line based on longest message

                        print(f"✔️ {report_name}")                      # -- Print report name with checkmark
                        print(separator_line)                           # -- Print separator line
                        print('\n'.join(messages))                      # -- Print success messages
                        print(separator_line)                           # -- Print separator line

                    return image_path                                   # -- Return image path

                except Exception as e:
                    print(f"⦸ Error in export_df_to_doc_image_excel: {str(e)}")  # -- Print error message
                    return None                                         # -- Return None on error
        except Exception as e:
            print(f"⦸ Error defining export_df_to_doc_image_excel: {str(e)}")  # -- Print error message for function definition
            export_df_to_doc_image_excel = lambda *args, **kwargs: None  # -- Fallback to a no-op function

        # -- Step 5: Define prepare_and_convert_df function
        def prepare_and_convert_df(DHIS2_data_key=None, hierarchy_columns=None, data_columns=None):
            """
            Prepare and convert a DataFrame from DHIS2_data with available columns, applying sorting,
            default values for missing data columns, and type conversions.

            Args:
                DHIS2_data_key (str): The key to look up the DHIS2 dataset.
                hierarchy_columns (List[str] or None): List of hierarchy columns to include.
                data_columns (List[str]): List of desired data columns.

            Returns:
                Optional[pd.DataFrame]: Prepared DataFrame or None if error occurs.
            """
            try:
                if DHIS2_data_key not in DHIS2_data:
                    print(f"⦸ Error: '{DHIS2_data_key}' not found in DHIS2_data. Report not processed.")
                    return None

                df_raw = DHIS2_data[DHIS2_data_key]

                if hierarchy_columns is None:
                    hierarchy_columns = []

                # Find available and missing columns
                available_columns = [col for col in data_columns if col in df_raw.columns]
                missing_columns = [col for col in data_columns if col not in df_raw.columns]

                # Print warning for each missing column
                for col in missing_columns:
                    print(f"✋🏿 Warning: Column '{col}' not found in '{DHIS2_data_key}'.")

                # Print summary warning if no columns are found
                if not available_columns:
                    print(f"✋🏿 Warning: None of the requested columns found in '{DHIS2_data_key}'. Missing columns: {missing_columns}")

                # Proceed with available columns and hierarchy
                df = df_raw[hierarchy_columns + available_columns].copy()
                df.sort_values(by=hierarchy_columns, inplace=True, ignore_index=True)

                # Convert available columns to numeric
                for col in available_columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

                # Add missing columns with default value 0
                for col in data_columns:
                    if col not in df.columns:
                        df[col] = 0

                # Reorder columns to match input order
                df = df[hierarchy_columns + data_columns]

                return df

            except Exception as e:
                print(f"⦸ Error preparing and converting df for '{DHIS2_data_key}': {str(e)}")
                return None
        
        # -- Step 6: Define filter_gap_and_check_empty_df function
        # -- Function: Filter gap and check empty df
        try:
            def filter_gap_and_check_empty_df(
                df=None, 
                msg=None, 
                opNonZero=None, 
                opNeg=None, 
                opPos=None, 
                opNonPos=None, 
                opNonNeg=None, 
                opZero=None, 
                opLT100=None):
                """
                Filters a df based on column-specific conditions and handles empty results.
                (Docstring truncated for brevity; see original for full details.)
                """
                try:
                    if df is None or df.empty:                          # -- Check if input DataFrame is invalid
                        raise ValueError("Input DataFrame is None or empty")   # -- Raise error if None or empty
                    if not msg:                                         # -- Check if message is provided
                        raise ValueError("No gap message provided for empty result")  # -- Raise error if missing

                    operator_map = {                                    # -- Define operator mapping for conditions
                        'opNonZero': lambda x: x != 0,                  # -- Non-zero condition
                        'opNeg': lambda x: x < 0,                       # -- Negative condition
                        'opPos': lambda x: x > 0,                       # -- Positive condition
                        'opZero': lambda x: x == 0,                     # -- Zero condition
                        'opLT100': lambda x: x < 100                    # -- Less than 100 condition
                    }

                    conditions = []                                     # -- Initialize conditions list
                    for arg, cols in {                                  # -- Iterate over operator arguments
                        'opNonZero': opNonZero, 'opNeg': opNeg, 'opPos': opPos, 
                        'opNonPos': opNonPos, 'opNonNeg': opNonNeg, 'opZero': opZero, 'opLT100': opLT100
                    }.items():
                        if cols:                                        # -- Check if columns are provided for operator
                            for col in cols:                            # -- Iterate over columns
                                if col not in df.columns:               # -- Check if column exists
                                    raise ValueError(f"Column '{col}' not found in {df}")  # -- Raise error if missing
                                numeric_series = pd.to_numeric(df[col], errors='coerce').fillna(0)  # -- Convert to numeric
                                conditions.append(operator_map[arg](numeric_series))  # -- Apply operator condition

                    if not conditions:                                  # -- Check if any conditions were added
                        print("No filtering conditions provided")       # -- Print warning
                        return df                                       # -- Return original DataFrame

                    combined_condition = pd.DataFrame(conditions).T.any(axis=1)  # -- Combine conditions with OR logic
                    df_filtered = df[combined_condition]            # -- Filter DataFrame

                    if df_filtered.empty:                           # -- Check if filtered DataFrame is empty
                        print("✋🏿Checked:")                        # -- Print check message
                        print("-" * len(msg))                       # -- Print separator line
                        print(msg)                                  # -- Print empty result message
                        print("-" * len(msg))                       # -- Print separator line
                        return None                                 # -- Return None for empty result

                    return df_filtered                              # -- Return filtered DataFrame

                except Exception as e:
                    print(f"⦸ Error filtering gaps and checking empty df: {str(e)}")  # -- Print error message
                    return None                                     # -- Return None on error
        except Exception as e:
            print(f"⦸ Error defining filter_gap_and_check_empty_df: {str(e)}")  # -- Print error message for function definition
            filter_gap_and_check_empty_df = lambda *args, **kwargs: None  # -- Fallback to a no-op function

         # -- Step 7: Constants Initialization for Positive Data Processing
        # -- Define column lists for HTS, AGYW, PMTCT, and KP datasets
        HTS_cols = {
            "positive": [
                "Number of people who tested HIV positive and received results (Inpatient)",  # -- Inpatient positive results
                "Number of people who tested HIV positive and received results (Outpatient)", # -- Outpatient positive results
                "Number of people who tested HIV positive and received results (Standalone)", # -- Standalone positive results
                "Number of people who tested HIV positive and received results (Community)"   # -- Community positive results
            ],
            "known_positive": [
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Inpatient)",  # -- Known positive inpatient
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Outpatient)", # -- Known positive outpatient
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Standalone)", # -- Known positive standalone
                "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)"   # -- Known positive community
            ]
        }

        AGYW_cols = [
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",  # -- AGYW community positive
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"     # -- AGYW walk-in positive
        ]

        PMTCT_col = ["Number of pregnant women tested HIV positive"]  # -- PMTCT positive results

        KP_cols = [
            "HTS-3a Number of MSM that have received an HIV test during the reporting period in KP-specific programs and received HIV Positive results",                             # -- MSM positive results
            "HTS-3b Number of TG that have received an HIV test during the reporting period in KP-specific programs and HIV positive results",                                       # -- TG positive results
            "HTS-3c Number of sex workers that have received an HIV test during the reporting period in KP-specific programs and received HIV-positive results",                     # -- Sex workers positive
            "HTS-3d Number of people who inject drugs (PWID) that have received an HIV test during the reporting period in KP-specific programs and received HIV positive results",  # -- PWID positive
            "HTS-3e Number of other vulnerable populations (OVP) that have received an HIV test during the reporting period and received HIV-positive results",                      # -- OVP positive
            "HTS-3f Number of people in prisons and other closed settings that have received an HIV test during the reporting period and received HIV-positive results"              # -- Prison positive
        ]

        # -- Step 8: Prepare and process HTS data
        Pre_HTS_MSF_positive = prepare_and_convert_df("HTS MSF", MSF_hierarchy, HTS_cols["positive"] + HTS_cols["known_positive"])           # -- Prepare HTS DataFrame
        Pre_HTS_MSF_positive["HTS total tested - positive"] = Pre_HTS_MSF_positive[HTS_cols["positive"]].sum(axis=1)                         # -- Calculate total positive
        Pre_HTS_MSF_positive["HTS total tested - previously known positive"] = Pre_HTS_MSF_positive[HTS_cols["known_positive"]].sum(axis=1)  # -- Calculate known positive
        Pre_HTS_MSF_positive["HTS total tested - new positive (excluding previously known)"] = np.where(
            Pre_HTS_MSF_positive["HTS total tested - previously known positive"] > 0,                                                        # -- Check if known positive exists
            Pre_HTS_MSF_positive["HTS total tested - positive"] - Pre_HTS_MSF_positive["HTS total tested - previously known positive"],      # -- Calculate new positive
            Pre_HTS_MSF_positive["HTS total tested - positive"]                                                                              # -- If not, keep total positive
        )

        # -- Step 9: Prepare AGYW data
        Pre_AGYW_MSF_positive = prepare_and_convert_df("AGYW MSF", MSF_hierarchy, AGYW_cols)                      # -- Prepare AGYW DataFrame
        Pre_AGYW_MSF_positive["AGYW total tested - new positive"] = Pre_AGYW_MSF_positive[AGYW_cols].sum(axis=1)  # -- Calculate total AGYW positive

        # -- Step 10: Prepare PMTCT data
        Pre_PMTCT_MSF_positive = prepare_and_convert_df("PMTCT MSF", MSF_hierarchy, PMTCT_col)                    # -- Prepare PMTCT DataFrame
        Pre_PMTCT_MSF_positive.rename(columns={PMTCT_col[0]: "PMTCT total tested - new positive"}, inplace=True)  # -- Rename PMTCT column

        # -- Step 11: Prepare KP data
        Pre_KP_MSF_positive = prepare_and_convert_df("KP Prev MSF", MSF_hierarchy, KP_cols)                    # -- Prepare KP DataFrame
        Pre_KP_MSF_positive["KP_Prev total tested - new positive"] = Pre_KP_MSF_positive[KP_cols].sum(axis=1)  # -- Calculate total KP positive

        # -- Step 12: Merge all DataFrames sequentially
        base_cols = ["ReportPeriod", "Cluster", "LGA", "FacilityName"]                                           # -- Define base columns for merging
        Pre_MSF_positives_all = DHIS2_data["Report Rate Facility"][base_cols].merge(
            Pre_HTS_MSF_positive[base_cols + ["HTS total tested - new positive (excluding previously known)"]],  # -- Merge HTS data
            on=base_cols, how="left"
        ).merge(
            Pre_AGYW_MSF_positive[base_cols + ["AGYW total tested - new positive"]],    # -- Merge AGYW data
            on=base_cols, how="left"
        ).merge(
            Pre_PMTCT_MSF_positive[base_cols + ["PMTCT total tested - new positive"]],  # -- Merge PMTCT data
            on=base_cols, how="left"
        ).merge(
            Pre_KP_MSF_positive[base_cols + ["KP_Prev total tested - new positive"]],   # -- Merge KP data
            on=base_cols, how="left"
        )

        # -- Step 13: Ensure numeric types & fill NaNs
        for col in [
            "HTS total tested - new positive (excluding previously known)",  # -- HTS new positive column
            "AGYW total tested - new positive",                             # -- AGYW new positive column
            "PMTCT total tested - new positive",                            # -- PMTCT new positive column
            "KP_Prev total tested - new positive"                           # -- KP new positive column
        ]:
            Pre_MSF_positives_all[col] = pd.to_numeric(Pre_MSF_positives_all[col], errors='coerce').fillna(0).astype(int)  # -- Convert to int, fill NaNs with 0
            Pre_MSF_positives_all["Total new positive"] = Pre_MSF_positives_all[[  # -- Sum total new positive
                "HTS total tested - new positive (excluding previously known)",
                "AGYW total tested - new positive",
                "PMTCT total tested - new positive",
                "KP_Prev total tested - new positive"
            ]].sum(axis=1)

            
        def widget_display_df(styler, widget=None):
            """
            Apply table styles to a Pandas DataFrame Styler object and display it in an Output widget.
            
            Args:
                styler (pandas.io.formats.style.Styler): The Styler object to apply styles to.
                widget (ipywidgets.Output, optional): An existing Output widget to use. If None, a new one is created.
            
            Returns:
                ipywidgets.Output: The widget containing the styled DataFrame.
            """
            # Apply table styles
            styled_styler = styler.set_table_styles([
                # Table styling
                {'selector': 'table', 
                'props': [('background-color', 'white'), 
                        ('border-collapse', 'collapse')]},
                # Header styling
                {'selector': 'th', 
                'props': [('background-color', '#f0f0f0'), 
                        ('text-align', 'right'), 
                        ('padding', '5px')]},
                # Cell styling
                {'selector': 'td', 
                'props': [('text-align', 'right'), 
                        ('padding', '5px')]},
                # Hover effect for rows (background color)
                {'selector': 'tr:hover', 
                'props': [('background-color', '#e0f7fa')]},
                # Bold text for cells in hovered rows
                {'selector': 'tr:hover td', 
                'props': [('font-weight', 'bold')]}
            ])
            
            # Create or use an Output widget
            if widget is None:
                widget = widgets.Output()
            
            # Display the styled DataFrame in the widget
            with widget:
                display(HTML(styled_styler.to_html()))
            
            # Display the widget
            display(widget)
            
            return widget 

    except Exception as e:
        print(f"⦸ Error in load_functions: {str(e)}")                    # -- Print error message for load_functions

### MSF reporting rate

#### - Reporting rate: LGA

In [ ]:
# -- Define the main function to process LGA report rate gap
def process_lga_report_rate_gap(display_output=None):
    """
    Process LGA report rate gap, exporting results as image and Excel files.
    Caches the styled df and df shape for faster display in subsequent calls.
    Reprocesses if the df shape changes.
    
    Args:
        display_output (bool, optional): If True, displays the styled df for LGAs with gap.
            Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        MSF_report_rate_columns = [                           # -- Define list of MSF report rate columns
            'AGYW Monthly Summary Form - Reporting rate',
            'ART MSF - Reporting rate',
            'Care & Support MSF - Reporting rate',
            'HTS Summary Form - Reporting rate',
            'NSP Summary Form - Reporting rate',
            'PMTCT MSF - Reporting rate',
            'Prevention Summary Form - Reporting rate'
        ]
        MSF_report_rate_msg = "No ANSO report rate gap found"  # -- Define message for no gaps
        report_name = "ANSO MSF report rate"                  # -- Define report name

        # -- Step 2: Prepare data
        global df_Report_Rate_LGA
        df_Report_Rate_LGA = prepare_and_convert_df(          # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='Report_Rate_LGA',                 # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=MSF_report_rate_columns              # -- Include specified report rate columns
        )
        if df_Report_Rate_LGA is None:                        # -- Check if data preparation failed or DataFrame is empty
            return                                            # -- Exit function if no data
        
        # Drop the 'FacilityName' column if it exists
        df_Report_Rate_LGA = df_Report_Rate_LGA.drop(columns='FacilityName')  # -- Drop the column
        
        wrap_column_headers(df_Report_Rate_LGA)
        MSF_report_rate_columns2 = wrap_column_headers2(MSF_report_rate_columns)

        # -- Step 3: Set export variables
        report_month = df_Report_Rate_LGA['ReportPeriod'].iloc[0]  # -- Extract report month from DataFrame
        report_sheet_name = "All LGAs"                        # -- Define Excel sheet name
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name

        # -- Step 4: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_lga_report_rate_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_Report_Rate_LGA.shape      # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print("-" * len(cached_display_name))     # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print("-" * len(cached_display_name))     # -- Print separator line
                    display(process_lga_report_rate_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 5: Filter for gaps
        df_Report_Rate_LGA_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_Report_Rate_LGA,                            # -- Input DataFrame
            msg=MSF_report_rate_msg,                          # -- Message for empty result
            opNonZero=MSF_report_rate_columns2,                                   # -- No non-zero filter
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=MSF_report_rate_columns2                  # -- Filter for values less than 100
        )
        if df_Report_Rate_LGA_gap is None:                    # -- Check if no gaps found
            if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
                del process_lga_report_rate_gap.cached_style  # -- Clear cached style
            if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_lga_report_rate_gap.cached_shape  # -- Clear cached shape
            return                                            # -- Exit function

        # -- Step 6: Style the DataFrame
        df_Report_Rate_LGA_style = (                          # -- Apply styling to filtered DataFrame
            df_Report_Rate_LGA_gap.style                      # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red_report_rate, subset=MSF_report_rate_columns2)  # -- Highlight outliers in red for report rate columns
            .map(outlier_green_report_rate)                                # -- Apply green outlier styling (assumed general application)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_lga_report_rate_gap.cached_style = df_Report_Rate_LGA_style  # -- Cache styled DataFrame
        process_lga_report_rate_gap.cached_shape = df_Report_Rate_LGA.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to image and Excel formats
            report_name=report_name,                          # -- Pass report name
            df_style=df_Report_Rate_LGA_style,                # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=sub_folder2_image_file_report_rate, # -- Pass image file path
            doc_description=None,                             # -- No document description (not used)
            doc_indicators_to_italicize=None,                 # -- No indicators to italicize (not used)
            doc_indicators_to_underline=None,                 # -- No indicators to underline (not used)
            xlm_file_path=doc_file_report_rate_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 9: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            widget_display_df(df_Report_Rate_LGA_style)                 # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing LGA report rate gap: {str(e)}")  # -- Print error message
        if hasattr(process_lga_report_rate_gap, 'cached_style'):  # -- Check if cache exists
            del process_lga_report_rate_gap.cached_style      # -- Clear cached style
        if hasattr(process_lga_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_lga_report_rate_gap.cached_shape      # -- Clear cached shape

#### - Reporting rate: Facility

In [ ]:
# -- Define the main function to process facility report rate gap
def process_facility_report_rate_gap(display_output=None):
    """
    Process facility report rate gaps for each LGA, exporting results as images and Excel files.
    Caches styled dfs for each LGA and displays them on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the styled df images for each LGA with gaps.
            Defaults to None (will treat as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        MSF_report_rate_columns = [                           # -- Define list of MSF report rate columns
            'AGYW Monthly Summary Form - Reporting rate',
            'ART MSF - Reporting rate',
            'Care & Support MSF - Reporting rate',
            'HTS Summary Form - Reporting rate',
            'NSP Summary Form - Reporting rate',
            'PMTCT MSF - Reporting rate',
            'Prevention Summary Form - Reporting rate'
        ]

        # -- Step 2: Prepare data
        df_Report_Rate_Facility = prepare_and_convert_df(     # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="Report Rate Facility",            # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=MSF_report_rate_columns              # -- Include specified report rate columns
        )
        if df_Report_Rate_Facility is None:                   # -- Check if data preparation failed or DataFrame is empty
            return                                            # -- Exit function if no data
        
        wrap_column_headers(df_Report_Rate_Facility)
        MSF_report_rate_columns2 = wrap_column_headers2(MSF_report_rate_columns)


        # -- Step 3: Check and display cached styled DataFrames
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cached styled DataFrames exist
                cached_shape = getattr(process_facility_report_rate_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_Report_Rate_Facility.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    for lga, style in process_facility_report_rate_gap.cached_styles.items():  # -- Iterate over cached styles
                        cached_display_name = f"✔️ Displaying {lga} facility report rate gap "  # -- Define display message for LGA
                        print("-" * len(cached_display_name)) # -- Print separator line
                        print(cached_display_name)            # -- Print display message
                        print("-" * len(cached_display_name)) # -- Print separator line
                        display(style)                        # -- Display cached styled DataFrame for LGA
                    return                                    # -- Exit function

        # -- Step 4: Initialize cache
        if not hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cache attribute exists
            process_facility_report_rate_gap.cached_styles = {}  # -- Initialize dictionary to store styled DataFrames per LGA

        # -- Step 5: Identify unique LGAs
        lga_list = pd.Series(df_Report_Rate_Facility['LGA'].unique())  # -- Extract unique LGA names as a Series

        # -- Step 6: Process each LGA for report rate gaps
        for current_lga in lga_list:                          # -- Iterate over each unique LGA
            # -- Step 6.1: Filter DataFrame for current LGA
            global lga_filtered, lga_filtered_style
            lga_filtered = df_Report_Rate_Facility[df_Report_Rate_Facility['LGA'] == current_lga]  # -- Filter DataFrame to current LGA

            MSF_report_rate_msg = f"No {current_lga} report rate gap found"  # -- Define message for no gaps

            # -- Step 6.2: Apply filtering for gaps
            lga_filtered_gap = filter_gap_and_check_empty_df(  # -- Filter LGA-specific subset for gaps
                df=lga_filtered,                              # -- Input LGA-filtered DataFrame
                msg=MSF_report_rate_msg,                      # -- Message for empty result
                opNonZero=MSF_report_rate_columns2,                               # -- No non-zero filter
                opNeg=None,                                   # -- No negative filter
                opPos=None,                                   # -- No positive filter
                opZero=None,                                  # -- No zero filter
                opLT100=MSF_report_rate_columns2               # -- Filter for values less than 100
            )

            if lga_filtered_gap is None:                      # -- Check if no gaps found for this LGA
                if current_lga in process_facility_report_rate_gap.cached_styles:  # -- Check if LGA is in cache
                    del process_facility_report_rate_gap.cached_styles[current_lga]  # -- Remove LGA from cache
                continue                                      # -- Skip to next LGA

            # -- Step 6.3: Style the DataFrame
            lga_filtered_style = (                            # -- Apply styling to filtered LGA DataFrame
                lga_filtered_gap.style                        # -- Start with DataFrame style object
                .hide(axis='index')                           # -- Hide index column
                .map(outlier_red_report_rate, subset=MSF_report_rate_columns2)  # -- Highlight outliers in red for report rate columns
                .map(outlier_green_report_rate)                           # -- Apply green outlier styling (assumed general application)
            )

            # -- Step 6.4: Cache the styled DataFrame for this LGA
            process_facility_report_rate_gap.cached_styles[current_lga] = lga_filtered_style  # -- Store styled DataFrame in cache

            # -- Step 6.5: Define export variables
            report_name = f"{current_lga} Facility Report Rate Gap"  # -- Define report name for current LGA
            report_image_name = f"{current_lga}.png"          # -- Define image file name for current LGA
            report_sheet_name = f"{current_lga}"              # -- Define Excel sheet name for current LGA

            # -- Step 6.6: Export results
            export_df_to_doc_image_excel(                     # -- Export LGA-specific DataFrame to image and Excel
                report_name=report_name,                      # -- Pass report name
                #df_shape=lga_filtered_gap,                     # -- Pass filtered LGA DataFrame
                df_style=lga_filtered_style,                  # -- Pass styled LGA DataFrame
                img_file_name=report_image_name,              # -- Pass image file name
                img_file_path=sub_folder2_image_file_report_rate,  # -- Pass image file path
                doc_description=None,                         # -- No document description (not used)
                doc_indicators_to_italicize=None,             # -- No indicators to italicize (not used)
                doc_indicators_to_underline=None,             # -- No indicators to underline (not used)
                xlm_file_path=doc_file_report_rate_xlsx,      # -- Pass Excel file path
                xlm_sheet_name=report_sheet_name              # -- Pass Excel sheet name
            )

            # -- Step 6.7: Optionally display styled DataFrame
            if display_output:                                # -- Check if display is requested
                widget_display_df(lga_filtered_style)                   # -- Display styled DataFrame for current LGA
            
            break

        # -- Step 7: Cache overall unfiltered DataFrame shape
        process_facility_report_rate_gap.cached_shape = df_Report_Rate_Facility.shape  # -- Cache unfiltered DataFrame shape after processing

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing facility report rate gaps: {str(e)}")  # -- Print error message
        if hasattr(process_facility_report_rate_gap, 'cached_styles'):  # -- Check if cache exists
            process_facility_report_rate_gap.cached_styles.clear()  # -- Clear cached styles dictionary
        if hasattr(process_facility_report_rate_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_facility_report_rate_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function on error

### AGYW

#### - AGYW HTS

In [ ]:
# -- Define the main function to process AGYW HTS gap
def process_AGYW_HTS_gap(display_output=None):
    """
    Process AGYW HTS gap for each LGA, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for LGAs with gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_HTS_columns = [                                  # -- Define list of AGYW HTS columns in desired order
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting period (Community)",
            "Number of AGYW reached with HIV Prevention Program - defined package of service during the reporting (Walk-In)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)"
        ]
        name = "AGYW HTS gap"                                 # -- Define general name
        AGYW_HTS_gap_columns = ['AGYW HTS gap']               # -- Define gap column name
        AGYW_HTS_msg = f"No {name}"                           # -- Define message for no gaps
        report_name = f"{name}1"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_HTS = prepare_and_convert_df(                 # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_HTS_columns                     # -- Include specified AGYW HTS columns
        )
        if df_AGYW_HTS is None:                               # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        wrap_column_headers(df_AGYW_HTS)
        AGYW_HTS_columns2 = wrap_column_headers2(AGYW_HTS_columns)

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_HTS_gap, 'cached_style'): # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_HTS_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_HTS.shape             # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_HTS_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW reached with HIV Prevention
        df_AGYW_HTS["Total AGYW reached with HIV Prevention"] = (  # -- Calculate total reached
            df_AGYW_HTS[AGYW_HTS_columns2[0]] + df_AGYW_HTS[AGYW_HTS_columns2[1]]  # -- Sum Community and Walk-In
        )
        # -- Step 4.2: Total AGYW received HIV test & know status
        df_AGYW_HTS["Total AGYW received HIV test & know status"] = (  # -- Calculate total tested
            df_AGYW_HTS[AGYW_HTS_columns2[2]] + df_AGYW_HTS[AGYW_HTS_columns2[3]]  # -- Sum Community and Walk-In
        )
        # -- Step 4.3: AGYW HTS gap
        df_AGYW_HTS[AGYW_HTS_gap_columns[0]] = np.where(      # -- Calculate gap
            df_AGYW_HTS["Total AGYW received HIV test & know status"] > df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # -- Condition for gap
            df_AGYW_HTS["Total AGYW received HIV test & know status"] - df_AGYW_HTS["Total AGYW reached with HIV Prevention"],  # -- Positive gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_HTS)                                    # -- Apply wrapping to DataFrame headers
        df_columns_wrap = wrap_column_headers2(AGYW_HTS_columns)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(AGYW_HTS_gap_columns)            # -- Wrap gap column names

        # -- Step 5: Filter and validate gaps
        df_AGYW_HTS_gap = filter_gap_and_check_empty_df(      # -- Filter DataFrame for gaps
            df=df_AGYW_HTS,                                   # -- Input DataFrame
            msg=AGYW_HTS_msg,                                 # -- Message for empty result
            opNonZero=gap_columns_wrap,                   # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        if df_AGYW_HTS_gap is None:                           # -- Check if no gaps found
            if hasattr(process_AGYW_HTS_gap, 'cached_style'): # -- Check if cache exists
                del process_AGYW_HTS_gap.cached_style         # -- Clear cached style
            if hasattr(process_AGYW_HTS_gap, 'cached_shape'): # -- Check if cached shape exists
                del process_AGYW_HTS_gap.cached_shape         # -- Clear cached shape
            return                                            # -- Exit function
        
        if df_AGYW_HTS_gap is None:                               # -- Check if data preparation failed
            return 

        # -- Step 6: Style the DataFrame
        df_AGYW_HTS_gap_style = (                             # -- Apply styling to filtered DataFrame
            df_AGYW_HTS_gap.style                             # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=gap_columns_wrap)    # -- Highlight outliers in gap column
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_HTS_gap.cached_style = df_AGYW_HTS_gap_style  # -- Cache styled DataFrame
        process_AGYW_HTS_gap.cached_shape = df_AGYW_HTS.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_HTS['ReportPeriod'].iloc[0]    # -- Extract report month from DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create description for Word document
        if (df_AGYW_HTS[AGYW_HTS_gap_columns[0]] != 0).any():  # -- Check if any gaps exist
            report_description = (                                # -- Define report description
                f"Report Name: {gap_columns_wrap[0]}"
                f"\n{AGYW_HTS_columns[2]}\nplus {AGYW_HTS_columns[3]}"
                f"\nshould not be greater than"
                f"\n{AGYW_HTS_columns[0]}\nplus {AGYW_HTS_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_style=df_AGYW_HTS_gap_style,                   # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_HTS_columns,     # -- Italicize AGYW HTS columns
            doc_indicators_to_underline=AGYW_HTS_gap_columns, # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            widget_display_df(df_AGYW_HTS_gap_style)                    # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_HTS_gap, 'cached_style'):     # -- Check if cache exists
            del process_AGYW_HTS_gap.cached_style             # -- Clear cached style
        if hasattr(process_AGYW_HTS_gap, 'cached_shape'):     # -- Check if cached shape exists
            del process_AGYW_HTS_gap.cached_shape             # -- Clear cached shape
        return                                                # -- Exit function

#### - AGYW Positive

In [ ]:
# -- Define the main function to process AGYW Positive gap
def process_AGYW_Positive_gap(display_output=None):
    """
    Process AGYW Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_columns = [                             # -- Define list of AGYW Positive columns in desired order
            "Number of AGYW that received an HIV test during the reporting period and know their status (Community)",
            "Number of AGYW that received an HIV test during the reporting period and know their status (Walk-In)",
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)"
        ]
        name = "AGYW Positive gap"                            # -- Define general name
        AGYW_Positive_gap_columns = ["AGYW tested Positive gap"]  # -- Define gap column name
        AGYW_Positive_msg = f"No {name}"                      # -- Define message for no gaps
        report_name = f"{name}2"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_Positive = prepare_and_convert_df(            # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_Positive_columns                # -- Include specified AGYW Positive columns
        )
        if df_AGYW_Positive is None:                          # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        wrap_column_headers(df_AGYW_Positive)
        AGYW_Positive_columns2 = wrap_column_headers2(AGYW_Positive_columns)

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_Positive.shape        # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_Positive_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW tested
        df_AGYW_Positive["Total AGYW tested"] = (             # -- Calculate total tested
            df_AGYW_Positive[AGYW_Positive_columns2[0]] +      # -- Add Community tested
            df_AGYW_Positive[AGYW_Positive_columns2[1]]        # -- Add Walk-In tested
        )
        # -- Step 4.2: Total AGYW tested positive
        df_AGYW_Positive["Total AGYW tested positive"] = (    # -- Calculate total tested positive
            df_AGYW_Positive[AGYW_Positive_columns2[2]] +      # -- Add Community positive
            df_AGYW_Positive[AGYW_Positive_columns2[3]]        # -- Add Walk-In positive
        )
        # -- Step 4.3: AGYW tested Positive gap
        df_AGYW_Positive[AGYW_Positive_gap_columns[0]] = np.where(  # -- Calculate gap
            df_AGYW_Positive["Total AGYW tested positive"] > df_AGYW_Positive["Total AGYW tested"],  # -- Condition for gap
            df_AGYW_Positive["Total AGYW tested positive"] - df_AGYW_Positive["Total AGYW tested"],  # -- Positive gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_Positive)                # -- Apply wrapping to DataFrame headers
        df_columns_wrap = wrap_column_headers2(AGYW_Positive_columns)  # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(AGYW_Positive_gap_columns)  # -- Wrap gap column names
        
        # -- Step 5: Filter and validate gaps
        df_AGYW_Positive_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_Positive,                              # -- Input DataFrame
            msg=AGYW_Positive_msg,                            # -- Message for empty result
            opNonZero=gap_columns_wrap,              # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        
        if df_AGYW_Positive_gap is None:                      # -- Check if no gaps found
            if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_Positive_gap.cached_style    # -- Clear cached style
            if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_Positive_gap.cached_shape    # -- Clear cached shape
            return                                            # -- Exit function
        
        if df_AGYW_Positive_gap is None:                          # -- Check if data preparation failed
            return 

        # -- Step 6: Style the DataFrame
        df_AGYW_Positive_gap_style = (                        # -- Apply styling to filtered DataFrame
            df_AGYW_Positive_gap.style                        # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=gap_columns_wrap)  # -- Highlight outliers in gap column
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_Positive_gap.cached_style = df_AGYW_Positive_gap_style  # -- Cache styled DataFrame
        process_AGYW_Positive_gap.cached_shape = df_AGYW_Positive.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_Positive_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create description for Word document
        if (df_AGYW_Positive_gap[gap_columns_wrap[0]] != 0).any():  # -- Check if any gaps exist
            report_description = (                            # -- Define report description if gaps present
                f"Report Name: {gap_columns_wrap[0]}"
                f"\n{AGYW_Positive_columns[2]}\nplus {AGYW_Positive_columns[3]}"
                f"\nshould not be greater than"
                f"\n{AGYW_Positive_columns[0]}\nplus {AGYW_Positive_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_style=df_AGYW_Positive_gap_style,              # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_Positive_columns,  # -- Italicize AGYW Positive columns
            doc_indicators_to_underline=AGYW_Positive_gap_columns,  # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            widget_display_df(df_AGYW_Positive_gap_style)               # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_Positive_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_Positive_gap.cached_style        # -- Clear cached style
        if hasattr(process_AGYW_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_Positive_gap.cached_shape        # -- Clear cached shape
        return                                            # -- Exit function

#### - AGYW Linkage

In [ ]:
# -- Define the main function to process AGYW Positive Linkage gap
def process_AGYW_Positive_Linkage_gap(display_output=None):
    """
    Process AGYW Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_Positive_Linkage_columns = [                     # -- Define list of AGYW Positive Linkage columns in desired order
            "Number of AGYW who tested HIV Positive during the reporting period (Community)",
            "Number of AGYW who tested HIV Positive during the reporting period (Walk-In)",
            "Total number of AGYW who tested HIV Positive and are successfully linked to treatment during the reporting period (Community & Walk-In)",
            "Linked/Referred for treatment to GF supported site (subset of 4)",
            "Linked/Referred for treatment to non-GF supported site (subset of 4)",
            "Number of AGYW newly started on ART during the reporting period"
        ] 
        name = "AGYW Positive Linkage gap"                    # -- Define general name
        AGYW_Positive_Linkage_gap_columns = [                 # -- Define list of gap column names
            "AGYW positive linked to treatment gap",
            "AGYW positive linkage to GF/non-GF supported site gap",
            "AGYW newly started on ART gap"
        ]
        AGYW_Positive_Linkage_msg = f"No {name}"              # -- Define message for no gaps
        report_name = f"{name}3"                              # -- Define report name

        # -- Step 2: Prepare data
        df_AGYW_Positive_Linkage = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW MSF',                        # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=AGYW_Positive_Linkage_columns        # -- Include specified AGYW Positive Linkage columns
        )
        if df_AGYW_Positive_Linkage is None:                  # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        wrap_column_headers(df_AGYW_Positive_Linkage)
        AGYW_Positive_Linkage_columns2 = wrap_column_headers2(AGYW_Positive_Linkage_columns)

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: Total AGYW Tested Positive
        df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] = (  # -- Calculate total tested positive
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[0]] +  # -- Add Community positive
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[1]]    # -- Add Walk-In positive
        )
        # -- Step 4.2: AGYW positive linked to treatment gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[0]] = np.where(  # -- Calculate linkage gap
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # -- Condition for gap
            df_AGYW_Positive_Linkage["Total AGYW Tested Positive"] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )
        # -- Step 4.3: AGYW positive linkage to GF/non-GF supported site gap
        total_linked_to_sites = (                             # -- Precompute total linked to sites
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[3]] +  # -- Add GF supported site
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[4]]    # -- Add non-GF supported site
        )
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[1]] = np.where(  # -- Calculate site linkage gap
            total_linked_to_sites != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # -- Condition for gap
            total_linked_to_sites - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )
        # -- Step 4.4: AGYW newly started on ART gap
        df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_gap_columns[2]] = np.where(  # -- Calculate ART gap
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]] != df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[5]],  # -- Condition for gap
            df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[2]] - df_AGYW_Positive_Linkage[AGYW_Positive_Linkage_columns2[5]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                    # -- Check if display is requested
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_Positive_Linkage_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_Positive_Linkage.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:             # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    print(cached_display_name)                # -- Print display message
                    print(f"-" * len(cached_display_name))    # -- Print separator line
                    display(process_AGYW_Positive_Linkage_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                    # -- Exit function

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_Positive_Linkage)        # -- Apply wrapping to DataFrame headers
        df_columns_wrap = wrap_column_headers2(AGYW_Positive_Linkage_columns)  # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(AGYW_Positive_Linkage_gap_columns)  # -- Wrap gap column names

        # -- Step 5: Filter and validate gaps
        df_AGYW_Positive_Linkage_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_Positive_Linkage,                      # -- Input DataFrame
            msg=AGYW_Positive_Linkage_msg,                    # -- Message for empty result
            opNonZero=gap_columns_wrap,      # -- Filter for non-zero gaps
            opNeg=None,                                       # -- No negative filter
            opPos=None,                                       # -- No positive filter
            opZero=None,                                      # -- No zero filter
            opLT100=None                                      # -- No less-than-100 filter
        )
        if df_AGYW_Positive_Linkage_gap is None:              # -- Check if no gaps found
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_Positive_Linkage_gap.cached_style  # -- Clear cached style
            if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_Positive_Linkage_gap.cached_shape  # -- Clear cached shape
            return                                            # -- Exit function
        
        if df_AGYW_Positive_Linkage_gap is None:                          # -- Check if data preparation failed
            return 

        # -- Step 6: Style the DataFrame
        df_AGYW_Positive_Linkage_gap_style = (                # -- Apply styling to filtered DataFrame
            df_AGYW_Positive_Linkage_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                               # -- Hide index column
            .map(outlier_red, subset=gap_columns_wrap)  # -- Highlight outliers in gap columns
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_Positive_Linkage_gap.cached_style = df_AGYW_Positive_Linkage_gap_style  # -- Cache styled DataFrame
        process_AGYW_Positive_Linkage_gap.cached_shape = df_AGYW_Positive_Linkage.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_Positive_Linkage_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                       # -- Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        report_description = []                               # -- Initialize list to collect descriptions
        # -- Step 9.1: Add description for AGYW Positive Linkage gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[0]] != 0).any():  # -- Check if linkage gap exists
            report_description.append(                           # -- Add description for linkage gap
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[0]}"
                f"\n{AGYW_Positive_Linkage_columns[0]}\nplus {AGYW_Positive_Linkage_columns[1]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns2[2]}"
                f"\nNote: Where this AGYW linkage gap is true, please ignore the outlier."
            )
        # -- Step 9.2: Add description for AGYW Positive Linkage to GF/non-GF supported site gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[1]] != 0).any():  # -- Check if site linkage gap exists
            report_description.append(                        # -- Add description for site linkage gap
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[1]}"
                f"\n{AGYW_Positive_Linkage_columns[3]}\nplus {AGYW_Positive_Linkage_columns[4]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns2[2]}"
            )
        # -- Step 9.3: Add description for AGYW newly started on ART gap
        if (df_AGYW_Positive_Linkage_gap[gap_columns_wrap[2]] != 0).any():  # -- Check if ART gap exists
            report_description.append(                        # -- Add description for ART gap
                f"Report Name: {AGYW_Positive_Linkage_gap_columns[2]}"
                f"\n{AGYW_Positive_Linkage_columns[5]}"
                f"\nshould be equal to {AGYW_Positive_Linkage_columns[2]}"
            )
        # -- Step 9.4: Join all descriptions
        report_description = "\n\n".join(report_description)

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                         # -- Export DataFrame to multiple formats
            report_name=report_name,                          # -- Pass report name
            df_style=df_AGYW_Positive_Linkage_gap_style,      # -- Pass styled DataFrame
            img_file_name=report_image_name,                  # -- Pass image file name
            img_file_path=report_image_path,                  # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,          # -- Pass Word document path
            doc_description=report_description,               # -- Pass description
            doc_indicators_to_italicize=AGYW_Positive_Linkage_columns,  # -- Italicize AGYW Positive Linkage columns
            doc_indicators_to_underline=AGYW_Positive_Linkage_gap_columns,  # -- Underline gap columns
            xlm_file_path=doc_file_msf_outlier_xlsx,          # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                  # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                    # -- Check if display is requested
            widget_display_df(df_AGYW_Positive_Linkage_gap_style)       # -- Display styled DataFrame

    except Exception as e:                                    # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_Positive_Linkage_gap.cached_style  # -- Clear cached style
        if hasattr(process_AGYW_Positive_Linkage_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_Positive_Linkage_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function

#### - AGYW TB Screening

In [ ]:
# -- Define the main function to process AGYW TB Screening gap
def process_AGYW_TB_Screening_gap(display_output=None):
    """
    Process AGYW TB Screening gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        AGYW_TB_Screening_columns = [                     # -- Define list of AGYW TB Screening columns in desired order
            "Number of AGYW newly started on ART during the reporting period",
            "Number of AGYW screened for TB amongst those newly started on ART during the reporting period"
        ]
        name = "AGYW TB Screening gap"                    # -- Define general name
        AGYW_TB_Screening_gap_columns = ["AGYW TB screening gap"]  # -- Define list of gap column names
        report_name = f"{name}4"                          # -- Define report name
        AGYW_TB_Screening_msg = f"No {report_name}"       # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_AGYW_TB_Screening = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key='AGYW MSF',                    # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,              # -- Use MSF hierarchy columns
            data_columns=AGYW_TB_Screening_columns        # -- Include specified AGYW TB Screening columns
        )
        if df_AGYW_TB_Screening is None:                  # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        wrap_column_headers(df_AGYW_TB_Screening)
        AGYW_TB_Screening_columns2 = wrap_column_headers2(AGYW_TB_Screening_columns)

        # -- Step 4: Calculate derived metrics
        # -- Step 4.1: AGYW TB Screening gap
        df_AGYW_TB_Screening[AGYW_TB_Screening_gap_columns[0]] = np.where(  # -- Calculate TB Screening gap
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[1]] != df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[0]],  # -- Condition for gap
            df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[1]] - df_AGYW_TB_Screening[AGYW_TB_Screening_columns2[0]],  # -- Gap value
            0                                                 # -- Default to 0 if no gap
        )

        # -- Step 3: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_AGYW_TB_Screening_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_AGYW_TB_Screening.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:           # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)              # -- Print display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    display(process_AGYW_TB_Screening_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                  # -- Exit function

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_AGYW_TB_Screening)          # -- Apply wrapping to DataFrame headers
        df_columns_wrap = wrap_column_headers2(AGYW_TB_Screening_columns)  # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(AGYW_TB_Screening_gap_columns)  # -- Wrap gap column names

        # -- Step 5: Filter and validate gaps
        df_AGYW_TB_Screening_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_AGYW_TB_Screening,                      # -- Input DataFrame
            msg=AGYW_TB_Screening_msg,                    # -- Message for empty result
            opNonZero=gap_columns_wrap,      # -- Filter for non-zero gaps
            opNeg=None,                                   # -- No negative filter
            opPos=None,                                   # -- No positive filter
            opZero=None,                                  # -- No zero filter
            opLT100=None                                  # -- No less-than-100 filter
        )
        if df_AGYW_TB_Screening_gap is None:              # -- Check if no gaps found
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
                del process_AGYW_TB_Screening_gap.cached_style  # -- Clear cached style
            if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_AGYW_TB_Screening_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function
        
        if df_AGYW_TB_Screening_gap is None:                          # -- Check if data preparation failed
            return 

        # -- Step 6: Style the DataFrame
        df_AGYW_TB_Screening_gap_style = (                # -- Apply styling to filtered DataFrame
            df_AGYW_TB_Screening_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                           # -- Hide index column
            .map(outlier_red, subset=gap_columns_wrap)  # -- Highlight outliers in gap columns
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_AGYW_TB_Screening_gap.cached_style = df_AGYW_TB_Screening_gap_style  # -- Cache styled DataFrame
        process_AGYW_TB_Screening_gap.cached_shape = df_AGYW_TB_Screening.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8: Define export variables
        report_month = df_AGYW_TB_Screening_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                   # -- Define Excel sheet name

        # -- Step 9: Create descriptions for Word document
        # -- Step 9.1: Add description for AGYW Positive Linkage gap
        if (df_AGYW_TB_Screening_gap[gap_columns_wrap[0]] != 0).any():  # -- Check if TB Screening gap exists
            report_description = (                        # -- Add description for TB Screening gap
                f"Report Name: {AGYW_TB_Screening_gap_columns[0]}"
                f"\n{AGYW_TB_Screening_columns[1]}\nshould be equal to {AGYW_TB_Screening_columns[0]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                     # -- Export DataFrame to multiple formats
            report_name=report_name,                      # -- Pass report name
            df_style=df_AGYW_TB_Screening_gap_style,      # -- Pass styled DataFrame
            img_file_name=report_image_name,              # -- Pass image file name
            img_file_path=report_image_path,              # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,      # -- Pass Word document path
            doc_description=report_description,           # -- Pass description
            doc_indicators_to_italicize=AGYW_TB_Screening_columns,  # -- Italicize AGYW Positive Linkage columns
            doc_indicators_to_underline=AGYW_TB_Screening_gap_columns,  # -- Underline gap columns
            xlm_file_path=doc_file_msf_outlier_xlsx,      # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name              # -- Pass Excel sheet name
        )

        # -- Step 11: Optionally display styled DataFrame
        if display_output:                                # -- Check if display is requested
            widget_display_df(df_AGYW_TB_Screening_gap_style)       # -- Display styled DataFrame

    except Exception as e:                                # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
            del process_AGYW_TB_Screening_gap.cached_style  # -- Clear cached style
        if hasattr(process_AGYW_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_AGYW_TB_Screening_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function

### ART

#### - ART Linkage

In [ ]:
# -- Define the main function to process ART positive and enrollment gap
def process_ART_PosEnrolment_gap(display_output=None):          # -- Define function
    """
    Process ART Positive and Enrolment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_PosEnrolment_columns = [                       # -- Define list of ART positive and enrollment columns
            "ART 1: Number of HIV positive persons newly enrolled in clinical care during the month",
            "ART 2: Number of people living with HIV newly started on ART during the month (excludes ART transfer-in)"
        ]
        ART_PosEnrolment_columns_desrpt = ART_PosEnrolment_columns + ["Total Tested Positive"]  # -- Extended list for description
        name = "ART Positive-Enrolment gap"                # -- General report name
        ART_PosEnrolment_gap_columns = ["ART Enrolment gap", "ART Linkage gap"]  # -- Gap column names
        report_name = f"{name}5"                           # -- Report name with suffix

        # -- Step 2: Prepare data
        df_ART_PosEnrolment = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key='ART MSF',                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_PosEnrolment_columns          # -- ART columns
        )

        # -- Step 3: Merge with external DataFrame
        df_ART_PosEnrolment = Pre_MSF_positives_all.merge(  # -- Merge with positives data
            df_ART_PosEnrolment,                           # -- Merge target
            on=["ReportPeriod", "Cluster", "LGA", "FacilityName"],  # -- Merge keys
            how="left"                                    # -- Keep all rows from df_ART_PosEnrolment
        )
        df_ART_PosEnrolment = df_ART_PosEnrolment.fillna(0)  # -- Step 3.1: Fill NaN with 0
        float_columns = df_ART_PosEnrolment.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                          # -- Step 3.2: Cast float to int
            df_ART_PosEnrolment[col] = df_ART_PosEnrolment[col].astype(int)

        wrap_column_headers(df_ART_PosEnrolment)
        ART_PosEnrolment_columns2 = wrap_column_headers2(ART_PosEnrolment_columns)

        # -- Step 6: Calculate derived metrics
        # -- Step 6.1: ART enrolment gap
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[0]] = np.where(  
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]] != df_ART_PosEnrolment["Total new positive"],
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]] - df_ART_PosEnrolment["Total new positive"],
            0
        )
        # -- Step 6.2: ART linkage gap
        df_ART_PosEnrolment[ART_PosEnrolment_gap_columns[1]] = np.where(  
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[1]] != df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]],
            df_ART_PosEnrolment[ART_PosEnrolment_columns2[1]] - df_ART_PosEnrolment[ART_PosEnrolment_columns2[0]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Check cache existence
                cached_shape = getattr(process_ART_PosEnrolment_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_PosEnrolment.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    for cluster, style in process_ART_PosEnrolment_gap.cached_styles.items():  # -- Iterate cached styles
                        cached_display_name = f"✔️ Displaying {cluster} {report_name} "  # -- Display message
                        print("-" * len(cached_display_name))  # -- Separator
                        print(cached_display_name)            # -- Message
                        print("-" * len(cached_display_name))  # -- Separator
                        display(style)                        # -- Display cached style
                    return                                 # -- Exit if cache used
                    
        # -- Step 5: Initialize cache
        if not hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Check if cache exists
            process_ART_PosEnrolment_gap.cached_styles = {}  # -- Initialize cache

        # -- Step 7: Identify unique clusters
        cluster_list = pd.Series(df_ART_PosEnrolment['Cluster'].unique())  # -- Extract unique clusters

        # -- Step 8: Process each cluster
        for current_cluster in cluster_list:               # -- Iterate over clusters
            cluster_filtered = df_ART_PosEnrolment[df_ART_PosEnrolment['Cluster'] == current_cluster]  # -- Step 8.1: Filter cluster
            
            ART_PosEnrolment_msg = f"No {current_cluster} {report_name}"  # -- Message for no gaps

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=cluster_filtered,
                msg=ART_PosEnrolment_msg,
                opNonZero=ART_PosEnrolment_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

            if cluster_filtered_gap is None:               # -- Check if no gaps
                if current_cluster in process_ART_PosEnrolment_gap.cached_styles:  # -- Remove from cache
                    del process_ART_PosEnrolment_gap.cached_styles[current_cluster]
                continue                                   # -- Skip cluster

            cluster_filtered_style = (                     # -- Step 8.3: Style DataFrame
                cluster_filtered_gap.style
                .hide(axis='index')
                .map(outlier_red, subset=ART_PosEnrolment_gap_columns)
            )

            process_ART_PosEnrolment_gap.cached_styles[current_cluster] = cluster_filtered_style  # -- Step 8.4: Cache style

            # -- Step 8.5: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # -- Cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # -- Extract report month
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # -- Image file name
            #report_image_path = rf"{sub_folder2_image_file_msf_outlier}\{report_image_name}"  # -- Image path
            report_sheet_name = f"{current_cluster}_{report_name}"  # -- Excel sheet name

            # -- Step 9: Create descriptions
            report_description = []                        # -- Initialize descriptions
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[0]] != 0).any():  # -- Step 9.1: Enrolment gap desc
                report_description.append(
                    f"Report Name: {ART_PosEnrolment_gap_columns[0]}\n"
                    f"{ART_PosEnrolment_columns_desrpt[0]}\nshould be equal to {ART_PosEnrolment_columns_desrpt[2]}\n"
                    f"Note: Where this ART enrolment gap is true, please ignore the outlier."
                )
            if (cluster_filtered_gap[ART_PosEnrolment_gap_columns[1]] != 0).any():  # -- Step 9.2: Linkage gap desc
                report_description.append(
                    f"Report Name: {ART_PosEnrolment_gap_columns[1]}\n"
                    f"{ART_PosEnrolment_columns_desrpt[1]}\nshould be equal to {ART_PosEnrolment_columns_desrpt[0]}\n"
                    f"Note: Where this ART linkage gap is true, please ignore the outlier."
                )
            report_description = "\n\n".join(report_description)  # -- Step 9.3: Join descriptions

            # -- Step 10: Export results
            export_df_to_doc_image_excel(                  # -- Export DataFrame
                report_name=report_name_cluster,
                df_style=cluster_filtered_style,
                img_file_name=report_image_name,
                img_file_path=sub_folder2_image_file_msf_outlier,
                doc_description=report_description,
                doc_indicators_to_italicize=ART_PosEnrolment_columns_desrpt,
                doc_indicators_to_underline=ART_PosEnrolment_gap_columns,
                xlm_file_path=doc_file_msf_outlier_xlsx,
                xlm_sheet_name=report_sheet_name
            )

            if display_output:                             # -- Step 11: Display styled DataFrame
                widget_display_df(cluster_filtered_style)

        # -- Step 12: Cache overall unfiltered DataFrame shape
        process_ART_PosEnrolment_gap.cached_shape = df_ART_PosEnrolment.shape  # -- Cache shape

    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_PosEnrolment_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_PosEnrolment_gap.cached_styles.clear()
        if hasattr(process_ART_PosEnrolment_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_PosEnrolment_gap.cached_shape
        return

#### - ART Regimen Line, MMD and DSD

In [ ]:
# -- Define the main function to process ART regimen line, MMD and SDS gap
def process_ART_RegimentLine_MMD_DSD_gap(display_output=None):          # -- Define function
    """
    Process ART Regimen line, MMD and DSD gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_RegimenLine_MMD_DSD_columns = [                       # -- Define list of ART positive and enrollment columns
            "ART 3: Number of people living with HIV who are currently receiving ART during the month (All regimens)",
            "Number of people living with HIV who are currently receiving ART during the month (All regimens): By Regimen Line",
            "Number of people living with HIV who are currently receiving ART during the month (All regimens) - Multi-Month Dispensing",
            "Number of people living with HIV who are currently receiving ART during the month (All regimens) - DSD Model"
        ]
        name = "ART Regimen-Line MMD DSD gap"              # -- General report name
        ART_RegimenLine_MMD_DSD_gap_columns = ["ART Regimen Line gap", "ART MMD gap", "ART DSD gap"]  # -- Gap column names
        report_name = f"{name}6"                           # -- Report name with suffix
        ART_RegimenLine_MMD_DSD_msg = f"No {report_name}"  # -- Message for no gaps

        # -- Step 2: Prepare data
        df_ART_RegimenLine_MMD_DSD = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF",                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_RegimenLine_MMD_DSD_columns          # -- ART regimne columns
        )
        if df_ART_RegimenLine_MMD_DSD is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        wrap_column_headers(df_ART_RegimenLine_MMD_DSD)
        ART_RegimenLine_MMD_DSD_columns2 = wrap_column_headers2(ART_RegimenLine_MMD_DSD_columns)

        # -- Step : ART regimen line gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[0]] = np.where(  
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] != df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[1]],
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[1]],
            0
        )
        # -- Step : ART MMD gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[1]] = np.where(  
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] < df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[2]],
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[2]],
            0
        )
        # -- Step : ART DSD gap
        df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_gap_columns[2]] = np.where(  
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] < df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[3]],
            df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[0]] - df_ART_RegimenLine_MMD_DSD[ART_RegimenLine_MMD_DSD_columns2[3]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_style'):  # -- Check cache existence
                cached_shape = getattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_RegimenLine_MMD_DSD.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message
                    print("-" * len(cached_display_name))  # -- Separator
                    print(cached_display_name)            # -- Message
                    print("-" * len(cached_display_name))  # -- Separator
                    display(process_ART_RegimentLine_MMD_DSD_gap.cached_style)                        # -- Display cached style
                return                                 # -- Exit if cache used
                    
        # -- Step 5: Filter and validate gaps
        df_ART_RegimenLine_MMD_DSD_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=df_ART_RegimenLine_MMD_DSD,
                msg=ART_RegimenLine_MMD_DSD_msg,
                opNonZero=ART_RegimenLine_MMD_DSD_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

        if df_ART_RegimenLine_MMD_DSD_gap is None:              # -- Check if no gaps found
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_style'):  # -- Check if cache exists
                del process_ART_RegimentLine_MMD_DSD_gap.cached_style  # -- Clear cached style
            if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ART_RegimentLine_MMD_DSD_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function      

        if df_ART_RegimenLine_MMD_DSD_gap is None:                          # -- Check if data preparation failed
            return                         

        df_ART_RegimenLine_MMD_DSD_gap_style = (                     # -- Step 8.3: Style DataFrame
            df_ART_RegimenLine_MMD_DSD_gap.style
            .hide(axis='index')
            .map(outlier_red, subset=ART_RegimenLine_MMD_DSD_gap_columns)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_RegimentLine_MMD_DSD_gap.cached_style = df_ART_RegimenLine_MMD_DSD_gap_style  # -- Cache styled DataFrame
        process_ART_RegimentLine_MMD_DSD_gap.cached_shape = df_ART_RegimenLine_MMD_DSD.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8.5: Define export variables
        report_month = df_ART_RegimenLine_MMD_DSD_gap['ReportPeriod'].iloc[0]  # -- Extract report month
        report_image_name = f"{report_month}_{report_name}.png"                # -- Image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Image path
        report_sheet_name = f"{report_name}"                                   # -- Excel sheet name

        # -- Step 9: Create descriptions
        report_description = []                        # -- Initialize descriptions
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[0]] != 0).any():  # -- Step 9.1: ART regimen line gap
            report_description.append(
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[0]}\n"
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be equal to {ART_RegimenLine_MMD_DSD_columns[1]}"
            )
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[1]] != 0).any():  # -- Step 9.2: ART MMD gap
            report_description.append(
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[1]}\n"
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be greater than {ART_RegimenLine_MMD_DSD_columns[2]}\n"
            )
        if (df_ART_RegimenLine_MMD_DSD_gap[ART_RegimenLine_MMD_DSD_gap_columns[2]] != 0).any():  # -- Step 9.2: ART DSD gap
            report_description.append(
                f"Report Name: {ART_RegimenLine_MMD_DSD_gap_columns[2]}\n"
                f"{ART_RegimenLine_MMD_DSD_columns[0]}\nshould be greater than {ART_RegimenLine_MMD_DSD_columns[3]}\n"
            )
        report_description = "\n\n".join(report_description)  # -- Step 9.3: Join descriptions

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame
            report_name=report_name,
            df_style=df_ART_RegimenLine_MMD_DSD_gap_style,
            img_file_name=report_image_name,
            img_file_path=report_image_path,
            doc_description=report_description,
            doc_indicators_to_italicize=ART_RegimenLine_MMD_DSD_columns,
            doc_indicators_to_underline=ART_RegimenLine_MMD_DSD_gap_columns,
            xlm_file_path=doc_file_msf_outlier_xlsx,
            xlm_sheet_name=report_sheet_name
        )

        if display_output:                             # -- Step 11: Display styled DataFrame
            widget_display_df(df_ART_RegimenLine_MMD_DSD_gap_style)
            
    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_RegimentLine_MMD_DSD_gap.cached_style.clear()
        if hasattr(process_ART_RegimentLine_MMD_DSD_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_RegimentLine_MMD_DSD_gap.cached_shape
        return

#### - ART TB Screening

In [ ]:
# -- Define the main function to process ARTTB Screening gap
def process_ART_TB_Screening_gap(display_output=None):          # -- Define function
    """
    Process ART Regimen line, MMD and DSD gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Screening_columns = ["ART 2: Number of people living with HIV newly started on ART during the month (excludes ART transfer-in)"]
        place_holder = ["ART 10: Number of PLHIV on ART (Including PMTCT) who were Clinically Screened for TB in HIV Treatment Settings"]
        ART_TB_Screening_columns_new_old = ['Newly on ART', 'Already on ART']  # -- Define list of ART TB Screening columns

        ART_TB_Screening_columns_desrpt = [
            ART_TB_Screening_columns[0],
            place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[0],
            place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[1]
        ]  # -- Extended list for description

        name = "ART Tx_New TB Screening gap"              # -- General report name
        ART_TB_Screening_gap_columns = ['ART Tx New TB screening gap']  # -- Gap column names
        report_name = f"{name}7"                           # -- Report name with suffix
        ART_RegimenLine_MMD_DSD_msg = f"No {report_name}"  # -- Message for no gaps

        # -- Step 2: Prepare data
        df_ART_TB_Screening = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF",                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_TB_Screening_columns          # -- ART regimne columns
        )
        if df_ART_TB_Screening is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data
        
        # -- Step 2.1: Prepare TB screening data
        df_ART_TB_Screening_new_old = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF_tb screening",        # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_TB_Screening_columns_new_old   # -- ART regimne columns
        )
        if df_ART_TB_Screening_new_old is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data
        
        # -- Step 3: Merge with external DataFrame
        df_ART_TB_Screening = df_ART_TB_Screening.merge(  # -- Merge with ART_tb screening data
            df_ART_TB_Screening_new_old,                           # -- Merge target
            on=["ReportPeriod", "Cluster", "LGA", "FacilityName"],  # -- Merge keys
            how="left"                                    # -- Keep all rows from df_ART_PosEnrolment
        )
        df_ART_TB_Screening = df_ART_TB_Screening.fillna(0)  # -- Step 3.1: Fill NaN with 0
        float_columns = df_ART_TB_Screening.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                          # -- Step 3.2: Cast float to int
            df_ART_TB_Screening[col] = df_ART_TB_Screening[col].astype(int)
        
        # -- Step 3.3: Rename columns
        df_ART_TB_Screening = df_ART_TB_Screening.rename(columns={
            'Newly on ART': place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[0],
            'Already on ART': place_holder[0] + ' - ' + ART_TB_Screening_columns_new_old[1]}
        )   

        wrap_column_headers(df_ART_TB_Screening)
        ART_TB_Screening_columns2 = wrap_column_headers2(ART_TB_Screening_columns_desrpt)

        # -- Step : ART TB Screening gap
        df_ART_TB_Screening[ART_TB_Screening_gap_columns[0]] = np.where(  
            df_ART_TB_Screening[ART_TB_Screening_columns2[0]] != df_ART_TB_Screening[ART_TB_Screening_columns2[1]],
            df_ART_TB_Screening[ART_TB_Screening_columns2[0]] - df_ART_TB_Screening[ART_TB_Screening_columns2[1]],
            0
        )
        
        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_TB_Screening_gap, 'cached_style'):  # -- Check cache existence
                cached_shape = getattr(process_ART_TB_Screening_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_TB_Screening.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message
                    print("-" * len(cached_display_name))  # -- Separator
                    print(cached_display_name)            # -- Message
                    print("-" * len(cached_display_name))  # -- Separator
                    display(process_ART_TB_Screening_gap.cached_style)                        # -- Display cached style
                return                                 # -- Exit if cache used
                    
        # -- Step 5: Filter and validate gaps
        df_ART_TB_Screening_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=df_ART_TB_Screening,
                msg=ART_RegimenLine_MMD_DSD_msg,
                opNonZero=ART_TB_Screening_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

        if df_ART_TB_Screening_gap is None:              # -- Check if no gaps found
            if hasattr(process_ART_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
                del process_ART_TB_Screening_gap.cached_style  # -- Clear cached style
            if hasattr(process_ART_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ART_TB_Screening_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function                       

        if df_ART_TB_Screening_gap is None:      # -- Check if data preparation failed
            return

        df_ART_TB_Screening_gap_style = (                     # -- Step 8.3: Style DataFrame
            df_ART_TB_Screening_gap.style
            .hide(axis='index')
            .map(outlier_red, subset=ART_TB_Screening_gap_columns)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_TB_Screening_gap.cached_style = df_ART_TB_Screening_gap_style  # -- Cache styled DataFrame
        process_ART_TB_Screening_gap.cached_shape = df_ART_TB_Screening.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8.5: Define export variables
        report_month = df_ART_TB_Screening_gap['ReportPeriod'].iloc[0]  # -- Extract report month
        report_image_name = f"{report_month}_{report_name}.png"                # -- Image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Image path
        report_sheet_name = f"{report_name}"                                   # -- Excel sheet name

        # -- Step 9: Create descriptions
        if (df_ART_TB_Screening_gap[ART_TB_Screening_gap_columns[0]] != 0).any():  # -- Step 9.1: ART regimen line gap
            report_description = (
                f"Report Name: {ART_TB_Screening_gap_columns[0]}\n"
                f"{ART_TB_Screening_columns_desrpt[0]}\nshould be equal to {ART_TB_Screening_columns_desrpt[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame
            report_name=report_name,
            df_style=df_ART_TB_Screening_gap_style,
            img_file_name=report_image_name,
            img_file_path=report_image_path,
            doc_description=report_description,
            doc_indicators_to_italicize=ART_TB_Screening_columns_desrpt,
            doc_indicators_to_underline=ART_TB_Screening_gap_columns,
            xlm_file_path=doc_file_msf_outlier_xlsx,
            xlm_sheet_name=report_sheet_name
        )

        if display_output:                             # -- Step 11: Display styled DataFrame
            widget_display_df(df_ART_TB_Screening_gap_style)
            
    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_TB_Screening_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_TB_Screening_gap.cached_style.clear()
        if hasattr(process_ART_TB_Screening_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_TB_Screening_gap.cached_shape
        return

#### - ART TB Presumptive

In [ ]:
# -- Define the main function to process ART Presumptive Test gap
def process_ART_TB_Presumptive_Test_gap(display_output=None):          # -- Define function
    """
    Process ART Presumptive Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Presumptive_Test_columns = [                       # -- Define list of ART presumptive test columns
            "ART 11: Number of PLHIV on ART with Presumptive TB during the month",
            "ART 12: Number of PLHIV on ART with Presumptive TB and Tested for TB during the month"
        ]
        name = "ART TB Presumptive Test gap"              # -- General report name
        ART_TB_Presumptive_Test_gap_columns = ["ART TB presumptive test gap"]  # -- Gap column names
        report_name = f"{name}8"                           # -- Report name with suffix
        ART_TB_Presumptive_Test_msg = f"No {report_name}"  # -- Message for no gaps

        # -- Step 2: Prepare data
        df_ART_TB_Presumptive_Test = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF",                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_TB_Presumptive_Test_columns          # -- ART TB presumptive test columns
        )
        if df_ART_TB_Presumptive_Test is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        wrap_column_headers(df_ART_TB_Presumptive_Test)
        ART_TB_Presumptive_Test_columns2 = wrap_column_headers2(ART_TB_Presumptive_Test_columns)

        # -- Step : ART regimen line gap
        df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_gap_columns[0]] = np.where(  
            df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[0]] != df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[1]],
            df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[0]] - df_ART_TB_Presumptive_Test[ART_TB_Presumptive_Test_columns2[1]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_style'):  # -- Check cache existence
                cached_shape = getattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_TB_Presumptive_Test.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message
                    print("-" * len(cached_display_name))  # -- Separator
                    print(cached_display_name)            # -- Message
                    print("-" * len(cached_display_name))  # -- Separator
                    display(process_ART_TB_Presumptive_Test_gap.cached_style)                        # -- Display cached style
                return                                 # -- Exit if cache used
                    
        # -- Step 5: Filter and validate gaps
        df_ART_TB_Presumptive_Test_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=df_ART_TB_Presumptive_Test,
                msg=ART_TB_Presumptive_Test_msg,
                opNonZero=ART_TB_Presumptive_Test_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

        if df_ART_TB_Presumptive_Test_gap is None:              # -- Check if no gaps found
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_style'):  # -- Check if cache exists
                del process_ART_TB_Presumptive_Test_gap.cached_style  # -- Clear cached style
            if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ART_TB_Presumptive_Test_gap.cached_shape  # -- Clear cached shape
            return   
        
        if df_ART_TB_Presumptive_Test_gap is None:      # -- Check if data preparation failed
            return                                       # -- Exit function                              # -- Skip cluster

        df_ART_TB_Presumptive_Test_gap_style = (                     # -- Step 8.3: Style DataFrame
            df_ART_TB_Presumptive_Test_gap.style
            .hide(axis='index')
            .map(outlier_red, subset=ART_TB_Presumptive_Test_gap_columns)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_TB_Presumptive_Test_gap.cached_style = df_ART_TB_Presumptive_Test_gap_style  # -- Cache styled DataFrame
        process_ART_TB_Presumptive_Test_gap.cached_shape = df_ART_TB_Presumptive_Test.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8.5: Define export variables
        report_month = df_ART_TB_Presumptive_Test_gap['ReportPeriod'].iloc[0]  # -- Extract report month
        report_image_name = f"{report_month}_{report_name}.png"                # -- Image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Image path
        report_sheet_name = f"{report_name}"                                   # -- Excel sheet name

        # -- Step 9: Create descriptions
        if (df_ART_TB_Presumptive_Test_gap[ART_TB_Presumptive_Test_gap_columns[0]] != 0).any():  # -- Step 9.1: ART regimen line gap
            report_description = (
                f"Report Name: {ART_TB_Presumptive_Test_gap_columns[0]}\n"
                f"{ART_TB_Presumptive_Test_columns[0]}\nshould be equal to {ART_TB_Presumptive_Test_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame
            report_name=report_name,
            df_style=df_ART_TB_Presumptive_Test_gap_style,
            img_file_name=report_image_name,
            img_file_path=report_image_path,
            doc_description=report_description,
            doc_indicators_to_italicize=ART_TB_Presumptive_Test_columns,
            doc_indicators_to_underline=ART_TB_Presumptive_Test_gap_columns,
            xlm_file_path=doc_file_msf_outlier_xlsx,
            xlm_sheet_name=report_sheet_name
        )

        if display_output:                             # -- Step 11: Display styled DataFrame
            widget_display_df(df_ART_TB_Presumptive_Test_gap_style)
            
    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_TB_Presumptive_Test_gap.cached_style.clear()
        if hasattr(process_ART_TB_Presumptive_Test_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_TB_Presumptive_Test_gap.cached_shape
        return

#### - ART TB Treatment

In [ ]:
# -- Define the main function to process ART TB Treatment gap
def process_ART_TB_Treatment_gap(display_output=None):          # -- Define function
    """
    Process ART TB Treatment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_TB_Treatment_columns = [                       # -- Define list of ART TB treatment columns
            "ART 13: Number of of PLHIV on ART who have Active TB Disease",
            "ART 14: Number of PLHIV on ART with active TB disease who initiated TB treatment"
        ]
        name = "ART TB Treatment gap"              # -- General report name
        ART_TB_Treatment_gap_columns = ["ART TB treatment gap"]  # -- Gap column names
        report_name = f"{name}9"                           # -- Report name with suffix
        ART_TB_Treatment_msg = f"No {report_name}"  # -- Message for no gaps

        # -- Step 2: Prepare data
        df_ART_TB_Treatment = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF",                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_TB_Treatment_columns          # -- ART TB pretretament columns
        )
        if df_ART_TB_Treatment is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        wrap_column_headers(df_ART_TB_Treatment)
        ART_TB_Treatment_columns_wrap = wrap_column_headers2(ART_TB_Treatment_columns)

        # -- Step : ART regimen line gap
        df_ART_TB_Treatment[ART_TB_Treatment_gap_columns[0]] = np.where(  
            df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[0]] != df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[1]],
            df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[0]] - df_ART_TB_Treatment[ART_TB_Treatment_columns_wrap[1]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_TB_Treatment_gap, 'cached_style'):  # -- Check cache existence
                cached_shape = getattr(process_ART_TB_Treatment_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_TB_Treatment.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message
                    print("-" * len(cached_display_name))  # -- Separator
                    print(cached_display_name)            # -- Message
                    print("-" * len(cached_display_name))  # -- Separator
                    display(process_ART_TB_Treatment_gap.cached_style)  # -- Display cached style
                return                                 # -- Exit if cache used
                    
        # -- Step 5: Filter and validate gaps
        df_ART_TB_Treatment_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=df_ART_TB_Treatment,
                msg=ART_TB_Treatment_msg,
                opNonZero=ART_TB_Treatment_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

        if df_ART_TB_Treatment_gap is None:              # -- Check if no gaps found
            if hasattr(process_ART_TB_Treatment_gap, 'cached_style'):  # -- Check if cache exists
                del process_ART_TB_Treatment_gap.cached_style  # -- Clear cached style
            if hasattr(process_ART_TB_Treatment_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ART_TB_Treatment_gap.cached_shape  # -- Clear cached shape
            return  
        
        if df_ART_TB_Treatment_gap is None:      # -- Check if data preparation failed
            return                                      # -- Exit function                             

        df_ART_TB_Treatment_gap_style = (                     # -- Step 8.3: Style DataFrame
            df_ART_TB_Treatment_gap.style
            .hide(axis='index')
            .map(outlier_red, subset=ART_TB_Treatment_gap_columns)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_TB_Treatment_gap.cached_style = df_ART_TB_Treatment_gap_style  # -- Cache styled DataFrame
        process_ART_TB_Treatment_gap.cached_shape = df_ART_TB_Treatment.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8.5: Define export variables
        report_month = df_ART_TB_Treatment_gap['ReportPeriod'].iloc[0]  # -- Extract report month
        report_image_name = f"{report_month}_{report_name}.png"                # -- Image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Image path
        report_sheet_name = f"{report_name}"                                   # -- Excel sheet name

        # -- Step 9: Create descriptions
        if (df_ART_TB_Treatment_gap[ART_TB_Treatment_gap_columns[0]] != 0).any():  # -- Step 9.1: ART regimen line gap
            report_description = (
                f"Report Name: {ART_TB_Treatment_gap_columns[0]}\n"
                f"{ART_TB_Treatment_columns[1]}\nshould be equal to {ART_TB_Treatment_columns[0]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame
            report_name=report_name,
            df_style=df_ART_TB_Treatment_gap_style,
            img_file_name=report_image_name,
            img_file_path=report_image_path,
            doc_description=report_description,
            doc_indicators_to_italicize=ART_TB_Treatment_columns,
            doc_indicators_to_underline=ART_TB_Treatment_gap_columns,
            xlm_file_path=doc_file_msf_outlier_xlsx,
            xlm_sheet_name=report_sheet_name
        )

        if display_output:                             # -- Step 11: Display styled DataFrame
            widget_display_df(df_ART_TB_Treatment_gap_style)
            
    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_TB_Treatment_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_TB_Treatment_gap.cached_style.clear()
        if hasattr(process_ART_TB_Treatment_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_TB_Treatment_gap.cached_shape
        return

#### - ART Viral Load Suppression

In [ ]:
# -- Define the main function to process ART Viral Load Suppression gap
def process_ART_Viral_Load_Suppression_gap(display_output=None):          # -- Define function
    """
    Process ART Viral Laod Suppression gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        ART_Viral_Load_Suppression_columns = [                       # -- Define list of ART viral load suppression columns
            "ART 6: Number of PLHIV on ART for at least 6 months with a VL test result during the month: Routine",
            "ART 6: Number of PLHIV on ART for at least 6 months with a VL test result during the month: Targeted",
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month: Routine",
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month: Targeted"
        ]
        ART_Viral_Load_Suppression_columns_cal = [
            "ART 6: Number of PLHIV on ART for at least 6 months with a VL test result during the month - Routine and Targated",
            "ART 7: Number of PLHIV on ART (for at least 6 months) who have virologic suppression (<1000 copies/ml) during the month - Routine and Targated"
        ]
        name = "ART Viral Load Suppression gap"            # -- General report name
        ART_Viral_Load_Suppression_gap_columns = ["ART viral load suppression gap"]  # -- Gap column names
        report_name = f"{name}10"                          # -- Report name with suffix
        No_gap_msg = f"No {report_name}"                   # -- Message for no gaps

        # -- Step 2: Prepare data
        df_ART_Viral_Load_Suppression = prepare_and_convert_df(      # -- Fetch and prepare DataFrame
            DHIS2_data_key="ART MSF",                      # -- DHIS2 data key
            hierarchy_columns=MSF_hierarchy,               # -- MSF hierarchy columns
            data_columns=ART_Viral_Load_Suppression_columns          # -- ART TB pretretament columns
        )
        if df_ART_Viral_Load_Suppression is None:      # -- Check if data preparation failed
            return                                        # -- Exit function if no data

        # -- Step : Calculate total viral laod results recieved
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_cal[0]] = (  
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[0]] + df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[1]]
        )

        # -- Step : Calculate total viral laod results suppressed
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_cal[1]] = (  
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[2]] + df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns[3]]
        )

        # -- Step: Drop the original Routine and Targeted columns
        df_ART_Viral_Load_Suppression = df_ART_Viral_Load_Suppression.drop(
            columns=[
                ART_Viral_Load_Suppression_columns[0],  # Routine (ART 6)
                ART_Viral_Load_Suppression_columns[1],  # Targeted (ART 6)
                ART_Viral_Load_Suppression_columns[2],  # Routine (ART 7)
                ART_Viral_Load_Suppression_columns[3]   # Targeted (ART 7)
            ]
        )

        wrap_column_headers(df_ART_Viral_Load_Suppression)
        ART_Viral_Load_Suppression_columns_wrap = wrap_column_headers2(ART_Viral_Load_Suppression_columns_cal)

        # -- Step : Viral load suppression gap
        df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_gap_columns[0]] = np.where(  
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[1]] > df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[0]],
            df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[1]] - df_ART_Viral_Load_Suppression[ART_Viral_Load_Suppression_columns_wrap[0]],
            0
        )

        # -- Step 4: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_style'):  # -- Check cache existence
                cached_shape = getattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_ART_Viral_Load_Suppression.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message
                    print("-" * len(cached_display_name))  # -- Separator
                    print(cached_display_name)            # -- Message
                    print("-" * len(cached_display_name))  # -- Separator
                    display(process_ART_Viral_Load_Suppression_gap.cached_style)  # -- Display cached style
                return                                 # -- Exit if cache used
                    
        # -- Step 5: Filter and validate gaps
        df_ART_Viral_Load_Suppression_gap = filter_gap_and_check_empty_df(  # -- Step 8.2: Filter gaps
                df=df_ART_Viral_Load_Suppression,
                msg=No_gap_msg,
                opNonZero=ART_Viral_Load_Suppression_gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

        if df_ART_Viral_Load_Suppression_gap is None:              # -- Check if no gaps found
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_style'):  # -- Check if cache exists
                del process_ART_Viral_Load_Suppression_gap.cached_style  # -- Clear cached style
            if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ART_Viral_Load_Suppression_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function                            
        
        if df_ART_Viral_Load_Suppression_gap is None:      # -- Check if data preparation failed
            return 

        df_ART_Viral_Load_Suppression_gap_style = (                     # -- Step 8.3: Style DataFrame
            df_ART_Viral_Load_Suppression_gap.style
            .hide(axis='index')
            .map(outlier_red, subset=ART_Viral_Load_Suppression_gap_columns)
        )

        # -- Step 7: Cache styled DataFrame and shape
        process_ART_Viral_Load_Suppression_gap.cached_style = df_ART_Viral_Load_Suppression_gap_style  # -- Cache styled DataFrame
        process_ART_Viral_Load_Suppression_gap.cached_shape = df_ART_Viral_Load_Suppression.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 8.5: Define export variables
        report_month = df_ART_Viral_Load_Suppression_gap['ReportPeriod'].iloc[0]  # -- Extract report month
        report_image_name = f"{report_month}_{report_name}.png"                # -- Image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Image path
        report_sheet_name = f"{report_name}"                                   # -- Excel sheet name

        # -- Step 9: Create descriptions
        if (df_ART_Viral_Load_Suppression_gap[ART_Viral_Load_Suppression_gap_columns[0]] != 0).any():  # -- Step 9.1: ART regimen line gap
            report_description = (
                f"Report Name: {ART_Viral_Load_Suppression_gap_columns[0]}\n"
                f"{ART_Viral_Load_Suppression_columns[2]}\nplus{ART_Viral_Load_Suppression_columns[3]}\n"
                f"should not be greater than {ART_Viral_Load_Suppression_columns[0]}\nplus{ART_Viral_Load_Suppression_columns[1]}"
            )

        # -- Step 10: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame
            report_name=report_name,
            df_style=df_ART_Viral_Load_Suppression_gap_style,
            img_file_name=report_image_name,
            img_file_path=report_image_path,
            doc_description=report_description,
            doc_indicators_to_italicize=ART_Viral_Load_Suppression_columns,
            doc_indicators_to_underline=ART_Viral_Load_Suppression_gap_columns,
            xlm_file_path=doc_file_msf_outlier_xlsx,
            xlm_sheet_name=report_sheet_name
        )

        if display_output:                             # -- Step 11: Display styled DataFrame
            widget_display_df(df_ART_Viral_Load_Suppression_gap_style)
            
    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_styles'):  # -- Clear cache on error
            process_ART_Viral_Load_Suppression_gap.cached_style.clear()
        if hasattr(process_ART_Viral_Load_Suppression_gap, 'cached_shape'):  # -- Clear shape on error
            del process_ART_Viral_Load_Suppression_gap.cached_shape
        return

### HTS

#### - HTS New Positive

In [ ]:
# -- Define the main function to process HTS New Positive gap
def process_HTS_New_Positive_gap(display_output=None):
    """
    Process HTS New Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_New_Positive_columns = [                     # -- Define list of HTS New Positive columns
            "Number of people who tested HIV positive and received results (Inpatient, Outpatient, Standalone)",
            "Number of people who tested HIV positive and received results (Community)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling (Inpatient, Outpatient, Standalone)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)",
            "HTS total tested - positive",
            "HTS total tested - previously known positive",
            "HTS total tested - new positive (excluding previously known)",
            "Number of people who tested HIV positive and received results (Inpatient)",
            "Number of people who tested HIV positive and received results (Outpatient)",
            "Number of people who tested HIV positive and received results (Standalone)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Inpatient)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Outpatient)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Standalone)"
        ]

        HTS_New_Positive_columns_order = [               # -- Define the desired order of columns
            "Number of people who tested HIV positive and received results (Inpatient, Outpatient, Standalone)",
            "Number of people who tested HIV positive and received results (Community)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling (Inpatient, Outpatient, Standalone)",
            "Total number of people tested HIV positive that were identified as known positive during post-test counselling.(Community)",
            "HTS total tested - positive",
            "HTS total tested - previously known positive",
            "HTS total tested - new positive (excluding previously known)"
        ]
        
        Gap_title_special = ["Community testing gap"]    # -- Define special gap title
        name = "HTS New Positive gap"                    # -- Define general name
        HTS_New_Positive_gap_columns = ["HTS total tested - new positive (excluding previously known)"]  # -- Define gap column
        report_name = f"{name}11"                        # -- Define report name
        No_gap_msg = f"No {report_name}"                 # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HTS_New_Positive = Pre_HTS_MSF_positive.copy()  # -- Copy of HTS total positives

        # -- Step 3: Calculate total HTS New Positive results
        df_HTS_New_Positive[HTS_New_Positive_columns[0]] = (  
            df_HTS_New_Positive.iloc[:, 4:7].sum(axis=1)  # -- Calculate total HTS New Positive results
        )

        # -- Step 4: Calculate total HTS New Positive results previously known
        df_HTS_New_Positive[HTS_New_Positive_columns[2]] = (  
            df_HTS_New_Positive.iloc[:, 8:11].sum(axis=1)  # -- Calculate total HTS New Positive results previously known
        )

        # -- Step 5: Drop the original HTS SDP columns
        df_HTS_New_Positive = df_HTS_New_Positive.drop(
            columns=[
                HTS_New_Positive_columns[7],
                HTS_New_Positive_columns[8],
                HTS_New_Positive_columns[9],
                HTS_New_Positive_columns[10],
                HTS_New_Positive_columns[11],
                HTS_New_Positive_columns[12]
            ]
        )

        # -- Step 6: Reorder the columns to match the desired order
        df_HTS_New_Positive = df_HTS_New_Positive[MSF_hierarchy + HTS_New_Positive_columns_order]  # -- Reorder columns

        wrap_column_headers(df_HTS_New_Positive)  # -- Wrap column headers for better readability
        HTS_New_Positive_columns_wrap = wrap_column_headers2(HTS_New_Positive_columns_order)
        HTS_New_Positive_gap_columns_wrap = wrap_column_headers2(HTS_New_Positive_gap_columns)

        # -- Step 7: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_New_Positive_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_HTS_New_Positive.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)             # -- Print display message
                    print(f"-" * len(cached_display_name))  # -- Print separator line
                    display(process_HTS_New_Positive_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                 # -- Exit function

        # -- Step 8: Filter and validate gaps
        df_HTS_New_Positive_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HTS_New_Positive,                      # -- Input DataFrame
            msg=No_gap_msg,                              # -- Message for empty result
            opNonZero=None,                              # -- No non-zero filter
            opNeg=HTS_New_Positive_columns_wrap,         # -- Filter for negative values
            opPos=None,                                  # -- No positive filter
            opZero=None,                                 # -- No zero filter
            opLT100=None                                 # -- No less-than-100 filter
        )
        
        if df_HTS_New_Positive_gap is None:              # -- Check if no gaps found
            if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # -- Check if cache exists
                del process_HTS_New_Positive_gap.cached_style  # -- Clear cached style
            if hasattr(process_HTS_New_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HTS_New_Positive_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function
        
        if df_HTS_New_Positive_gap is None:
            None

        # -- Step 9: Style the DataFrame
        df_HTS_New_Positive_style = (                # -- Apply styling to filtered DataFrame
            df_HTS_New_Positive_gap.style            # -- Start with DataFrame style object
            .hide(axis='index')                      # -- Hide index column
            .map(outlier_red_LT0, subset=HTS_New_Positive_columns_wrap)  # -- Highlight outliers in gap columns
        )

        # -- Step 10: Cache styled DataFrame and shape
        process_HTS_New_Positive_gap.cached_style = df_HTS_New_Positive_style  # -- Cache styled DataFrame
        process_HTS_New_Positive_gap.cached_shape = df_HTS_New_Positive.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 11: Define export variables
        report_month = df_HTS_New_Positive['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"  # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                   # -- Define Excel sheet name

        # -- Step 12: Create descriptions for Word document
        report_description = []
        if (df_HTS_New_Positive[HTS_New_Positive_gap_columns_wrap[0]] < 0).any():  # -- Check if HTS positive gap exists
            report_description.append(                        # -- Add description for HTS positive gap
                f"Report Name: {HTS_New_Positive_gap_columns[0]}\n"
                f"{HTS_New_Positive_columns[6]}\nshould not be less than 0"
            )
        if (df_HTS_New_Positive[HTS_New_Positive_columns_wrap[1]] != 0).any() or (df_HTS_New_Positive[HTS_New_Positive_columns_wrap[3]] != 0).any():  # -- Check if HTS community testing gap exists
            report_description.append(                        # -- Add description for HTS community testing gap
                f"Report Name: {Gap_title_special[0]}\n"
                f"{HTS_New_Positive_columns[1]}\n"
                f"plus {HTS_New_Positive_columns[3]} should not be greater than 0"
            )
        report_description = "\n\n".join(report_description)  # -- Join descriptions

        # -- Step 13: Export results
        export_df_to_doc_image_excel(                     # -- Export DataFrame to multiple formats
            report_name=report_name,                      # -- Pass report name
            df_style=df_HTS_New_Positive_style,           # -- Pass styled DataFrame
            img_file_name=report_image_name,              # -- Pass image file name
            img_file_path=report_image_path,              # -- Pass image file path
            doc_file_path=doc_file_msf_outlier_docx,      # -- Pass Word document path
            doc_description=report_description,           # -- Pass description
            doc_indicators_to_italicize=HTS_New_Positive_columns,  # -- Italicize HTS columns
            doc_indicators_to_underline=HTS_New_Positive_gap_columns,  # -- Underline gap columns
            xlm_file_path=doc_file_msf_outlier_xlsx,      # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name              # -- Pass Excel sheet name
        )

        # -- Step 14: Optionally display styled DataFrame
        if display_output:                                # -- Check if display is requested
            widget_display_df(df_HTS_New_Positive_style)  # -- Display styled DataFrame

    except Exception as e:                                # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_HTS_New_Positive_gap, 'cached_style'):  # -- Check if cache exists
            del process_HTS_New_Positive_gap.cached_style  # -- Clear cached style
        if hasattr(process_HTS_New_Positive_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HTS_New_Positive_gap.cached_shape  # -- Clear cached shape
        return                                            # -- Exit function#
    # -- End of function

#### - HTS TB Screening

In [ ]:
# -- Define the main function to process HTS TB Screening gap
def process_HTS_TB_Screening_gap(display_output=None):
    """
    Process HTS TB Screening gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_TB_Screening_columns = [                     # -- Define list of HTS TB Screening columns
            "Number of people who tested HIV negative and received their results. (Inpatient)",
            "Number of people who tested HIV negative and received their results. (Outpatient)",
            "Number of people who tested HIV negative and received their results. (Standalone)",
            "Number of people who tested HIV negative and received their results. (Community)",
            "Number of people who tested HIV positive and received results (Inpatient)",
            "Number of people who tested HIV positive and received results (Outpatient)",
            "Number of people who tested HIV positive and received results (Standalone)",
            "Number of people who tested HIV positive and received results (Community)",
            "Number of HTS clients clinically screened for TB (Inpatient)",
            "Number of HTS clients clinically screened for TB (Outpatient)",
            "Number of HTS clients clinically screened for TB (Standalone)",
            "Number of HTS clients clinically screened for TB (Community)"
        ]
        HTS_TB_Screening_columns_spec = [                # -- Define specific columns for summary
            "Number of people who tested HIV negative and received their results. (Inpatient, Outpatient, Standalone)",
            "Number of HTS clients clinically screened for TB (Inpatient, Outpatient, Standalone)"
        ]
        name = "HTS TB Screening gap"                   # -- Define general name
        Gap_columns = ["HTS TB screening gap"]          # -- Define gap column name
        report_name = f"{name}12"                       # -- Define report name
        No_gap_msg = f"No {report_name}"                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HTS_TB_Screening = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,            # -- Use MSF hierarchy columns
            data_columns=HTS_TB_Screening_columns       # -- Include specified HTS TB Screening columns
        )
        if df_HTS_TB_Screening is None:                 # -- Check if data preparation failed
            return                                      # -- Exit function if no data

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: Calculate total HTS testing
        df_HTS_TB_Screening["HTS total tested"] = (  
            df_HTS_TB_Screening.iloc[:, 4:12].sum(axis=1)  # -- Sum columns for total HTS testing
        )

        # -- Step 3.2: Calculate total HTS TB screening
        df_HTS_TB_Screening["HTS TB total screened"] = (  
            df_HTS_TB_Screening.iloc[:, 12:16].sum(axis=1)  # -- Sum columns for total HTS TB screening
        )

        # -- Step 3.3: Calculate HTS TB Screening gap
        df_HTS_TB_Screening[Gap_columns[0]] = np.where(  
            df_HTS_TB_Screening["HTS total tested"] != df_HTS_TB_Screening["HTS TB total screened"],  # -- Condition for gap
            df_HTS_TB_Screening["HTS TB total screened"] - df_HTS_TB_Screening["HTS total tested"],  # -- Gap value
            0                                               # -- Default to 0 if no gap
        )

        # -- Step 4: Drop original columns
        df_HTS_TB_Screening = df_HTS_TB_Screening.drop(
            columns=HTS_TB_Screening_columns  # -- Drop original detailed columns
        )

        wrap_column_headers(df_HTS_TB_Screening)  # -- Wrap column headers for better readability
        HTS_TB_Screening_columns_wrap = wrap_column_headers2(HTS_TB_Screening_columns)

        # -- Step 5: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_TB_Screening_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_HTS_TB_Screening.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)            # -- Print display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    display(process_HTS_TB_Screening_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                # -- Exit function

        # -- Step 6: Filter and validate gaps
        df_HTS_TB_Screening_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HTS_TB_Screening,                      # -- Input DataFrame
            msg=No_gap_msg,                              # -- Message for empty result
            opNonZero=Gap_columns,                       # -- Filter for non-zero gaps
            opNeg=None,                                  # -- No negative filter
            opPos=None,                                  # -- No positive filter
            opZero=None,                                 # -- No zero filter
            opLT100=None                                 # -- No less-than-100 filter
        )
        if df_HTS_TB_Screening_gap is None:              # -- Check if no gaps found
            if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
                del process_HTS_TB_Screening_gap.cached_style  # -- Clear cached style
            if hasattr(process_HTS_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HTS_TB_Screening_gap.cached_shape  # -- Clear cached shape
            return                                        # -- Exit function
        
        if df_HTS_TB_Screening_gap is None:                 # -- Check if data preparation failed
            return 

        # -- Step 7: Style the DataFrame
        df_HTS_TB_Screening_gap_style = (                # -- Apply styling to filtered DataFrame
            df_HTS_TB_Screening_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                          # -- Hide index column
            .map(outlier_red, subset=Gap_columns)        # -- Highlight outliers in gap column
        )

        # -- Step 8: Cache styled DataFrame and shape
        process_HTS_TB_Screening_gap.cached_style = df_HTS_TB_Screening_gap_style  # -- Cache styled DataFrame
        process_HTS_TB_Screening_gap.cached_shape = df_HTS_TB_Screening.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 9: Define export variables
        report_month = df_HTS_TB_Screening_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"         # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"     # -- Define image file path
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 10: Create descriptions for Word document
        if (df_HTS_TB_Screening_gap[Gap_columns[0]] != 0).any():       # -- Check if any gaps exist
            report_description = (                                     # -- Define report description
                f"Report Name: {Gap_columns[0]}\n"
                f"{HTS_TB_Screening_columns_spec[1]}\n"
                f"should be equal to {HTS_TB_Screening_columns_spec[0]}"
            )

        # -- Step 11: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame to multiple formats
            report_name=report_name,                   # -- Pass report name
            df_style=df_HTS_TB_Screening_gap_style,    # -- Pass styled DataFrame
            img_file_name=report_image_name,           # -- Pass image file name
            img_file_path=report_image_path,           # -- Pass image file path
            doc_description=report_description,        # -- Pass description
            doc_indicators_to_italicize=HTS_TB_Screening_columns_spec,  # -- Italicize specific columns
            doc_indicators_to_underline=Gap_columns,   # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name           # -- Pass Excel sheet name
        )

        # -- Step 12: Optionally display styled DataFrame
        if display_output:                             # -- Check if display is requested
            widget_display_df(df_HTS_TB_Screening_gap_style)  # -- Display styled DataFrame

    except Exception as e:                             # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_HTS_TB_Screening_gap, 'cached_style'):  # -- Check if cache exists
            del process_HTS_TB_Screening_gap.cached_style  # -- Clear cached style
        if hasattr(process_HTS_TB_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HTS_TB_Screening_gap.cached_shape  # -- Clear cached shape
        return                                          # -- Exit function
    # -- End of function

#### - HTS Positive Enrolment

In [ ]:
# -- Define the main function to process HTS Enrolment gap
def process_HTS_Enrolment_gap(display_output=None):
    """
    Process HTS Enrolment gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_Enrolment_columns = [                     # -- Define list of HTS Enrolment columns
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Inpatient)",
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Outpatient)",
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Standalone)",
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Community)"
        ]
        HTS_Enrolment_columns_spec = [                # -- Define specific columns for summary
            "Number of people tested HIV positive who are successfully enrolled in HIV Care (Inpatient, Outpatient, Standalone)",
            "HTS total tested - new positive (excluding previously known)"
        ]
        columns_to_keep = MSF_hierarchy + ["HTS total tested - new positive (excluding previously known)"]  # -- Columns to retain from Pre_MSF_positives_all
        Pre_MSF_positives_subset = Pre_MSF_positives_all[columns_to_keep]  # -- Subset of Pre_MSF_positives_all DataFrame
        name = "HTS Enrolment gap"                   # -- Define general name
        Gap_columns = ["HTS enrolment gap"]          # -- Define gap column name
        report_name = f"{name}13"                    # -- Define report name
        No_gap_msg = f"No {report_name}"             # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HTS_Enrolment = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,         # -- Use MSF hierarchy columns
            data_columns=HTS_Enrolment_columns       # -- Include specified HTS Enrolment columns
        )
        if df_HTS_Enrolment is None:                 # -- Check if data preparation failed
            return                                   # -- Exit function if no data

        # -- Step 3: Merge with Pre_MSF_positives_all subset
        df_HTS_Enrolment = Pre_MSF_positives_subset.merge(  # -- Merge with positives data
            df_HTS_Enrolment,                           # -- Merge target
            on=MSF_hierarchy,                           # -- Merge keys
            how="left"                                  # -- Keep all rows from Pre_MSF_positives_subset
        )

        # -- Step 4: Sort and clean data
        df_HTS_Enrolment.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # -- Sort by hierarchy columns
        df_HTS_Enrolment = df_HTS_Enrolment.fillna(0)  # -- Fill NaN with 0
        float_columns = df_HTS_Enrolment.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                     # -- Convert float columns to int
            df_HTS_Enrolment[col] = df_HTS_Enrolment[col].astype(int)

        # -- Step 5: Calculate derived metrics
        # -- Step 5.1: Calculate total enrolment
        df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] = (  
            df_HTS_Enrolment.iloc[:, 5:9].sum(axis=1)  # -- Sum columns for total enrolment
        )

        # -- Step 5.2: Calculate enrolment gap
        df_HTS_Enrolment[Gap_columns[0]] = np.where(  
            df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] != df_HTS_Enrolment[HTS_Enrolment_columns_spec[1]],  # -- Condition for gap
            df_HTS_Enrolment[HTS_Enrolment_columns_spec[0]] - df_HTS_Enrolment[HTS_Enrolment_columns_spec[1]],  # -- Gap value
            0                                               # -- Default to 0 if no gap
        )

        # -- Step 6: Drop original columns
        df_HTS_Enrolment = df_HTS_Enrolment.drop(
            columns=HTS_Enrolment_columns  # -- Drop original detailed columns
        )

        # -- Step 7: Wrap column headers for better readability
        wrap_column_headers(df_HTS_Enrolment)  
        HTS_Enrolment_columns_wrap = wrap_column_headers2(HTS_Enrolment_columns)

        # -- Step 8: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_HTS_Enrolment_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_Enrolment_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_HTS_Enrolment.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)            # -- Print display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    display(process_HTS_Enrolment_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                # -- Exit function

        # -- Step 9: Filter and validate gaps
        df_HTS_Enrolment_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HTS_Enrolment,                      # -- Input DataFrame
            msg=No_gap_msg,                           # -- Message for empty result
            opNonZero=Gap_columns,                    # -- Filter for non-zero gaps
            opNeg=None,                               # -- No negative filter
            opPos=None,                               # -- No positive filter
            opZero=None,                              # -- No zero filter
            opLT100=None                              # -- No less-than-100 filter
        )
        if df_HTS_Enrolment_gap is None:              # -- Check if no gaps found
            if hasattr(process_HTS_Enrolment_gap, 'cached_style'):  # -- Check if cache exists
                del process_HTS_Enrolment_gap.cached_style  # -- Clear cached style
            if hasattr(process_HTS_Enrolment_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HTS_Enrolment_gap.cached_shape  # -- Clear cached shape
            return                                     # -- Exit function
        
        if df_HTS_Enrolment_gap is None:                 # -- Check if data preparation failed
            return 

        # -- Step 10: Style the DataFrame
        df_HTS_Enrolment_gap_style = (                # -- Apply styling to filtered DataFrame
            df_HTS_Enrolment_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                       # -- Hide index column
            .map(outlier_red, subset=Gap_columns)     # -- Highlight outliers in gap column
        )

        # -- Step 11: Cache styled DataFrame and shape
        process_HTS_Enrolment_gap.cached_style = df_HTS_Enrolment_gap_style  # -- Cache styled DataFrame
        process_HTS_Enrolment_gap.cached_shape = df_HTS_Enrolment.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 12: Define export variables
        report_month = df_HTS_Enrolment_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"      # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define image file path
        report_sheet_name = report_name                              # -- Define Excel sheet name

        # -- Step 13: Create descriptions for Word document
        if (df_HTS_Enrolment_gap[Gap_columns[0]] != 0).any():       # -- Check if any gaps exist
            report_description = (                                 # -- Define report description
                f"Report Name: {Gap_columns[0]}\n"
                f"{HTS_Enrolment_columns_spec[1]}\n"
                f"should be equal to {HTS_Enrolment_columns_spec[0]}"
            )

        # -- Step 14: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame to multiple formats
            report_name=report_name,                   # -- Pass report name
            df_style=df_HTS_Enrolment_gap_style,       # -- Pass styled DataFrame
            img_file_name=report_image_name,           # -- Pass image file name
            img_file_path=report_image_path,           # -- Pass image file path
            doc_description=report_description,        # -- Pass description
            doc_indicators_to_italicize=HTS_Enrolment_columns_spec,  # -- Italicize specific columns
            doc_indicators_to_underline=Gap_columns,   # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name           # -- Pass Excel sheet name
        )

        # -- Step 15: Optionally display styled DataFrame
        if display_output:                             # -- Check if display is requested
            widget_display_df(df_HTS_Enrolment_gap_style)  # -- Display styled DataFrame

    except Exception as e:                             # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_HTS_Enrolment_gap, 'cached_style'):  # -- Check if cache exists
            del process_HTS_Enrolment_gap.cached_style  # -- Clear cached style
        if hasattr(process_HTS_Enrolment_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HTS_Enrolment_gap.cached_shape  # -- Clear cached shape
        return                                          # -- Exit function
    # -- End of function

#### - HTS Couple Counselling

In [ ]:
# -- Define the main function to process HTS Couple Counselling gap
def process_HTS_Couple_Counselling_gap(display_output=None):
    """
    Process HTS Couple Counselling gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_Couple_Counselling_columns = [                     # -- Define list of HTS Couple Counselling columns
            "No of couples counselled, tested for HIV and received result (Inpatient)",
            "No of couples counselled, tested for HIV and received result (Outpatient)",
            "No of couples counselled, tested for HIV and received result (Standalone)",
            "No of couples counselled, tested for HIV and received result (Community)",
            "No of couples counselled, tested for HIV and received discordant result (Inpatient)",
            "No of couples counselled, tested for HIV and received discordant result (Outpatient)",
            "No of couples counselled, tested for HIV and received discordant result (Standalone)",
            "No of couples counselled, tested for HIV and received discordant result (Community)"
        ]
        HTS_Couple_Counselling_columns_spec = [                # -- Define specific columns for summary
            "No of couples counselled, tested for HIV and received result (Inpatient, Outpatient, Standalone)",
            "No of couples counselled, tested for HIV and received discordant result (Inpatient, Outpatient, Standalone)"
        ]
        name = "HTS Discordant Couple Test gap"                # -- Define general name
        Gap_columns = ["HTS Discordant Couple Test gap"]       # -- Define gap column name
        report_name = f"{name}14"                              # -- Define report name
        No_gap_msg = f"No {report_name}"                       # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HTS_Couple_Counselling = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                         # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                  # -- Use MSF hierarchy columns
            data_columns=HTS_Couple_Counselling_columns        # -- Include specified HTS Couple Counselling columns
        )
        if df_HTS_Couple_Counselling is None:                 # -- Check if data preparation failed
            return                                            # -- Exit function if no data

        # -- Step 3: Calculate derived metrics
        # -- Step 3.1: Calculate total couples counselled and tested
        df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]] = (  
            df_HTS_Couple_Counselling.iloc[:, 4:8].sum(axis=1)  # -- Sum columns for total couples counselled and tested
        )

        # -- Step 3.2: Calculate total discordant results
        df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]] = (  
            df_HTS_Couple_Counselling.iloc[:, 8:12].sum(axis=1)  # -- Sum columns for total discordant results
        )

        # -- Step 3.3: Calculate discordant couple test gap
        df_HTS_Couple_Counselling[Gap_columns[0]] = np.where(  
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]] > df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]],  # -- Condition for gap
            df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[0]] - df_HTS_Couple_Counselling[HTS_Couple_Counselling_columns_spec[1]],  # -- Gap value
            0                                                  # -- Default to 0 if no gap
        )

        # -- Step 4: Drop original columns
        df_HTS_Couple_Counselling = df_HTS_Couple_Counselling.drop(
            columns=HTS_Couple_Counselling_columns  # -- Drop original detailed columns
        )

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HTS_Couple_Counselling)  
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(HTS_Couple_Counselling_columns)
        Gap_columns_wrap = wrap_column_headers2(Gap_columns)

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display is requested
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HTS_Couple_Counselling_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_HTS_Couple_Counselling.shape  # -- Get current unfiltered shape
                if cached_shape == current_shape:          # -- Compare shapes
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Define display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    print(cached_display_name)             # -- Print display message
                    print("-" * len(cached_display_name))  # -- Print separator line
                    display(process_HTS_Couple_Counselling_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                 # -- Exit function

        # -- Step 7: Filter and validate gaps
        df_HTS_Couple_Counselling_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HTS_Couple_Counselling,                      # -- Input DataFrame
            msg=No_gap_msg,                                    # -- Message for empty result
            opNonZero=Gap_columns_wrap,                        # -- Filter for non-zero gaps
            opNeg=None,                                        # -- No negative filter
            opPos=None,                                        # -- No positive filter
            opZero=None,                                       # -- No zero filter
            opLT100=None                                       # -- No less-than-100 filter
        )
        if df_HTS_Couple_Counselling_gap is None:              # -- Check if no gaps found
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # -- Check if cache exists
                del process_HTS_Couple_Counselling_gap.cached_style  # -- Clear cached style
            if hasattr(process_HTS_Couple_Counselling_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HTS_Couple_Counselling_gap.cached_shape  # -- Clear cached shape
            return                                             # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HTS_Couple_Counselling_gap_style = (                # -- Apply styling to filtered DataFrame
            df_HTS_Couple_Counselling_gap.style                # -- Start with DataFrame style object
            .hide(axis='index')                                # -- Hide index column
            .map(outlier_red, subset=Gap_columns)              # -- Highlight outliers in gap column
        )

        # -- Step 9: Cache styled DataFrame and shape
        process_HTS_Couple_Counselling_gap.cached_style = df_HTS_Couple_Counselling_gap_style  # -- Cache styled DataFrame
        process_HTS_Couple_Counselling_gap.cached_shape = df_HTS_Couple_Counselling.shape  # -- Cache unfiltered DataFrame shape

        # -- Step 10: Define export variables
        report_month = df_HTS_Couple_Counselling_gap['ReportPeriod'].iloc[0]  # -- Extract report month from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"               # -- Define image file name
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"           # -- Define image file path
        report_sheet_name = report_name                                       # -- Define Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_HTS_Couple_Counselling_gap[Gap_columns[0]] != 0).any():        # -- Check if any gaps exist
            report_description = (                                           # -- Define report description
                f"Report Name: {Gap_columns[0]}\n"
                f"{HTS_Couple_Counselling_columns_spec[1]}\n"
                f"should be equal to {HTS_Couple_Counselling_columns_spec[0]}"
            )

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                  # -- Export DataFrame to multiple formats
            report_name=report_name,                   # -- Pass report name
            df_style=df_HTS_Couple_Counselling_gap_style,  # -- Pass styled DataFrame
            img_file_name=report_image_name,           # -- Pass image file name
            img_file_path=report_image_path,           # -- Pass image file path
            doc_description=report_description,        # -- Pass description
            doc_indicators_to_italicize=HTS_Couple_Counselling_columns_spec,  # -- Italicize specific columns
            doc_indicators_to_underline=Gap_columns,   # -- Underline gap column
            xlm_file_path=doc_file_msf_outlier_xlsx,   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name           # -- Pass Excel sheet name
        )

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                             # -- Check if display is requested
            widget_display_df(df_HTS_Couple_Counselling_gap_style)  # -- Display styled DataFrame

    except Exception as e:                             # -- Catch any exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error message
        if hasattr(process_HTS_Couple_Counselling_gap, 'cached_style'):  # -- Check if cache exists
            del process_HTS_Couple_Counselling_gap.cached_style  # -- Clear cached style
        if hasattr(process_HTS_Couple_Counselling_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HTS_Couple_Counselling_gap.cached_shape  # -- Clear cached shape
        return                                          # -- Exit function
    # -- End of function

#### - HTS CD4 Test

In [ ]:
# -- Define the main function to process HTS CD4 gap
def process_HTS_CD4_gap(display_output=None):
    """
    Process HTS CD4 gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        HTS_CD4_columns = [                     # -- Define list of HTS CD4 columns
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Inpatient)",
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Outpatient)",
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0 (Standalone)",
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period\xa0\xa0(Community)",
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Inpatient)",
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Outpatient)",
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Standalone)",
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period\xa0\xa0(Community)"
        ]
        HTS_CD4_columns_spec = [                # -- Define specific columns for summary
            "Number of newly diagnosed PLHIV who received CD4 <200 cells/mm3 test during the reporting period  (Inpatient, Outpatient, Standalone)",
            "Number of newly diagnosed PLHIV who received CD4 >200 cells/mm3 test during the reporting period  (Inpatient, Outpatient, Standalone)",
            "HTS total tested - new positive (excluding previously known)",
            "Total Number of newly diagnosed PLHIV who received CD4 (<200 and >200) cells/mm3"
        ]
        columns_to_keep = MSF_hierarchy + ["HTS total tested - new positive (excluding previously known)"]  # -- Columns to retain from Pre_MSF_positives_all
        Pre_MSF_positives_subset = Pre_MSF_positives_all[columns_to_keep]  # -- Subset of Pre_MSF_positives_all DataFrame
        name = "HTS CD4 gap"                   # -- Define general name
        Gap_columns = ["HTS CD4 gap"]          # -- Define gap column name
        report_name = f"{name}15"              # -- Define report name
        No_gap_msg = f"No {report_name}"       # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HTS_CD4 = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",           # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,    # -- Use MSF hierarchy columns
            data_columns=HTS_CD4_columns        # -- Include specified HTS CD4 columns
        )
        if df_HTS_CD4 is None:                  # -- Check if data preparation failed
            return                              # -- Exit function if no data

        # -- Step 3: Merge with Pre_MSF_positives_all subset
        df_HTS_CD4 = Pre_MSF_positives_subset.merge(  # -- Merge with positives data
            df_HTS_CD4,                               # -- Merge target
            on=MSF_hierarchy,                         # -- Merge keys
            how="left"                                # -- Keep all rows from Pre_MSF_positives_subset
        )

        # -- Step 4: Sort and clean data
        df_HTS_CD4.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # -- Sort by hierarchy columns
        df_HTS_CD4 = df_HTS_CD4.fillna(0)  # -- Fill NaN with 0
        float_columns = df_HTS_CD4.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                     # -- Convert float columns to int
            df_HTS_CD4[col] = df_HTS_CD4[col].astype(int)

        # -- Step 5: Calculate derived metrics
        # -- Step 5.1: Calculate total CD4 <200
        df_HTS_CD4[HTS_CD4_columns_spec[0]] = (  
            df_HTS_CD4.iloc[:, 5:9].sum(axis=1)  # -- Sum columns for total CD4 <200
        )

        # -- Step 5.2: Calculate total CD4 >200
        df_HTS_CD4[HTS_CD4_columns_spec[1]] = (  
            df_HTS_CD4.iloc[:, 9:13].sum(axis=1)  # -- Sum columns for total CD4 >200
        )

        # -- Step 5.3: Calculate total CD4 <200 and >200
        df_HTS_CD4[HTS_CD4_columns_spec[3]] = (  
            df_HTS_CD4[HTS_CD4_columns_spec[0]] + df_HTS_CD4[HTS_CD4_columns_spec[1]]  # -- Sum columns for total CD4
        )

        # -- Step 5.4: Calculate CD4 gap
        df_HTS_CD4[Gap_columns[0]] = np.where(  
            df_HTS_CD4[HTS_CD4_columns_spec[2]] != df_HTS_CD4[HTS_CD4_columns_spec[3]],  # -- Condition for gap
            df_HTS_CD4[HTS_CD4_columns_spec[2]] - df_HTS_CD4[HTS_CD4_columns_spec[3]],  # -- Gap value
            0                                               # -- Default to 0 if no gap
        )

        # -- Step 6: Drop original columns
        columns_to_drop = HTS_CD4_columns + [HTS_CD4_columns_spec[0], HTS_CD4_columns_spec[1]]
        df_HTS_CD4 = df_HTS_CD4.drop(
            columns=columns_to_drop  # -- Drop original detailed columns
        )

        # -- Step 7: Wrap column headers for better readability
        wrap_column_headers(df_HTS_CD4)  
        HTS_CD4_columns_wrap = wrap_column_headers2(HTS_CD4_columns_spec)

        # -- Step 8: Check and display cached styled DataFrames
        if display_output:                                 # -- Check if display requested
            if hasattr(process_HTS_CD4_gap, 'cached_styles'):  # -- Check cache existence
                cached_shape = getattr(process_HTS_CD4_gap, 'cached_shape', None)  # -- Get cached shape
                current_shape = df_HTS_CD4.shape  # -- Get current shape
                if cached_shape == current_shape:          # -- Compare shapes
                    for cluster, style in process_HTS_CD4_gap.cached_styles.items():  # -- Iterate cached styles
                        cached_display_name = f"✔️ Displaying {cluster} {report_name} "  # -- Display message
                        print("-" * len(cached_display_name))  # -- Separator
                        print(cached_display_name)            # -- Message
                        print("-" * len(cached_display_name))  # -- Separator
                        display(style)                        # -- Display cached style
                    return                                 # -- Exit if cache used

        # -- Step 9: Initialize cache
        if not hasattr(process_HTS_CD4_gap, 'cached_styles'):  # -- Check if cache exists
            process_HTS_CD4_gap.cached_styles = {}  # -- Initialize cache

        # -- Step 10: Identify unique clusters
        cluster_list = pd.Series(df_HTS_CD4['Cluster'].unique())  # -- Extract unique clusters

        # -- Step 11: Process each cluster
        for current_cluster in cluster_list:               # -- Iterate over clusters
            cluster_filtered = df_HTS_CD4[df_HTS_CD4['Cluster'] == current_cluster]  # -- Filter cluster
            
            HTS_CD4_msg = f"No {current_cluster} {report_name}"  # -- Message for no gaps

            cluster_filtered_gap = filter_gap_and_check_empty_df(  # -- Filter gaps
                df=cluster_filtered,
                msg=HTS_CD4_msg,
                opNonZero=Gap_columns,
                opNeg=None,
                opPos=None,
                opZero=None,
                opLT100=None
            )

            if cluster_filtered_gap is None:               # -- Check if no gaps
                if current_cluster in process_HTS_CD4_gap.cached_styles:  # -- Remove from cache
                    del process_HTS_CD4_gap.cached_styles[current_cluster]
                continue                                   # -- Skip cluster

            cluster_filtered_style = (                     # -- Style DataFrame
                cluster_filtered_gap.style
                .hide(axis='index')
                .map(outlier_red, subset=Gap_columns)
            )

            process_HTS_CD4_gap.cached_styles[current_cluster] = cluster_filtered_style  # -- Cache style

            # -- Step 12: Define export variables
            report_name_cluster = f"{current_cluster}_{report_name}"  # -- Cluster-specific report name
            report_month = cluster_filtered_gap['ReportPeriod'].iloc[0]  # -- Extract report month
            report_image_name = f"{report_month}_{report_name_cluster}.png"  # -- Image file name
            report_sheet_name = f"{current_cluster}_{report_name}"  # -- Excel sheet name

            # -- Step 13: Create descriptions
            report_description = []                        # -- Initialize descriptions
            if (cluster_filtered_gap[Gap_columns[0]] != 0).any():  # -- Check for gaps
                report_description.append(
                    f"Report Name: {Gap_columns[0]}\n"
                    f"{HTS_CD4_columns_spec[2]}\nshould be equal to {HTS_CD4_columns_spec[3]}\n"
                    f"Note: Where this CD4 gap is true, please ignore the outlier."
                )
            report_description = "\n\n".join(report_description)  # -- Join descriptions

            # -- Step 14: Export results
            export_df_to_doc_image_excel(                  # -- Export DataFrame
                report_name=report_name_cluster,
                df_style=cluster_filtered_style,
                img_file_name=report_image_name,
                img_file_path=sub_folder2_image_file_msf_outlier,
                doc_description=report_description,
                doc_indicators_to_italicize=HTS_CD4_columns_spec,
                doc_indicators_to_underline=Gap_columns,
                xlm_file_path=doc_file_msf_outlier_xlsx,
                xlm_sheet_name=report_sheet_name
            )

            if display_output:                             # -- Display styled DataFrame
                widget_display_df(cluster_filtered_style)

        # -- Step 15: Cache overall unfiltered DataFrame shape
        process_HTS_CD4_gap.cached_shape = df_HTS_CD4.shape  # -- Cache shape

    except Exception as e:                                 # -- Catch exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error
        if hasattr(process_HTS_CD4_gap, 'cached_styles'):  # -- Clear cache on error
            process_HTS_CD4_gap.cached_styles.clear()
        if hasattr(process_HTS_CD4_gap, 'cached_shape'):  # -- Clear shape on error
            del process_HTS_CD4_gap.cached_shape
        return

### HIVST MSF

#### - HIVST Distribution Mode

In [ ]:
# -- Define the main function to process HIVST Distribution Mode gap
def process_HIVST_Distr_Mode_gap(display_output=None):
    """
    Process HIVST Distribution Mode gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:
        # -- Step 1: Initialize constants
        df_columns = [                     # -- Define list of HIVST columns
            "Number of individual HIVST kits distributed - Assisted (Distribution By)",
            "Number of individual HIVST kits distributed - Uassisted (Distribution By)"
        ]
        df_columns_spec = [                # -- Define specific columns for summary
            "Number of individual HIVST kits distributed - Assisted",
            "Number of individual HIVST kits distributed - Uassisted",
            "Number of individual HIVST kits distributed - Assisted (Distribution By) (Distribution Mode)",
            "Number of individual HIVST kits distributed - Uassisted (Distribution By) (Distribution Mode)"
        ]
        columns_to_keep = MSF_hierarchy + ["Assisted", "Unassisted"]  # -- Columns to retain from HTS MSF_hivst approach
        name = "HIVST Distribution Mode gap"                   # -- Define general name for reporting
        gap_columns = ["Assisted distribution mode gap", "Uassisted distribution mode gap"]  # -- Define gap column names
        report_name = f"{name}16"                    # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"             # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_HIVST_Distri_Mode = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,         # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                  # -- Include specified HTS Enrolment columns
        )
        if df_HIVST_Distri_Mode is None:                 # -- Check if data preparation failed
            print(f"⦸ No data retrieved for {report_name}")  # -- Notify user of data failure
            return                                   # -- Exit function if no data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_HIVST_Mode_extra = prepare_and_convert_df(    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF_hivst approach",     # -- Specify DHIS2 data key for HIVST approach
            hierarchy_columns=MSF_hierarchy,             # -- Use MSF hierarchy columns
            data_columns=["Assisted", "Unassisted"]      # -- Include specified HIVST mode columns
        )
        if df_HIVST_Mode_extra is None:                 # -- Check if data preparation failed
            print(f"⦸ No extra data retrieved for {report_name}")  # -- Notify user of data failure
            return                                      # -- Exit function if no data

        df_HIVST_Mode_extra = df_HIVST_Mode_extra[columns_to_keep]  # -- Subset to retain only necessary columns

        # -- Step 4: Merge datasets
        df_HIVST_Distri_Mode = df_HIVST_Mode_extra.merge(  # -- Merge HIVST mode data with primary data
            df_HIVST_Distri_Mode,                          # -- Target DataFrame for merge
            on=MSF_hierarchy,                              # -- Merge on MSF hierarchy columns
            how="right"                                    # -- Right join to keep all rows from primary data
        )

        # -- Step 5: Rename columns for consistency
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode.rename(columns={
            "Assisted": f"{df_columns_spec[0]}",
            "Unassisted": f"{df_columns_spec[1]}",
            "Number of individual HIVST kits distributed - Assisted (Distribution By)": f"{df_columns_spec[2]}",
            "Number of individual HIVST kits distributed - Uassisted (Distribution By)": f"{df_columns_spec[3]}"
        })  # -- Rename columns to align with specified column names

        # -- Step 6: Clean and format data
        df_HIVST_Distri_Mode.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # -- Sort by hierarchy
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode.fillna(0)  # -- Replace NaN with 0
        float_columns = df_HIVST_Distri_Mode.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                     # -- Convert float columns to integers
            df_HIVST_Distri_Mode[col] = df_HIVST_Distri_Mode[col].astype(int)

        # -- Step 7: Calculate gaps
        df_HIVST_Distri_Mode[gap_columns[0]] = np.where(
            df_HIVST_Distri_Mode[df_columns_spec[2]] != df_HIVST_Distri_Mode[df_columns_spec[0]],  # -- Check for Assisted gap
            df_HIVST_Distri_Mode[df_columns_spec[2]] - df_HIVST_Distri_Mode[df_columns_spec[0]],  # -- Calculate gap
            0                                               # -- Set to 0 if no gap
        )
        df_HIVST_Distri_Mode[gap_columns[1]] = np.where(
            df_HIVST_Distri_Mode[df_columns_spec[3]] != df_HIVST_Distri_Mode[df_columns_spec[1]],  # -- Check for Unassisted gap
            df_HIVST_Distri_Mode[df_columns_spec[3]] - df_HIVST_Distri_Mode[df_columns_spec[1]],  # -- Calculate gap
            0                                               # -- Set to 0 if no gap
        )

        # -- Step 8: Reorder columns for output
        reorder_columns = MSF_hierarchy + [
            df_columns_spec[0], df_columns_spec[2], gap_columns[0], 
            df_columns_spec[1], df_columns_spec[3], gap_columns[1]
        ]
        df_HIVST_Distri_Mode = df_HIVST_Distri_Mode[reorder_columns]  # -- Reorder DataFrame columns

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Distri_Mode)  # -- Apply wrapping to DataFrame headers
        df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)  # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)  # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                 # -- Check if display output is requested
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_HIVST_Distr_Mode_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_HIVST_Distri_Mode.shape  # -- Get current DataFrame shape
                if cached_shape == current_shape:          # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))  # -- Print separator
                    print(cached_display_name)            # -- Print message
                    print("-" * len(cached_display_name))  # -- Print separator
                    display(process_HIVST_Distr_Mode_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_HIVST_Distri_Mode_gap = filter_gap_and_check_empty_df(  # -- Filter rows with non-zero gaps
            df=df_HIVST_Distri_Mode,                      # -- Input DataFrame
            msg=No_gap_msg,                           # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                # -- Filter for non-zero gaps
            opNeg=None,                               # -- No negative value filter
            opPos=None,                               # -- No positive value filter
            opZero=None,                              # -- No zero value filter
            opLT100=None                              # -- No less-than-100 filter
        )
        if df_HIVST_Distri_Mode_gap is None:              # -- Check if no gaps were found
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_HIVST_Distr_Mode_gap.cached_style  # -- Remove cached style
            if hasattr(process_HIVST_Distr_Mode_gap, 'cached_shape'):  # -- Clear cached shape
                del process_HIVST_Distr_Mode_gap.cached_shape  # -- Remove cached shape
            print(f"⦸ {No_gap_msg}")                     # -- Notify user of no gaps
            return                                     # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_HIVST_Distri_Mode_gap_style = (                # -- Apply styling to filtered DataFrame
            df_HIVST_Distri_Mode_gap.style                # -- Create style object
            .hide(axis='index')                       # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)     # -- Highlight non-zero gaps in red
        )

        # -- Step 13: Cache styled DataFrame and shape
        process_HIVST_Distr_Mode_gap.cached_style = df_HIVST_Distri_Mode_gap_style  # -- Store styled DataFrame
        process_HIVST_Distr_Mode_gap.cached_shape = df_HIVST_Distri_Mode.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_HIVST_Distri_Mode_gap['ReportPeriod'].iloc[0]  # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"      # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"  # -- Define path for image export
        report_sheet_name = report_name                              # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        report_description = []                                    # -- Initialize list for descriptions
        if (df_HIVST_Distri_Mode_gap[gap_columns_wrap[0]] != 0).any():  # -- Check for Assisted gaps
            report_description.append(                             # -- Add description for Assisted gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\n"
                f"should be equal to {df_columns_spec[0]}"
            )
        if (df_HIVST_Distri_Mode_gap[gap_columns_wrap[1]] != 0).any():  # -- Check for Unassisted gaps
            report_description.append(                             # -- Add description for Unassisted gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns_spec[3]}\n"
                f"should be equal to {df_columns_spec[1]}"
            )
        report_description = "\n\n".join(report_description)  # -- Combine descriptions with newlines

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                  # -- Export styled DataFrame and descriptions
            report_name=report_name,                   # -- Report identifier
            df_style=df_HIVST_Distri_Mode_gap_style,   # -- Styled DataFrame for export
            img_file_name=report_image_name,           # -- Image file name
            img_file_path=report_image_path,           # -- Image file path
            doc_description=report_description,        # -- Description for Word document
            doc_indicators_to_italicize=df_columns_spec,  # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,   # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,   # -- Excel file path
            xlm_sheet_name=report_sheet_name           # -- Excel sheet name
        )

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                             # -- Check if display is requested
            widget_display_df(df_HIVST_Distri_Mode_gap_style)  # -- Display styled DataFrame in widget

    except Exception as e:                             # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")  # -- Print error details
        if hasattr(process_HIVST_Distr_Mode_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_HIVST_Distr_Mode_gap.cached_style  # -- Remove cached style
        if hasattr(process_HIVST_Distr_Mode_gap, 'cached_shape'):  # -- Clear cached shape
            del process_HIVST_Distr_Mode_gap.cached_shape  # -- Remove cached shape
        return                                          # -- Exit function on error
    # -- End of function

#### - HIVST Testing Frequency

In [ ]:
# -- Define the main function to process HIVST Testing Frequency gap
def process_HIVST_Test_Freq_gap(display_output=None):                    # -- Define function with optional display parameter
    """
    Process HIVST Testing Frequency gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST testing frequency columns
            "Number of individual HIVST kits distributed (Directly Assisted & Unassisted)",
            "Number of individual HIVST kits distributed (Testing Frequency)"
        ]
        name = "HIVST Testing Frequency gap"                            # -- Define general name
        gap_columns = ["HIVST Testing Frequency gap"]                   # -- Define gap column name
        report_name = f"{name}17"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HIVST_Test_Freq = prepare_and_convert_df(                    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_HIVST_Test_Freq is None:                                  # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3.3: Calculate discordant couple test gap
        df_HIVST_Test_Freq[gap_columns[0]] = np.where(  
            df_HIVST_Test_Freq[df_columns[1]] > df_HIVST_Test_Freq[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Test_Freq[df_columns[1]] - df_HIVST_Test_Freq[df_columns[0]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Test_Freq)                         # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):    # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Test_Freq_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_HIVST_Test_Freq.shape                # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_HIVST_Test_Freq_gap.cached_style)   # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_HIVST_Test_Freq_gap = filter_gap_and_check_empty_df(        # -- Filter DataFrame for gaps
            df=df_HIVST_Test_Freq,                                      # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_HIVST_Test_Freq_gap is None:                              # -- Check if no gaps found
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):    # -- Check if cached style exists
                del process_HIVST_Test_Freq_gap.cached_style            # -- Remove cached styled DataFrame
            if hasattr(process_HIVST_Test_Freq_gap, 'cached_shape'):    # -- Check if cached shape exists
                del process_HIVST_Test_Freq_gap.cached_shape            # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HIVST_Test_Freq_gap_style = (                                # -- Apply styling to filtered DataFrame
            df_HIVST_Test_Freq_gap.style                                # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Test_Freq_gap.cached_style = df_HIVST_Test_Freq_gap_style  # -- Store styled DataFrame in cache
        process_HIVST_Test_Freq_gap.cached_shape = df_HIVST_Test_Freq.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_HIVST_Test_Freq_gap['ReportPeriod'].iloc[0]  # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_HIVST_Test_Freq_gap[gap_columns_wrap[0]] != 0).any():   # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_HIVST_Test_Freq_gap_style,                     # -- Pass styled DataFrame
            img_file_name=report_image_name,                           # -- Pass image file name
            img_file_path=report_image_path,                           # -- Pass image file path
            doc_description=report_description,                        # -- Pass description
            doc_indicators_to_italicize=df_columns,                    # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                   # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                           # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_HIVST_Test_Freq_gap_style)            # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_HIVST_Test_Freq_gap, 'cached_style'):       # -- Check if cached style exists
            del process_HIVST_Test_Freq_gap.cached_style               # -- Remove cached styled DataFrame
        if hasattr(process_HIVST_Test_Freq_gap, 'cached_shape'):       # -- Check if cached shape exists
            del process_HIVST_Test_Freq_gap.cached_shape               # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                                      # -- End of function

#### - HIVST Result

In [ ]:
# -- Define the main function to process HIVST Result gap
def process_HIVST_Result_gap(display_output=None):                    # -- Define function with optional display parameter
    """
    Process HIVST Result gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST testing frequency columns
            "Number of individual HIVST kits distributed (Directly Assisted & Unassisted)",
            "Number of individual reporting HIVST results"
        ]
        name = "HIVST Result gap"                            # -- Define general name
        gap_columns = ["HIVST Result gap"]                   # -- Define gap column name
        report_name = f"{name}18"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HIVST_Result = prepare_and_convert_df(                    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_HIVST_Result is None:                                  # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3.3: Calculate discordant couple test gap
        df_HIVST_Result[gap_columns[0]] = np.where(  
            df_HIVST_Result[df_columns[1]] != df_HIVST_Result[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Result[df_columns[0]] - df_HIVST_Result[df_columns[1]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Result)                         # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_HIVST_Result_gap, 'cached_style'):    # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Result_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_HIVST_Result.shape                # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_HIVST_Result_gap.cached_style)   # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_HIVST_Test_Freq_gap = filter_gap_and_check_empty_df(        # -- Filter DataFrame for gaps
            df=df_HIVST_Result,                                      # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                         # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_HIVST_Test_Freq_gap is None:                              # -- Check if no gaps found
            if hasattr(process_HIVST_Result_gap, 'cached_style'):    # -- Check if cached style exists
                del process_HIVST_Result_gap.cached_style            # -- Remove cached styled DataFrame
            if hasattr(process_HIVST_Result_gap, 'cached_shape'):    # -- Check if cached shape exists
                del process_HIVST_Result_gap.cached_shape            # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HIVST_Test_Freq_gap_style = (                                # -- Apply styling to filtered DataFrame
            df_HIVST_Test_Freq_gap.style                                # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Result_gap.cached_style = df_HIVST_Test_Freq_gap_style  # -- Store styled DataFrame in cache
        process_HIVST_Result_gap.cached_shape = df_HIVST_Result.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_HIVST_Test_Freq_gap['ReportPeriod'].iloc[0]  # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_HIVST_Test_Freq_gap[gap_columns[0]] != 0).any():   # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_HIVST_Test_Freq_gap_style,                     # -- Pass styled DataFrame
            img_file_name=report_image_name,                           # -- Pass image file name
            img_file_path=report_image_path,                           # -- Pass image file path
            doc_description=report_description,                        # -- Pass description
            doc_indicators_to_italicize=df_columns,                    # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                   # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                           # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_HIVST_Test_Freq_gap_style)            # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_HIVST_Result_gap, 'cached_style'):       # -- Check if cached style exists
            del process_HIVST_Result_gap.cached_style               # -- Remove cached styled DataFrame
        if hasattr(process_HIVST_Result_gap, 'cached_shape'):       # -- Check if cached shape exists
            del process_HIVST_Result_gap.cached_shape               # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                                      # -- End of function

#### - HIVST Reactive Confirmation and Linkage

In [ ]:
# -- Define the main function to process HIVST Reactive and Linkage gap
def process_HIVST_Reactive_Link_gap(display_output=None):                    # -- Define function with optional display parameter
    """
    Process HIVST Reactive and Linkaget gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                    # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                      # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of individual reporting HIVST results",
            "Number of individuals reporting reactive HIVST results referred for confirmatory test (HTS)",
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV positive test results.",
            "Number of individuals reporting reactive HIVST results referred for confirmatory test(HTS) who received HIV negative test results.",
            "Number of individuals with confirmed HIV-positive results who are successfully linked with HIV care and treatment"
        ]
        name = "HIVST Reactive Linkage gap"                                 # -- Define general name
        gap_columns = [
            "HIVST Reactive Referral for Confirmatory gap", 
            "HIVST Confirmatory Test Rsult gap",
            "HIVST Confirmed Positive Linkage gap"
        ]                                                                   # -- Define gap column names
        report_name = f"{name}19"                                           # -- Define report name
        No_gap_msg = f"No {report_name}"                                    # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HIVST_Reactive_Link = prepare_and_convert_df(                    # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                       # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                                # -- Use MSF hierarchy columns
            data_columns=df_columns                                         # -- Include specified HTS columns
        )
        if df_HIVST_Reactive_Link is None:                                  # -- Check if data preparation failed
            return                                                          # -- Exit function if no data
        
        # -- Step 3.3: Calculate discordant couple test gap
        df_HIVST_Reactive_Link[gap_columns[0]] = np.where(  
            df_HIVST_Reactive_Link[df_columns[1]] > df_HIVST_Reactive_Link[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Reactive_Link[df_columns[1]] - df_HIVST_Reactive_Link[df_columns[0]],  # -- Calculate gap if condition met
            0                                                               # -- Set gap to 0 if no discrepancy
        )                                                                   # -- Assign calculated referral gap to new column

        df_HIVST_Reactive_Link[gap_columns[1]] = np.where(
            df_HIVST_Reactive_Link[df_columns[2:4]].sum(axis=1) != df_HIVST_Reactive_Link[df_columns[1]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Reactive_Link[df_columns[2:4]].sum(axis=1) - df_HIVST_Reactive_Link[df_columns[1]],  # -- Calculate gap if condition met
            0                                                               # -- Set gap to 0 if no discrepancy
        )                                                                   # -- Assign calculated confirmatory test gap to new column

        df_HIVST_Reactive_Link[gap_columns[2]] = np.where(
            df_HIVST_Reactive_Link[df_columns[2]] != df_HIVST_Reactive_Link[df_columns[4]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Reactive_Link[df_columns[4]] - df_HIVST_Reactive_Link[df_columns[2]],  # -- Calculate gap if condition met
            0                                                               # -- Set gap to 0 if no discrepancy
        )                                                                   # -- Assign calculated linkage gap to new column

        reorder_columns = MSF_hierarchy + [
            df_columns[0], df_columns[1], gap_columns[0], 
            df_columns[2], df_columns[3], gap_columns[1],
            df_columns[4], gap_columns[2]
        ]                                                                   # -- Define column order for DataFrame

        df_HIVST_Reactive_Link = df_HIVST_Reactive_Link[reorder_columns]    # -- Reorder DataFrame columns

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Reactive_Link)                         # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)                # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                                  # -- Check if display is requested
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):    # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Reactive_Link_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_HIVST_Reactive_Link.shape                # -- Get current DataFrame shape
                if cached_shape == current_shape:                           # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))                   # -- Print separator line above message
                    print(cached_display_name)                              # -- Print cached display message
                    print("-" * len(cached_display_name))                   # -- Print separator line below message
                    display(process_HIVST_Reactive_Link_gap.cached_style)   # -- Render cached styled DataFrame
                    return                                                  # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_HIVST_Reactive_Link_gap = filter_gap_and_check_empty_df(        # -- Filter DataFrame for gaps
            df=df_HIVST_Reactive_Link,                                      # -- Input DataFrame for filtering
            msg=No_gap_msg,                                                 # -- Message for empty result
            opNonZero=gap_columns_wrap,                                     # -- Filter for non-zero gaps
            opNeg=None,                                                     # -- No negative value filter
            opPos=None,                                                     # -- No positive value filter
            opZero=None,                                                    # -- No zero value filter
            opLT100=None                                                    # -- No less-than-100 filter
        )                                                                   # -- Store filtered DataFrame with gaps
        if df_HIVST_Reactive_Link_gap is None:                              # -- Check if no gaps found
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):    # -- Check if cached style exists
                del process_HIVST_Reactive_Link_gap.cached_style            # -- Remove cached styled DataFrame
            if hasattr(process_HIVST_Reactive_Link_gap, 'cached_shape'):    # -- Check if cached shape exists
                del process_HIVST_Reactive_Link_gap.cached_shape            # -- Remove cached shape
            return                                                          # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HIVST_Reactive_Link_gap_style = (                                # -- Apply styling to filtered DataFrame
            df_HIVST_Reactive_Link_gap.style                                # -- Create style object from filtered DataFrame
            .hide(axis='index')                                             # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                      # -- Highlight non-zero gaps in red
        )                                                                   # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Reactive_Link_gap.cached_style = df_HIVST_Reactive_Link_gap_style  # -- Store styled DataFrame in cache
        process_HIVST_Reactive_Link_gap.cached_shape = df_HIVST_Reactive_Link.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_HIVST_Reactive_Link_gap['ReportPeriod'].iloc[0]  # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"            # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"        # -- Specify path for image export
        report_sheet_name = report_name                                    # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        report_description = []                                             # -- Initialize report description
        if (df_HIVST_Reactive_Link_gap[gap_columns_wrap[0]] != 0).any():   # -- Check if any non-zero gaps exist
            report_description.append(                                       # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"
            )                                                               # -- Add referral gap description
        if (df_HIVST_Reactive_Link_gap[gap_columns_wrap[1]] != 0).any():   # -- Check if any non-zero gaps exist
            report_description.append(                                       # -- Define report description
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\nplus {df_columns[3]}\n"
                f"should be equal to {df_columns[1]}"
            )                                                               # -- Add confirmatory test gap description
        if (df_HIVST_Reactive_Link_gap[gap_columns_wrap[2]] != 0).any():   # -- Check if any non-zero gaps exist
            report_description.append(                                       # -- Define report description
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns[4]}\n"
                f"should be equal to {df_columns[2]}"
            )                                                               # -- Add linkage gap description
        report_description = "\n\n".join(report_description)                # -- Join all descriptions into a single string

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                       # -- Export DataFrame to multiple formats
            report_name=report_name,                                        # -- Pass report name
            df_style=df_HIVST_Reactive_Link_gap_style,                     # -- Pass styled DataFrame
            img_file_name=report_image_name,                               # -- Pass image file name
            img_file_path=report_image_path,                               # -- Pass image file path
            doc_description=report_description,                            # -- Pass description
            doc_indicators_to_italicize=df_columns,                        # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                       # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                       # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                               # -- Pass Excel sheet name
        )                                                                   # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                                  # -- Check if display is requested
            widget_display_df(df_HIVST_Reactive_Link_gap_style)             # -- Render styled DataFrame in widget

    except Exception as e:                                                  # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")               # -- Log general error
        if hasattr(process_HIVST_Reactive_Link_gap, 'cached_style'):        # -- Check if cached style exists
            del process_HIVST_Reactive_Link_gap.cached_style                # -- Remove cached styled DataFrame
        if hasattr(process_HIVST_Reactive_Link_gap, 'cached_shape'):        # -- Check if cached shape exists
            del process_HIVST_Reactive_Link_gap.cached_shape                # -- Remove cached shape
        return                                                             # -- Exit function on general error
    # -- End of function                                                                                  # -- End of function

#### - HIVST Prevention Service

In [ ]:
# -- Define the main function to process HIVST Prevention Service gap
def process_HIVST_Prevention_Serv_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process HIVST Prevention Service gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of individual reporting HIVST results",
            "Number of individuals reporting non reactive HIVST results that referred prevention services.",
            "Number of individuals reporting non reactive HIVST results that accessed prevention services"
        ]
        name = "HIVST Prevention Service gap"                           # -- Define general name
        gap_columns = [
            "HIVST Non-Reactive Referral for Prevention Services gap", 
            "HIVST Non-Reactive Client that Accessed for Prevention Services gap"
        ]                                                               # -- Define gap column names
        report_name = f"{name}20"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HIVST_Prevention_Serv = prepare_and_convert_df(              # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_HIVST_Prevention_Serv is None:                            # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_HIVST_Prevention_Serv[gap_columns[0]] = np.where(  
            df_HIVST_Prevention_Serv[df_columns[1]] > df_HIVST_Prevention_Serv[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Prevention_Serv[df_columns[1]] - df_HIVST_Prevention_Serv[df_columns[0]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated referral gap to new column

        df_HIVST_Prevention_Serv[gap_columns[1]] = np.where(  
            df_HIVST_Prevention_Serv[df_columns[2]] != df_HIVST_Prevention_Serv[df_columns[1]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Prevention_Serv[df_columns[2]] - df_HIVST_Prevention_Serv[df_columns[1]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated access gap to new column

        reorder_columns = MSF_hierarchy + [
            df_columns[0], df_columns[1], gap_columns[0], 
            df_columns[2], gap_columns[1]
        ]                                                               # -- Define column order for DataFrame

        df_HIVST_Prevention_Serv = df_HIVST_Prevention_Serv[reorder_columns]  # -- Reorder DataFrame columns

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Prevention_Serv)                   # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Prevention_Serv_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_HIVST_Prevention_Serv.shape          # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_HIVST_Prevention_Serv_gap.cached_style)  # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_HIVST_Prevention_Serv_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HIVST_Prevention_Serv,                                # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_HIVST_Prevention_Serv_gap is None:                        # -- Check if no gaps found
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # -- Check if cached style exists
                del process_HIVST_Prevention_Serv_gap.cached_style      # -- Remove cached styled DataFrame
            if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HIVST_Prevention_Serv_gap.cached_shape      # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HIVST_Prevention_Serv_gap_style = (                          # -- Apply styling to filtered DataFrame
            df_HIVST_Prevention_Serv_gap.style                          # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap[0])               # -- Highlight first gap column in red
            .map(outlier_yellow, subset=gap_columns_wrap[1])            # -- Highlight second gap column in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Prevention_Serv_gap.cached_style = df_HIVST_Prevention_Serv_gap_style  # -- Store styled DataFrame in cache
        process_HIVST_Prevention_Serv_gap.cached_shape = df_HIVST_Prevention_Serv.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_HIVST_Prevention_Serv_gap['ReportPeriod'].iloc[0]  # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        report_description = []                                         # -- Initialize report description
        if (df_HIVST_Prevention_Serv_gap[gap_columns_wrap[0]] != 0).any():  # -- Check if any non-zero gaps exist
            report_description.append(                                  # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should not be greater than {df_columns[0]}"
            )                                                           # -- Add referral gap description
        if (df_HIVST_Prevention_Serv_gap[gap_columns_wrap[1]] != 0).any():  # -- Check if any non-zero gaps exist
            report_description.append(                                  # -- Define report description
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\n"
                f"should be equal to {df_columns[1]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Add access gap description
        report_description = "\n\n".join(report_description)            # -- Join all descriptions into a single string

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_HIVST_Prevention_Serv_gap_style,               # -- Pass styled DataFrame
            img_file_name=report_image_name,                           # -- Pass image file name
            img_file_path=report_image_path,                           # -- Pass image file path
            doc_description=report_description,                        # -- Pass description
            doc_indicators_to_italicize=df_columns,                    # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                   # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                           # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_HIVST_Prevention_Serv_gap_style)       # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_style'):  # -- Check if cached style exists
            del process_HIVST_Prevention_Serv_gap.cached_style          # -- Remove cached styled DataFrame
        if hasattr(process_HIVST_Prevention_Serv_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HIVST_Prevention_Serv_gap.cached_shape          # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                              # -- End of function

#### - HIVST Partner Screening

In [ ]:
# -- Define the main function to process HIVST Partner Screening gap
def process_HIVST_Partner_Screening_gap(display_output=None):            # -- Define function with optional display parameter
    """
    Process HIVST Partner Screening  gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of partners of people living with HIV screened with HIVST kit (confirmed during follow up)",
            "Number of partners of people living with HIV reporting HIVST results"
        ]
        name = "HIVST Partner Screening gap"                            # -- Define general name
        gap_columns = ["HIVST Partner Screening gap"]                    # -- Define gap column names
        report_name = f"{name}21"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_HIVST_Partner_Screening = prepare_and_convert_df(            # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_HIVST_Partner_Screening is None:                          # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_HIVST_Partner_Screening[gap_columns[0]] = np.where(  
            df_HIVST_Partner_Screening[df_columns[1]] != df_HIVST_Partner_Screening[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_HIVST_Partner_Screening[df_columns[1]] - df_HIVST_Partner_Screening[df_columns[0]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated screening gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_HIVST_Partner_Screening)                 # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_HIVST_Partner_Screening_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_HIVST_Partner_Screening.shape        # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_HIVST_Partner_Screening_gap.cached_style)  # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_HIVST_Partner_Screening_gap = filter_gap_and_check_empty_df(  # -- Filter DataFrame for gaps
            df=df_HIVST_Partner_Screening,                              # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_HIVST_Partner_Screening_gap is None:                        # -- Check if no gaps found
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # -- Check if cached style exists
                del process_HIVST_Partner_Screening_gap.cached_style    # -- Remove cached styled DataFrame
            if hasattr(process_HIVST_Partner_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_HIVST_Partner_Screening_gap.cached_shape    # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_HIVST_Partner_Screening_gap_style = (                          # -- Apply styling to filtered DataFrame
            df_HIVST_Partner_Screening_gap.style                          # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)               # -- Highlight gap column in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_HIVST_Partner_Screening_gap.cached_style = df_HIVST_Partner_Screening_gap_style  # -- Store styled DataFrame in cache
        process_HIVST_Partner_Screening_gap.cached_shape = df_HIVST_Partner_Screening.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_HIVST_Partner_Screening_gap['ReportPeriod'].iloc[0]  # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_HIVST_Partner_Screening_gap[gap_columns_wrap[0]] != 0).any():  # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_HIVST_Partner_Screening_gap_style,               # -- Pass styled DataFrame
            img_file_name=report_image_name,                           # -- Pass image file name
            img_file_path=report_image_path,                           # -- Pass image file path
            doc_description=report_description,                        # -- Pass description
            doc_indicators_to_italicize=df_columns,                    # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                   # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                   # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                           # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_HIVST_Partner_Screening_gap_style)       # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_HIVST_Partner_Screening_gap, 'cached_style'):  # -- Check if cached style exists
            del process_HIVST_Partner_Screening_gap.cached_style        # -- Remove cached styled DataFrame
        if hasattr(process_HIVST_Partner_Screening_gap, 'cached_shape'):  # -- Check if cached shape exists
            del process_HIVST_Partner_Screening_gap.cached_shape        # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                          # -- End of function

### ICT MSF

#### - ICT Offereing

In [ ]:
# -- Define the main function to process ICT Index Acceptance gap
def process_ICT_Index_Acceptance_gap(display_output=None):               # -- Define function with optional display parameter
    """
    Process ICT Index Acceptance  gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of HIV Positive Clients Offered Index Testing",
            "Number of HIV Positive Clients Accepting Index Testing"
        ]
        name = "ICT Index Acceptance gap"                               # -- Define general name
        gap_columns = ["ICT Index Acceptance gap"]                          # -- Define gap column names
        report_name = f"{name}22"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_main[gap_columns[0]] = np.where(  
            df_main[df_columns[1]] > df_main[df_columns[0]],            # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_main[df_columns[1]] - df_main[df_columns[0]],            # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated acceptance gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):  # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Index_Acceptance_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_ICT_Index_Acceptance_gap.cached_style)  # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(                   # -- Filter DataFrame for gaps
            df=df_main,                                                 # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_mine_gap is None:                                         # -- Check if no gaps found
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):  # -- Check if cached style exists
                del process_ICT_Index_Acceptance_gap.cached_style       # -- Remove cached styled DataFrame
            if hasattr(process_ICT_Index_Acceptance_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ICT_Index_Acceptance_gap.cached_shape       # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_mine_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_mine_gap.style                                           # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight gap column in red
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_ICT_Index_Acceptance_gap.cached_style = df_mine_gap_style  # -- Store styled DataFrame in cache
        process_ICT_Index_Acceptance_gap.cached_shape = df_main.shape  # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]             # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():              # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_mine_gap_style,                                 # -- Pass styled DataFrame
            img_file_name=report_image_name,                            # -- Pass image file name
            img_file_path=report_image_path,                            # -- Pass image file path
            doc_description=report_description,                         # -- Pass description
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_mine_gap_style)                        # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_ICT_Index_Acceptance_gap, 'cached_style'):   # -- Check if cached style exists
            del process_ICT_Index_Acceptance_gap.cached_style           # -- Remove cached styled DataFrame
        if hasattr(process_ICT_Index_Acceptance_gap, 'cached_shape'):   # -- Check if cached shape exists
            del process_ICT_Index_Acceptance_gap.cached_shape           # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                          # -- End of function

#### - ICT Contact

In [ ]:
# -- Define the main function to process ICT Contact gap
def process_ICT_Contact_gap(display_output=None):                        # -- Define function with optional display parameter
    """
    Process ICT Contact gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of HIV Positive Clients Accepting Index Testing",   # -- Define list of HIVST reactive confirmed and linkage columns
            "Number of Children enumerated and Partners elicited from index client"
        ]
        name = "ICT Contact gap"                                        # -- Define general name
        gap_columns = ["ICT Contact gap"]                               # -- Define gap column names
        report_name = f"{name}23"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_main[gap_columns[0]] = np.where(  
            df_main[df_columns[1]] < df_main[df_columns[0]],            # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_main[df_columns[1]] - df_main[df_columns[0]],            # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated contact gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_ICT_Contact_gap, 'cached_style'):        # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Contact_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_ICT_Contact_gap.cached_style)       # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(                   # -- Filter DataFrame for gaps
            df=df_main,                                                 # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_mine_gap is None:                                         # -- Check if no gaps found
            if hasattr(process_ICT_Contact_gap, 'cached_style'):        # -- Check if cached style exists
                del process_ICT_Contact_gap.cached_style                # -- Remove cached styled DataFrame
            if hasattr(process_ICT_Contact_gap, 'cached_shape'):        # -- Check if cached shape exists
                del process_ICT_Contact_gap.cached_shape                # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_mine_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_mine_gap.style                                           # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)               # -- Highlight gap column in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_ICT_Contact_gap.cached_style = df_mine_gap_style        # -- Store styled DataFrame in cache
        process_ICT_Contact_gap.cached_shape = df_main.shape            # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]             # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():              # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be greater than {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_mine_gap_style,                                 # -- Pass styled DataFrame
            img_file_name=report_image_name,                            # -- Pass image file name
            img_file_path=report_image_path,                            # -- Pass image file path
            doc_description=report_description,                         # -- Pass description
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_mine_gap_style)                        # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_ICT_Contact_gap, 'cached_style'):            # -- Check if cached style exists
            del process_ICT_Contact_gap.cached_style                    # -- Remove cached styled DataFrame
        if hasattr(process_ICT_Contact_gap, 'cached_shape'):            # -- Check if cached shape exists
            del process_ICT_Contact_gap.cached_shape                    # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                          # -- End of function

#### - ICT HTS

In [ ]:
# -- Define the main function to process ICT HTS gap
def process_ICT_HTS_gap(display_output=None):                            # -- Define function with optional display parameter
    """
    Process ICT HTS gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of ICT HTS columns
            "Number of Children enumerated and Partners elicited from index client",
            "Number of contacts of index clients tested HIV Positive",
            "Number of contacts of index clients tested HIV Negative"
        ]
        name = "ICT Contact Testing gap"                                # -- Define general name
        gap_columns = ["ICT Contact Testing gap"]                       # -- Define gap column names
        report_name = f"{name}24"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_main[gap_columns[0]] = np.where(  
            df_main[df_columns[1:3]].sum(axis=1) != df_main[df_columns[0]],  # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated testing gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_ICT_HTS_gap, 'cached_style'):            # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_HTS_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_ICT_HTS_gap.cached_style)           # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(                   # -- Filter DataFrame for gaps
            df=df_main,                                                 # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_mine_gap is None:                                         # -- Check if no gaps found
            if hasattr(process_ICT_HTS_gap, 'cached_style'):            # -- Check if cached style exists
                del process_ICT_HTS_gap.cached_style                    # -- Remove cached styled DataFrame
            if hasattr(process_ICT_HTS_gap, 'cached_shape'):            # -- Check if cached shape exists
                del process_ICT_HTS_gap.cached_shape                    # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_mine_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_mine_gap.style                                           # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)               # -- Highlight gap column in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_ICT_HTS_gap.cached_style = df_mine_gap_style            # -- Store styled DataFrame in cache
        process_ICT_HTS_gap.cached_shape = df_main.shape                # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]             # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():              # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\npluse {df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_mine_gap_style,                                 # -- Pass styled DataFrame
            img_file_name=report_image_name,                            # -- Pass image file name
            img_file_path=report_image_path,                            # -- Pass image file path
            doc_description=report_description,                         # -- Pass description
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_mine_gap_style)                        # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_ICT_HTS_gap, 'cached_style'):                # -- Check if cached style exists
            del process_ICT_HTS_gap.cached_style                        # -- Remove cached styled DataFrame
        if hasattr(process_ICT_HTS_gap, 'cached_shape'):                # -- Check if cached shape exists
            del process_ICT_HTS_gap.cached_shape                        # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                          # -- End of function

#### - ICT Positive Linkage

In [ ]:
# -- Define the main function to process ICT Positive Linkagegap
def process_ICT_Positive_Link_gap(display_output=None):                  # -- Define function with optional display parameter
    """
    Process ICT Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled DataFrame and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the DataFrame for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of ICT HTS columns
            "Number of contacts of index clients tested HIV Positive",
            "Number of contacts of index clients linked to ART"
        ]
        name = "ICT Contact Testing gap"                                # -- Define general name
        gap_columns = ["ICT Contact Testing gap"]                       # -- Define gap column names
        report_name = f"{name}24"                                       # -- Define report name
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps

        # -- Step 2: Prepare data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="HTS MSF",                                   # -- Specify DHIS2 data key
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns                                     # -- Include specified HTS columns
        )
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        df_main[gap_columns[0]] = np.where(  
            df_main[df_columns[1]] != df_main[df_columns[0]],           # -- Check if Testing Frequency exceeds Assisted & Unassisted
            df_main[df_columns[1]] - df_main[df_columns[0]],            # -- Calculate gap if condition met
            0                                                           # -- Set gap to 0 if no discrepancy
        )                                                               # -- Assign calculated linkage gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Modify DataFrame headers for readability
        HTS_Couple_Counselling_columns_wrap = wrap_column_headers2(df_columns)  # -- Wrap specific column names for consistency
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names for output

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display is requested
            if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):   # -- Check if cached styled DataFrame exists
                cached_shape = getattr(process_ICT_Positive_Link_gap, 'cached_shape', None)  # -- Retrieve cached DataFrame shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure data consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Create message for cached display
                    print("-" * len(cached_display_name))               # -- Print separator line above message
                    print(cached_display_name)                          # -- Print cached display message
                    print("-" * len(cached_display_name))               # -- Print separator line below message
                    display(process_ICT_Positive_Link_gap.cached_style) # -- Render cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_mine_gap = filter_gap_and_check_empty_df(                   # -- Filter DataFrame for gaps
            df=df_main,                                                 # -- Input DataFrame for filtering
            msg=No_gap_msg,                                             # -- Message for empty result
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_mine_gap is None:                                         # -- Check if no gaps found
            if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):  # -- Check if cached style exists
                del process_ICT_Positive_Link_gap.cached_style          # -- Remove cached styled DataFrame
            if hasattr(process_ICT_Positive_Link_gap, 'cached_shape'):  # -- Check if cached shape exists
                del process_ICT_Positive_Link_gap.cached_shape          # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the DataFrame
        df_mine_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_mine_gap.style                                           # -- Create style object from filtered DataFrame
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight gap column in red
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_ICT_Positive_Link_gap.cached_style = df_mine_gap_style  # -- Store styled DataFrame in cache
        process_ICT_Positive_Link_gap.cached_shape = df_main.shape      # -- Store original DataFrame shape in cache

        # -- Step 10: Define export variables
        report_month = df_mine_gap['ReportPeriod'].iloc[0]             # -- Extract report period from filtered DataFrame
        report_image_name = f"{report_month}_{report_name}.png"        # -- Create image file name with report period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Specify path for image export
        report_sheet_name = report_name                                # -- Set Excel sheet name

        # -- Step 11: Create descriptions for Word document
        if (df_mine_gap[gap_columns_wrap[0]] != 0).any():              # -- Check if any non-zero gaps exist
            report_description = (                                      # -- Define report description
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )                                                           # -- Store description for Word document

        # -- Step 12: Export results
        export_df_to_doc_image_excel(                                   # -- Export DataFrame to multiple formats
            report_name=report_name,                                    # -- Pass report name
            df_style=df_mine_gap_style,                                 # -- Pass styled DataFrame
            img_file_name=report_image_name,                            # -- Pass image file name
            img_file_path=report_image_path,                            # -- Pass image file path
            doc_description=report_description,                         # -- Pass description
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specific columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap column in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Pass Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Pass Excel sheet name
        )                                                               # -- Execute export to image, Excel, and Word

        # -- Step 13: Optionally display styled DataFrame
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_mine_gap_style)                        # -- Render styled DataFrame in widget

    except Exception as e:                                              # -- Catch any unhandled exceptions
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Log general error
        if hasattr(process_ICT_Positive_Link_gap, 'cached_style'):      # -- Check if cached style exists
            del process_ICT_Positive_Link_gap.cached_style              # -- Remove cached styled DataFrame
        if hasattr(process_ICT_Positive_Link_gap, 'cached_shape'):      # -- Check if cached shape exists
            del process_ICT_Positive_Link_gap.cached_shape              # -- Remove cached shape
        return                                                         # -- Exit function on general error
    # -- End of function                                                                          # -- End of function

## PMCTCT MSF

#### - New ANC HTS

In [ ]:
# -- Define the main function to process PMTCT New ANC HTS gap
def process_PMTCT_ANC_Optmz_gap(display_output=None):                    # -- Define function with optional display parameter
    """
    Process PMTCT New ANC HTS gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of PMTCT New ANC columns
            "Number of new ANC Clients",                                # -- Column for new ANC clients
            "Number of pregnant women with previously known HIV positive infection"  # -- Column for known HIV positives
        ]                                                               
        df_columns_spec = ['Number of new ANC Clients',                 # -- Define all relevant columns
                           'Number of pregnant women with previously known HIV positive infection',  # -- Known HIV positives column
                           'Number of pregnant women HIV tested and received results (ANC)',  # -- ANC tested results column
                           'Number of pregnant women HIV tested and received results (L&D)',  # -- L&D tested results column
                           'Number of pregnant women HIV tested and received results (<72hrs Postpartum)'  # -- <72hrs Postpartum tested results column
        ]                                                               # -- Close list of specific columns
        df_columns2 = ['ANC', 'L&D', '<72hrs Postpartum']               # -- Define service delivery point columns
        name = "PMTCT New ANC HTS Optimization gap"                     # -- Define general name for reporting
        gap_columns = ["PMTCT New ANC HTS gap"]                        # -- Define gap column names
        report_name = f"{name}25"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_main2 = prepare_and_convert_df(                              # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF_sdp",                             # -- Specify DHIS2 data key for HIVST approach
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns2                                    # -- Include specified HIVST mode columns
        )                                                               # -- Store additional DataFrame
        if df_main2 is None:                                            # -- Check if data preparation failed
            return                                                      # -- Exit function if no data

        # -- Step 4: Merge datasets
        df_main = df_main.merge(                                        # -- Merge HIVST mode data with primary data
            df_main2,                                                   # -- Target DataFrame for merge
            on=MSF_hierarchy,                                           # -- Merge on MSF hierarchy columns
            how="left"                                                  # -- Right join to keep all rows from primary data
        )                                                               # -- Update df_main with merged data

        # -- Step 5: Rename columns for consistency
        df_main = df_main.rename(columns={                              # -- Rename columns to align with specified names
            f"{df_columns2[0]}": f"{df_columns_spec[2]}",               # -- Rename ANC column
            f"{df_columns2[1]}": f"{df_columns_spec[3]}",               # -- Rename L&D column
            f"{df_columns2[2]}": f"{df_columns_spec[4]}"                # -- Rename <72hrs Postpartum column
        })                                                              # -- Apply renamed columns to DataFrame

        # -- Step 6: Clean and format data
        df_main.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # -- Sort by hierarchy
        df_main = df_main.fillna(0)                                     # -- Replace NaN with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                                       # -- Convert float columns to integers
            df_main[col] = df_main[col].astype(int)                     # -- Apply integer conversion

        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT ANC HTS gap
            df_main[df_columns_spec[1:3]].sum(axis=1) != df_main[df_columns[0]],  # -- Check for Assisted gap
            df_main[df_columns_spec[1:3]].sum(axis=1) - df_main[df_columns[0]],  # -- Calculate gap as sum minus new ANC clients
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):    # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_ANC_Optmz_gap.cached_style)   # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):    # -- Clear cache if it exists
                del process_PMTCT_ANC_Optmz_gap.cached_style            # -- Remove cached style
            if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape'):    # -- Clear cached shape
                del process_PMTCT_ANC_Optmz_gap.cached_shape            # -- Remove cached shape
            print(f"⦸ {No_gap_msg}")                                   # -- Notify user of no gaps
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_ANC_Optmz_gap.cached_style = df_main_gap_style    # -- Store styled DataFrame
        process_PMTCT_ANC_Optmz_gap.cached_shape = df_main.shape        # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for Assisted gaps
            report_description = (                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[1]}\npluse {df_columns_spec[2]}\n"
                f"should be equal to {df_columns_spec[0]}"              # -- Describe expected equality
            )                                                           # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns_spec,                # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_style'):        # -- Clear cache if it exists
            del process_PMTCT_ANC_Optmz_gap.cached_style                # -- Remove cached style
        if hasattr(process_PMTCT_ANC_Optmz_gap, 'cached_shape'):        # -- Clear cached shape
            del process_PMTCT_ANC_Optmz_gap.cached_shape                # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                         

#### - PMTCT Positive

In [ ]:
# -- Define the main function to process PMTCT Positive gap
def process_PMTCT_Positive_gap(display_output=None):                    # -- Define function with optional display parameter
    """
    Process PMTCT Positive gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of PMTCT New ANC columns
            "Number of pregnant women HIV tested and received results (ANC)",  # -- ANC tested results column
            "Number of pregnant women HIV tested and received results (L&D)",  # -- L&D tested results column
            "Number of pregnant women HIV tested and received results (<72hrs Postpartum)",  # -- <72hrs Postpartum tested results column
            "Number of pregnant women tested HIV positive (ANC)",        # -- ANC positives column
            "Number of pregnant women tested HIV positive (L&D)",        # -- L&D positives column
            "Number of pregnant women tested HIV positive (<72hrs Postpartum)"  # -- <72hrs Postpartum positives column
        ]                                                               # -- Close list of specific columns
        df_columns2 = ['ANC', 'L&D', '<72hrs Postpartum']               # -- Define service delivery point columns
        name = "PMTCT Positive gap"                                     # -- Define general name for reporting
        gap_columns = ["PMTCT Positive (ANC) gap",                      # -- ANC gap column
                       "PMTCT Positive (L&D) gap",                      # -- L&D gap column
                       "PMTCT Positive (<72hrs Postpartum) gap"]        # -- <72hrs Postpartum gap column
        report_name = f"{name}26"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF_sdp",                             # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns2                                    # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3: Fetch and prepare additional HIVST mode data
        df_main2 = prepare_and_convert_df(                              # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF_sdp_pos",                         # -- Specify DHIS2 data key for HIVST approach
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns
            data_columns=df_columns2                                    # -- Include specified HIVST mode columns
        )                                                               # -- Store additional DataFrame
        if df_main2 is None:                                            # -- Check if data preparation failed
            return                                                      # -- Exit function if no data

        # -- Step 4: Merge datasets
        df_main = df_main.merge(                                        # -- Merge HIVST mode data with primary data
            df_main2,                                                   # -- Target DataFrame for merge
            on=MSF_hierarchy,                                           # -- Merge on MSF hierarchy columns
            how="left"                                                  # -- Right join to keep all rows from primary data
        )                                                               # -- Update df_main with merged data

        # -- Step 5: Rename columns for consistency
        df_main = df_main.rename(columns={                              # -- Rename columns to align with specified names
            f'ANC_x': f"{df_columns[0]}",                               # -- Rename ANC tested column
            'L&D_x': f"{df_columns[1]}",                                # -- Rename L&D tested column
            '<72hrs Postpartum_x': f"{df_columns[2]}",                  # -- Rename <72hrs Postpartum tested column
            'ANC_y': f"{df_columns[3]}",                                # -- Rename ANC positives column
            'L&D_y': f"{df_columns[4]}",                                # -- Rename L&D positives column
            '<72hrs Postpartum_y': f"{df_columns[5]}"                   # -- Rename <72hrs Postpartum positives column
        })                                                              # -- Apply renamed columns to DataFrame

        # -- Step 6: Clean and format data
        df_main.sort_values(by=MSF_hierarchy, inplace=True, ignore_index=True)  # -- Sort by hierarchy
        df_main = df_main.fillna(0)                                     # -- Replace NaN with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                                       # -- Convert float columns to integers
            df_main[col] = df_main[col].astype(int)                     # -- Apply integer conversion

        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT ANC positive gap
            df_main[df_columns[3]] > df_main[df_columns[0]],            # -- Check if ANC positives exceed tested
            df_main[df_columns[3]] - df_main[df_columns[0]],            # -- Calculate gap as positives minus tested
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        df_main[gap_columns[1]] = np.where(                             # -- Calculate PMTCT L&D positive gap
            df_main[df_columns[4]] > df_main[df_columns[1]],            # -- Check if L&D positives exceed tested
            df_main[df_columns[4]] - df_main[df_columns[1]],            # -- Calculate gap as positives minus tested
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        df_main[gap_columns[2]] = np.where(                             # -- Calculate PMTCT <72hrs Postpartum positive gap
            df_main[df_columns[5]] > df_main[df_columns[2]],            # -- Check if <72hrs Postpartum positives exceed tested
            df_main[df_columns[5]] - df_main[df_columns[2]],            # -- Calculate gap as positives minus tested
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Positive_gap, 'cached_style'):     # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Positive_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Positive_gap.cached_style)    # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Positive_gap, 'cached_style'):     # -- Clear cache if it exists
                del process_PMTCT_Positive_gap.cached_style             # -- Remove cached style
            if hasattr(process_PMTCT_Positive_gap, 'cached_shape'):     # -- Clear cached shape
                del process_PMTCT_Positive_gap.cached_shape             # -- Remove cached shape
            print(f"⦸ {No_gap_msg}")                                   # -- Notify user of no gaps
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Positive_gap.cached_style = df_main_gap_style     # -- Store styled DataFrame
        process_PMTCT_Positive_gap.cached_shape = df_main.shape         # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        report_description = []                                         # -- Initialize description list
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for ANC gaps
            report_description.append(                                  # -- Add description for ANC gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[3]}\n"
                f"should not be greater than {df_columns[0]}"           # -- Describe expected ANC constraint
            )                                                           # -- Close ANC description string
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():              # -- Check for L&D gaps
            report_description.append(                                  # -- Add description for L&D gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[4]}\n"
                f"should not be greater than {df_columns[1]}"           # -- Describe expected L&D constraint
            )                                                           # -- Close L&D description string
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():              # -- Check for <72hrs Postpartum gaps
            report_description.append(                                  # -- Add description for <72hrs Postpartum gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns[5]}\n"
                f"should not be greater than {df_columns[2]}"           # -- Describe expected <72hrs Postpartum constraint
            )                                                           # -- Close <72hrs Postpartum description string
        report_description = "\n\n".join(report_description)           # -- Join descriptions with newlines

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_Positive_gap, 'cached_style'):         # -- Clear cache if it exists
            del process_PMTCT_Positive_gap.cached_style                 # -- Remove cached style
        if hasattr(process_PMTCT_Positive_gap, 'cached_shape'):         # -- Clear cached shape
            del process_PMTCT_Positive_gap.cached_shape                 # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Previouly Known

In [ ]:
# -- Define the main function to process PMTCT Previously Known gap
def process_PMTCT_PK_gap(display_output=None):                          # -- Define function with optional display parameter
    """
    Process PMTCT Previously Known gap, comparing known HIV-positive pregnant women
    to those already on ART prior to pregnancy. Exports results as image, Excel, and
    Word files. Caches the styled DataFrame and displays it on subsequent calls if
    data shape is unchanged.

    Args:
        display_output (bool, optional): If True, displays the DataFrame for gaps.
            Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of PMTCT columns
            "Number of pregnant women with previously known HIV positive infection",  # -- Known HIV positives column
            "Number of HIV positive pregnant women already on ART prior to this pregnancy"  # -- ART prior to pregnancy column
        ]                                                               # -- Close list of columns
        name = "PMTCT Previously Known gap"                             # -- Define general name for reporting
        gap_columns = ["PMTCT Previously Known gap"]                    # -- Define gap column name
        report_name = f"{name}27"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified PMTCT columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data

        # -- Step 4: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT previously known gap
            df_main[df_columns[1]] != df_main[df_columns[0]],           # -- Check if ART counts differ from known positives
            df_main[df_columns[1]] - df_main[df_columns[0]],            # -- Calculate gap as ART minus known positives
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 5: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names (commented-out, undefined)
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 6: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_PK_gap, 'cached_style'):           # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_PK_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_PK_gap.cached_style)          # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 7: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_PK_gap, 'cached_style'):           # -- Clear cache if it exists
                del process_PMTCT_PK_gap.cached_style                   # -- Remove cached style
            if hasattr(process_PMTCT_PK_gap, 'cached_shape'):           # -- Clear cached shape
                del process_PMTCT_PK_gap.cached_shape                   # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 8: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 9: Cache styled DataFrame and shape
        process_PMTCT_PK_gap.cached_style = df_main_gap_style           # -- Store styled DataFrame
        process_PMTCT_PK_gap.cached_shape = df_main.shape               # -- Store DataFrame shape

        # -- Step 10: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 11: Create descriptions for Word document
        report_description = ""                                         # -- Initialize description string
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for previously known gaps
            report_description = (                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"                   # -- Describe expected equality
            )                                                           # -- Close description string

        # -- Step 12: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                   # -- Excel file path
            xlm_sheet_name=report_sheet_name                           # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 13: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

        # -- Step 14: Handle errors
    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_PK_gap, 'cached_style'):               # -- Clear cache if it exists
            del process_PMTCT_PK_gap.cached_style                       # -- Remove cached style
        if hasattr(process_PMTCT_PK_gap, 'cached_shape'):               # -- Clear cached shape
            del process_PMTCT_PK_gap.cached_shape                       # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Linkage

In [ ]:
# -- Define the main function to process PMTCT Positive Linkage gap
def process_PMTCT_Positive_Linkage_gap(display_output=None):            # -- Define function with optional display parameter
    """
    Process PMTCT Positive Linkage gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of PMTCT New ANC columns
            "Number of pregnant women tested HIV positive",             # -- Total HIV positives column
            "Number of HIV positive pregnant women newly started on ART during ANC <36wks of pregnancy",  # -- ART initiation ANC <36wks column
            "Number of HIV positive pregnant women newly started on ART during ANC >36wks of pregnancy",  # -- ART initiation ANC >36wks column
            "Number of HIV positive pregnant women newly started on ART during Labour",  # -- ART initiation Labour column
            "Number of HIV positive pregnant women newly started on ART during Post Partum (<72 hrs)",  # -- ART initiation Postpartum <72hrs column
            "Number of HIV positive pregnant women newly started on ART during Post Partum (>72 hrs - < 6 months)",  # -- ART initiation Postpartum >72hrs-<6 months column
            "Number of HIV positive pregnant women newly started on ART during Post Partum (>6 - 12 months)"  # -- ART initiation Postpartum >6-12 months column
        ]                                                               # -- Close list of columns
        name = "PMTCT Positive Linkage Known gap"                       # -- Define general name for reporting
        gap_columns = ["PMTCT Positive Linkage gap"]                    # -- Define gap column name
        report_name = f"{name}28"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[1:7]].sum(axis=1) != df_main[df_columns[0]],  # -- Check if ART initiations sum differs from positives
            df_main[df_columns[1:7]].sum(axis=1) - df_main[df_columns[0]],  # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Positive_Linkage_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Positive_Linkage_gap.cached_style     # -- Remove cached style
            if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Positive_Linkage_gap.cached_shape     # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Positive_Linkage_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Positive_Linkage_gap.cached_shape = df_main.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for positive linkage gaps
            report_description = (                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\npluse {df_columns[2]}\npluse {df_columns[3]}\npluse {df_columns[4]}"
                f"\npluse {df_columns[5]}\npluse {df_columns[6]}\n"
                f"should be equal to {df_columns[0]}"                   # -- Describe expected equality
            )                                                           # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_PMTCT_Positive_Linkage_gap.cached_style         # -- Remove cached style
        if hasattr(process_PMTCT_Positive_Linkage_gap, 'cached_shape'):  # -- Clear cached shape
            del process_PMTCT_Positive_Linkage_gap.cached_shape         # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Seroconversion

In [ ]:
# -- Define the main function to process PMTCT Seroconversion gap
def process_PMTCT_Seroconversion_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT Seroconversion gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of Seroconversion columns
            "Number of pregnant women retested after initial HIV negative test",  # -- Retested women column
            "Number of pregnant women retested who seroconverted to HIV positive after initial HIV negative test"  # -- Seroconverted women column
        ]                                                               # -- Close list of columns
        name = "PMTCT Seroconversion gap"                               # -- Define general name for reporting
        gap_columns = ["PMTCT Seroconversion check"]                    # -- Define gap column name
        report_name = f"{name}29"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[1]] > 0,                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[1]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Seroconversion_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Seroconversion_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Seroconversion_gap.cached_style       # -- Remove cached style
            if hasattr(process_PMTCT_Seroconversion_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Seroconversion_gap.cached_shape       # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)               # -- Highlight non-zero gaps in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Seroconversion_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Seroconversion_gap.cached_shape = df_main.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for positive linkage gaps
            report_description = (                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"Note: {df_columns[1]} is a quality indicator and should review thoroughly."  # -- Describe expected equality
            )                                                           # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

        # -- Step 18: Handle errors
    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_Seroconversion_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_PMTCT_Seroconversion_gap.cached_style           # -- Remove cached style
        if hasattr(process_PMTCT_Seroconversion_gap, 'cached_shape'):  # -- Clear cached shape
            del process_PMTCT_Seroconversion_gap.cached_shape           # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Coinfection (syphilis)

In [ ]:
# -- Define the main function to process PMTCT Syphilis Test gap
def process_PMTCT_Syphilis_Test_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT Syphilis Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of Syphilis Test columns
            "Number of new ANC Clients",
            "Number of new ANC Clients tested for syphilis",
            "Number of new ANC Clients tested positive for syphilis",
            "Number of the ANC Clients treated for Syphilis"  # -- Seroconverted women column
        ]                                                               # -- Close list of columns
        name = "PMTCT Syphilis Test gap"                               # -- Define general name for reporting
        gap_columns = ["PMTCT New ANC Syphilis Test gap",
                       "PMTCT Syphilis Positive gap",
                       "PMTCT Syphilis Positive Treatment gap"]                    # -- Define gap column name
        report_name = f"{name}30"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[1]] != df_main[df_columns[0]],                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[1]] - df_main[df_columns[0]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        ) 
        
        df_main[gap_columns[1]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[2]] > df_main[df_columns[1]],                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[2]] - df_main[df_columns[1]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        ) 
        
        df_main[gap_columns[2]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[3]] != df_main[df_columns[2]],                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[3]] - df_main[df_columns[2]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Syphilis_Test_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Syphilis_Test_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Syphilis_Test_gap.cached_style       # -- Remove cached style
            if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Syphilis_Test_gap.cached_shape       # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap[0])
            .map(outlier_red, subset=gap_columns_wrap[1:3])               # -- Highlight non-zero gaps in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Syphilis_Test_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Syphilis_Test_gap.cached_shape = df_main.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        report_description = []
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for positive linkage gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():              # -- Check for positive linkage gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\n"
                f"should not be greater than {df_columns[1]}"
            )
        if (df_main_gap[gap_columns_wrap[2]] != 0).any():              # -- Check for positive linkage gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[2]}\n"
                f"{df_columns[3]}\n"
                f"should be equal to {df_columns[2]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )
        report_description = "\n\n".join(report_description)                                                       # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

        # -- Step 18: Handle errors
    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_PMTCT_Syphilis_Test_gap.cached_style           # -- Remove cached style
        if hasattr(process_PMTCT_Syphilis_Test_gap, 'cached_shape'):  # -- Clear cached shape
            del process_PMTCT_Syphilis_Test_gap.cached_shape           # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Coinfection (HBV, HCV)

In [ ]:
# -- Define the main function to process PMTCT Hepatitis Test gap
def process_PMTCT_Hepatitis_Test_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT Hepatitis Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of Syphilis Test columns
            "Number of new ANC Clients",
            "Number of new ANC Clients tested for HBV ( ANC, L&D, <72hrs Post Partum)",
            "Number of new ANC Clients tested for HCV ( ANC, L&D, < 72hrs Post Partum)"
        ]                                                               # -- Close list of columns
        name = "PMTCT Syphilis Test gap"                               # -- Define general name for reporting
        gap_columns = [
            "PMTCT Hepatitis B Test gap",
            "PMTCT Hepatitis C Positive gap"
        ]                    
        report_name = f"{name}31"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[1]] != df_main[df_columns[0]],                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[1]] - df_main[df_columns[0]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        ) 
        
        df_main[gap_columns[1]] = np.where(                             # -- Calculate PMTCT positive linkage gap
            df_main[df_columns[2]] != df_main[df_columns[0]],                                 # -- Check if ART initiations sum differs from positives
            df_main[df_columns[2]] - df_main[df_columns[0]],                                     # -- Calculate gap as ART sum minus positives
            0                                                           # -- Set to 0 if no gap
        )                                                           # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)    # -- Wrap specific column names
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Hepatitis_Test_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Hepatitis_Test_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                   # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Hepatitis_Test_gap.cached_style       # -- Remove cached style
            if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Hepatitis_Test_gap.cached_shape       # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)               # -- Highlight non-zero gaps in yellow
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Hepatitis_Test_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Hepatitis_Test_gap.cached_shape = df_main.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]             # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"        # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"    # -- Define path for image export
        report_sheet_name = report_name                                # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        report_description = []
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():              # -- Check for positive linkage gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():              # -- Check for positive linkage gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"
                f"Note: Where this report is correct, please ignore the gap - only review."
            )
        report_description = "\n\n".join(report_description)                                                       # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

        # -- Step 18: Handle errors
    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")           # -- Print error details
        if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_PMTCT_Hepatitis_Test_gap.cached_style           # -- Remove cached style
        if hasattr(process_PMTCT_Hepatitis_Test_gap, 'cached_shape'):  # -- Clear cached shape
            del process_PMTCT_Hepatitis_Test_gap.cached_shape           # -- Remove cached shape
        return                                                      # -- Exit function on error
    # -- End of function                                                                          # -- End of function

#### - PMTCT Labour and Delivery

In [ ]:
# -- Define the main function to process PMTCT Labour and Delivery gap
def process_PMTCT_Labour_Delivery_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT Labour and Delivery gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of labour and delivery columns
            "Total deliveries at facility (booked and unbooked pregnant women) - Total",
            "Number of booked HIV positive pregnant women who delivered at facility - Total",
            "Number of unbooked HIV positive pregnant women who delivered at the facility - Total",
            "Number of live births by HIV positive women who delivered at the facility - Total"
        ]                                                               # -- Close list of columns
        name = "PMTCT Labour and Delivery gap"                          # -- Define general name for reporting
        gap_columns = [                                                 # -- Define list of gap columns
            "PMTCT Facility Delivery by PPW gap",
            "PMTCT Facility Livebirth by PPW gap"
        ]                                                               # -- Close list of gap columns
        report_name = f"{name}32"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for HTS MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified HTS Enrolment columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 7: Calculate gaps
        df_main[gap_columns[0]] = np.where(                             # -- Calculate PMTCT Facility Delivery by PPW gap
            df_main[df_columns[1:3]].sum(axis=1) > df_main[df_columns[0]],  # -- Check if sum of booked/unbooked exceeds total deliveries
            df_main[df_columns[1:3]].sum(axis=1) - df_main[df_columns[0]],  # -- Calculate gap as sum minus total deliveries
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column
        
        df_main[gap_columns[1]] = np.where(                             # -- Calculate PMTCT Facility Livebirth by PPW gap
            df_main[df_columns[3]] < df_main[df_columns[1:3]].sum(axis=1),  # -- Check if live births are less than sum of booked/unbooked
            df_main[df_columns[3]] - df_main[df_columns[1:3]].sum(axis=1),  # -- Calculate gap as live births minus sum
            0                                                           # -- Set to 0 if no gap
        )                                                               # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                    # -- Apply wrapping to DataFrame headers
        gap_columns_wrap = wrap_column_headers2(gap_columns)            # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                              # -- Check if display output is requested
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                           # -- Get current DataFrame shape
                if cached_shape == current_shape:                       # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "  # -- Display message for cached data
                    print("-" * len(cached_display_name))               # -- Print separator
                    print(cached_display_name)                          # -- Print message
                    print("-" * len(cached_display_name))               # -- Print separator
                    display(process_PMTCT_Labour_Delivery_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                              # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                    # -- Filter rows with non-zero gaps
            df=df_main,                                                 # -- Input DataFrame
            msg=No_gap_msg,                                             # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                 # -- Filter for non-zero gaps
            opNeg=None,                                                 # -- No negative value filter
            opPos=None,                                                 # -- No positive value filter
            opZero=None,                                                # -- No zero value filter
            opLT100=None                                                # -- No less-than-100 filter
        )                                                               # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                         # -- Check if no gaps were found
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Labour_Delivery_gap.cached_style       # -- Remove cached style
            if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Labour_Delivery_gap.cached_shape       # -- Remove cached shape
            return                                                      # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                           # -- Apply styling to filtered DataFrame
            df_main_gap.style                                           # -- Create style object
            .hide(axis='index')                                         # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                  # -- Highlight non-zero gaps in red
        )                                                               # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Labour_Delivery_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Labour_Delivery_gap.cached_shape = df_main.shape  # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]              # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"         # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"     # -- Define path for image export
        report_sheet_name = report_name                                 # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        report_description = []                                         # -- Initialize descriptions
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():               # -- Check for PMTCT Facility Delivery by PPW gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should not be greater than {df_columns[0]}"           # -- Describe expected equality
            )                                                           # -- Close description string
        if (df_main_gap[gap_columns_wrap[1]] != 0).any():               # -- Check for PMTCT Facility Livebirth by PPW gaps
            report_description.append(                                      # -- Define description for gap
                f"Report Name: {gap_columns[1]}\n"
                f"{df_columns[3]}\n"
                f"should not be less than {df_columns[1]}\nplus {df_columns[2]}"  # -- Describe expected equality
                f"Note: Where this PMTCT livebirth gap is true, please ignore the outlier."
            )                                                           # -- Close description string
        report_description = "\n\n".join(report_description)                                                       # -- Close description string

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                   # -- Export styled DataFrame and descriptions
            report_name=report_name,                                    # -- Report identifier
            df_style=df_main_gap_style,                                 # -- Styled DataFrame for export
            img_file_name=report_image_name,                            # -- Image file name
            img_file_path=report_image_path,                            # -- Image file path
            doc_description=report_description,                         # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                     # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                    # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                    # -- Excel file path
            xlm_sheet_name=report_sheet_name                            # -- Excel sheet name
        )                                                               # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                              # -- Check if display is requested
            widget_display_df(df_main_gap_style)                        # -- Display styled DataFrame in widget

    except Exception as e:                                              # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")            # -- Print error details
        if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_style'):  # -- Clear cache if it exists
            del process_PMTCT_Labour_Delivery_gap.cached_style           # -- Remove cached style
        if hasattr(process_PMTCT_Labour_Delivery_gap, 'cached_shape'):  # -- Clear cached shape
            del process_PMTCT_Labour_Delivery_gap.cached_shape           # -- Remove cached shape
        return                                                          # -- Exit function on error
    # -- End of function                                                # -- End of function

#### - PMTCT Facility HEI ARVs

In [ ]:
# -- Define the main function to process PMTCT Facility HEI ARVs gap
def process_PMTCT_Facility_HEI_ARVs_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT Facility HEI ARVs gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of facility HEI ARVs columns
            "Number of live births by HIV positive women who delivered at the facility - Total",  # -- Total live births column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery",  # -- ARV within 72hrs column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery"  # -- ARV after 72hrs column
        ]                                                               # -- Close list of facility HEI ARVs columns
        df_columns2 = MSF_hierarchy + ['Facility']                      # -- Combine hierarchy and Facility columns
        columns_to_keep = MSF_hierarchy + [df_columns[0]] + [f"{df_columns[1]} (within the Facility)"] + [f"{df_columns[2]} (within the Facility)"]  # -- Combine hierarchy and data columns
        name = "PMTCT Facility HEI ARVs gap"                            # -- Define general name for reporting
        gap_columns = ["PMTCT Facility HEI ARVs gap"]                   # -- Define list of gap columns
        report_name = f"{name}33"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified PMTCT columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3: Rename and prepare ARV within 72hrs data
        df_main2 = DHIS2_data['PMTCT MSF_sd<72_in-outside'][df_columns2].rename(  # -- Select and rename columns from <72hrs dataset
            columns={"Facility": f"{df_columns[1]} (within the Facility)"}   # -- Rename Facility to ARV within 72hrs
        )                                                                   # -- Store renamed DataFrame

        # -- Step 4: Rename and prepare ARV after 72hrs data
        df_main3 = DHIS2_data['PMTCT MSF_sd>72_in-outside'][df_columns2].rename(  # -- Select and rename columns from >72hrs dataset
            columns={"Facility": f"{df_columns[2]} (within the Facility)"}   # -- Rename Facility to ARV after 72hrs
        )                                                                   # -- Store renamed DataFrame

        # -- Step 5: Merge primary data with ARV data
        df_main = df_main.merge(df_main2, on=MSF_hierarchy, how='left')      # -- Merge primary data with ARV within 72hrs data
        df_main = df_main.merge(df_main3, on=MSF_hierarchy, how='left')      # -- Merge primary data with ARV after 72hrs data

        # -- Step 6: Retain specified columns
        df_main = df_main[columns_to_keep]                                   # -- Filter to hierarchy and data columns

        # -- Step 7: Clean and convert data types
        df_main[df_columns[0]] = pd.to_numeric(df_main[df_columns[0]], errors='coerce')  # -- Convert total live births to numeric
        df_main[f"{df_columns[1]} (within the Facility)"] = pd.to_numeric(df_main[f"{df_columns[1]} (within the Facility)"], errors='coerce')  # -- Convert ARV within 72hrs to numeric
        df_main[f"{df_columns[2]} (within the Facility)"] = pd.to_numeric(df_main[f"{df_columns[2]} (within the Facility)"], errors='coerce')  # -- Convert ARV after 72hrs to numeric
        df_main = df_main.fillna(0)                                         # -- Replace NaN with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                                           # -- Convert float columns to integers
            df_main[col] = df_main[col].astype(int)                         # -- Apply integer conversion

        # -- Step 8: Calculate PMTCT Facility Delivery by PPW gap
        df_main[gap_columns[0]] = np.where(                                 # -- Calculate PMTCT ARV prophylaxis gap
            df_main[[f"{df_columns[1]} (within the Facility)", f"{df_columns[2]} (within the Facility)"]].sum(axis=1) != df_main[df_columns[0]],  # -- Check if sum of ARV prophylaxis differs from total live births
            df_main[[f"{df_columns[1]} (within the Facility)", f"{df_columns[2]} (within the Facility)"]].sum(axis=1) - df_main[df_columns[0]],  # -- Calculate gap as sum minus total live births
            0                                                               # -- Set to 0 if no gap
        )                                                                   # -- Assign calculated gap to new column

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                        # -- Apply wrapping to DataFrame headers
        gap_columns_wrap = wrap_column_headers2(gap_columns)                # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                                  # -- Check if display output is requested
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                               # -- Get current DataFrame shape
                if cached_shape == current_shape:                           # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "    # -- Display message for cached data
                    print("-" * len(cached_display_name))                   # -- Print separator
                    print(cached_display_name)                              # -- Print message
                    print("-" * len(cached_display_name))                   # -- Print separator
                    display(process_PMTCT_Facility_HEI_ARVs_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                                  # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                        # -- Filter rows with non-zero gaps
            df=df_main,                                                     # -- Input DataFrame
            msg=No_gap_msg,                                                 # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                     # -- Filter for non-zero gaps
            opNeg=None,                                                     # -- No negative value filter
            opPos=None,                                                     # -- No positive value filter
            opZero=None,                                                    # -- No zero value filter
            opLT100=None                                                    # -- No less-than-100 filter
        )                                                                   # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                             # -- Check if no gaps were found
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_Facility_HEI_ARVs_gap.cached_style         # -- Remove cached style
            if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_Facility_HEI_ARVs_gap.cached_shape         # -- Remove cached shape
            return                                                          # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                               # -- Apply styling to filtered DataFrame
            df_main_gap.style                                               # -- Create style object
            .hide(axis='index')                                             # -- Hide row index for cleaner output
            .map(outlier_red, subset=gap_columns_wrap)                      # -- Highlight non-zero gaps in red
        )                                                                   # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_Facility_HEI_ARVs_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_Facility_HEI_ARVs_gap.cached_shape = df_main.shape    # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]                  # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"             # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"         # -- Define path for image export
        report_sheet_name = report_name                                     # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():                   # -- Check for PMTCT Facility Delivery by PPW gaps
            report_description = (                                          # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[1]}\nplus {df_columns[2]}\n"
                f"should be equal to {df_columns[0]}"  
                f"REPORT ONLY HEI ARVs FOR 2025 LIVE BIRTHS BY PPW."         
                f"Note: Where this PMTCT livebirth gap is true, please ignore the outlier."
            )                                                               # -- Close description

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                       # -- Export styled DataFrame and descriptions
            report_name=report_name,                                        # -- Report identifier
            df_style=df_main_gap_style,                                     # -- Styled DataFrame for export
            img_file_name=report_image_name,                                # -- Image file name
            img_file_path=report_image_path,                                # -- Image file path
            doc_description=report_description,                             # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                         # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                        # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                        # -- Excel file path
            xlm_sheet_name=report_sheet_name                                # -- Excel sheet name
        )                                                                   # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                                  # -- Check if display is requested
            widget_display_df(df_main_gap_style)                            # -- Display styled DataFrame in widget

    except Exception as e:                                                  # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")                # -- Print error details
        if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_style'):    # -- Clear cache if it exists
            del process_PMTCT_Facility_HEI_ARVs_gap.cached_style             # -- Remove cached style
        if hasattr(process_PMTCT_Facility_HEI_ARVs_gap, 'cached_shape'):    # -- Clear cached shape
            del process_PMTCT_Facility_HEI_ARVs_gap.cached_shape             # -- Remove cached shape
        return                                                              # -- Exit function on error
    # -- End of function                                                    # -- End of function

#### - PMTCT EID PCR Test

In [ ]:
# -- Define the main function to process PMTCT EID PCR Test gap
def process_PMTCT_EID_PCR_Test_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT EID PCR Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of facility HEI ARVs columns
            "Number of live births by HIV positive women who delivered at the facility - Total",  # -- Total live births column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery",  # -- ARV within 72hrs column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery",
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth"  # -- ARV after 72hrs column
        ] 
        df_columns_spec = [                                                  # -- Define list of facility HEI ARVs columns
            "Number of live births by HIV positive women who delivered at the facility - Total",  # -- Total live births column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis within 72 hrs of delivery (outside the Facility)",  # -- ARV within 72hrs column
            "Number of HIV-exposed infants born to HIV positive women who received ARV prophylaxis after 72 hrs of delivery (outside the Facility)",
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth"  # -- ARV after 72hrs column
        ]                                                                                                                   # -- Close list of facility HEI ARVs columns
        df_columns2 = MSF_hierarchy + ['Outside Facility']                      # -- Combine hierarchy and Facility columns
        columns_to_keep = (
            MSF_hierarchy + [df_columns_spec[0]] + [df_columns_spec[1]] + 
            [df_columns_spec[2]] + [df_columns_spec[3]] + [df_columns_spec[4]]
        ) # -- Combine hierarchy and data columns
        name = "PMTCT EID PCR Test gap"                            # -- Define general name for reporting
        gap_columns = ["PMTCT EID PCR Test gap"]                   # -- Define list of gap columns
        report_name = f"{name}34"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified PMTCT columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 3: Rename and prepare ARV within 72hrs data
        df_main2 = DHIS2_data['PMTCT MSF_sd<72_in-outside'][df_columns2].rename(  # -- Select and rename columns from <72hrs dataset
            columns={"Outside Facility": df_columns_spec[1]}   # -- Rename Facility to ARV within 72hrs
        )                                                                   # -- Store renamed DataFrame

        # -- Step 4: Rename and prepare ARV after 72hrs data
        df_main3 = DHIS2_data['PMTCT MSF_sd>72_in-outside'][df_columns2].rename(  # -- Select and rename columns from >72hrs dataset
            columns={"Outside Facility": df_columns_spec[2]}   # -- Rename Facility to ARV after 72hrs
        )                                                                   # -- Store renamed DataFrame

        # -- Step 5: Merge primary data with ARV data
        df_main = df_main.merge(df_main2, on=MSF_hierarchy, how='left')      # -- Merge primary data with ARV within 72hrs data
        df_main = df_main.merge(df_main3, on=MSF_hierarchy, how='left')      # -- Merge primary data with ARV after 72hrs data

        # -- Step 6: Retain specified columns
        df_main = df_main[columns_to_keep]                                   # -- Filter to hierarchy and data columns

        # -- Step 7: Clean and convert data types
        df_main = df_main.fillna(0)                                         # -- Replace NaN with 0
        float_columns = df_main.select_dtypes(include=['float64', 'float32']).columns  # -- Identify float columns
        for col in float_columns:                                           # -- Convert float columns to integers
            df_main[col] = df_main[col].astype(int)                         # -- Apply integer conversion

        # -- Step 8: Calculate PMTCT EID gap
        df_main[gap_columns[0]] = np.where(                                 # -- Calculate PMTCT ARV prophylaxis gap
            df_main[df_columns_spec[5:7]].sum(axis=1) != df_main[df_columns_spec[7:9]].sum(axis=1),  # -- Check if sum of ARV prophylaxis differs from total live births
            df_main[df_columns_spec[5:7]].sum(axis=1) - df_main[df_columns_spec[7:9]].sum(axis=1),  # -- Calculate gap as sum minus total live births
            0                                                               # -- Set to 0 if no gap
        ).astype(int)        

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                        # -- Apply wrapping to DataFrame headers
        df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)
        gap_columns_wrap = wrap_column_headers2(gap_columns)                # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                                  # -- Check if display output is requested
            if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_EID_PCR_Test_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                               # -- Get current DataFrame shape
                if cached_shape == current_shape:                           # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "    # -- Display message for cached data
                    print("-" * len(cached_display_name))                   # -- Print separator
                    print(cached_display_name)                              # -- Print message
                    print("-" * len(cached_display_name))                   # -- Print separator
                    display(process_PMTCT_EID_PCR_Test_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                                  # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                        # -- Filter rows with non-zero gaps
            df=df_main,                                                     # -- Input DataFrame
            msg=No_gap_msg,                                                 # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                     # -- Filter for non-zero gaps
            opNeg=None,                                                     # -- No negative value filter
            opPos=None,                                                     # -- No positive value filter
            opZero=None,                                                    # -- No zero value filter
            opLT100=None                                                    # -- No less-than-100 filter
        )                                                                   # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                             # -- Check if no gaps were found
            if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_EID_PCR_Test_gap.cached_style         # -- Remove cached style
            if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_EID_PCR_Test_gap.cached_shape         # -- Remove cached shape
            return                                                          # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                               # -- Apply styling to filtered DataFrame
            df_main_gap.style                                               # -- Create style object
            .hide(axis='index')                                             # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)                      # -- Highlight non-zero gaps in red
        )                                                                   # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_EID_PCR_Test_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_EID_PCR_Test_gap.cached_shape = df_main.shape    # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]                  # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"             # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"         # -- Define path for image export
        report_sheet_name = report_name                                     # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():                   # -- Check for PMTCT Facility Delivery by PPW gaps
            report_description = (                                          # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns_spec[0]}\nplus {df_columns_spec[1]}\nplus {df_columns_spec[2]}\n"
                f"should be equal to {df_columns[3]}\nplus {df_columns_spec[4]}\n"  
                f"REPORT ONLY EID PCR TEST FOR 2025 LIVE BIRTHS BY PPW."         
                f"Note: Where this PMTCT EID PCR Test gap is true, please ignore the outlier."
            )                                                               # -- Close description

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                       # -- Export styled DataFrame and descriptions
            report_name=report_name,                                        # -- Report identifier
            df_style=df_main_gap_style,                                     # -- Styled DataFrame for export
            img_file_name=report_image_name,                                # -- Image file name
            img_file_path=report_image_path,                                # -- Image file path
            doc_description=report_description,                             # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                         # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                        # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                        # -- Excel file path
            xlm_sheet_name=report_sheet_name                                # -- Excel sheet name
        )                                                                   # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                                  # -- Check if display is requested
            widget_display_df(df_main_gap_style)                            # -- Display styled DataFrame in widget

    except Exception as e:                                                  # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")                # -- Print error details
        if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_style'):    # -- Clear cache if it exists
            del process_PMTCT_EID_PCR_Test_gap.cached_style             # -- Remove cached style
        if hasattr(process_PMTCT_EID_PCR_Test_gap, 'cached_shape'):    # -- Clear cached shape
            del process_PMTCT_EID_PCR_Test_gap.cached_shape             # -- Remove cached shape
        return                                                              # -- Exit function on error
    # -- End of function                                                    # -- End of function

#### - PMTCT EID PCR Test Result

In [ ]:
# -- Define the main function to process PMTCT EID PCR Test gap
def process_PMTCT_EID_PCR_Test_Result_gap(display_output=None):              # -- Define function with optional display parameter
    """
    Process PMTCT EID PCR Test gap, exporting results as image, Excel, and Word files.
    Caches the styled df and displays it on subsequent calls if data shape unchanged.
    
    Args:
        display_output (bool, optional): If True, displays the df for gap. Defaults to None (treated as False unless explicitly True).
    """
    try:                                                                # -- Begin exception handling block
        # -- Step 1: Initialize constants
        df_columns = [                                                  # -- Define list of facility HEI ARVs columns
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test within 72 hrs of birth",
            "Number of Infants born to HIV positive women whose blood samples were taken for DNA PCR test between >72 hrs - < 2 months of birth",
            "Number of HIV PCR results received for babies whose samples were taken within 72 hrs of birth",
            "Number of HIV PCR results received for babies whose samples were taken between >72 hrs - < 2 months of birth"  # -- ARV after 72hrs column
        ]                                                                                                                  # -- Close list of facility HEI ARVs columns
        name = "PMTCT EID PCR Test Result gap"                            # -- Define general name for reporting
        gap_columns = ["PMTCT EID PCR Test Result gap"]                   # -- Define list of gap columns
        report_name = f"{name}35"                                       # -- Define unique report identifier
        No_gap_msg = f"No {report_name}"                                # -- Define message for no gaps scenario

        # -- Step 2: Fetch and prepare primary data
        df_main = prepare_and_convert_df(                               # -- Fetch and prepare DataFrame from DHIS2 data
            DHIS2_data_key="PMTCT MSF",                                 # -- Specify DHIS2 data key for PMTCT MSF dataset
            hierarchy_columns=MSF_hierarchy,                            # -- Use MSF hierarchy columns for organization
            data_columns=df_columns                                     # -- Include specified PMTCT columns
        )                                                               # -- Store prepared DataFrame
        if df_main is None:                                             # -- Check if data preparation failed
            return                                                      # -- Exit function if no data
        
        # -- Step 8: Calculate PMTCT EID gap
        df_main[gap_columns[0]] = np.where(                                 # -- Calculate PMTCT ARV prophylaxis gap
            df_main[df_columns[0:2]].sum(axis=1) != df_main[df_columns[2:4]].sum(axis=1),  # -- Check if sum of ARV prophylaxis differs from total live births
            df_main[df_columns[0:2]].sum(axis=1) - df_main[df_columns[2:4]].sum(axis=1),  # -- Calculate gap as sum minus total live births
            0                                                               # -- Set to 0 if no gap
        )       

        # -- Step 9: Wrap column headers for better readability
        wrap_column_headers(df_main)                                        # -- Apply wrapping to DataFrame headers
        #df_columns_spec_wrap = wrap_column_headers2(df_columns_spec)
        gap_columns_wrap = wrap_column_headers2(gap_columns)                # -- Wrap gap column names

        # -- Step 10: Check and display cached styled DataFrame
        if display_output:                                                  # -- Check if display output is requested
            if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_style'):  # -- Check for cached styled DataFrame
                cached_shape = getattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_shape', None)  # -- Retrieve cached shape
                current_shape = df_main.shape                               # -- Get current DataFrame shape
                if cached_shape == current_shape:                           # -- Compare shapes to ensure consistency
                    cached_display_name = f"✔️ Displaying {report_name} "    # -- Display message for cached data
                    print("-" * len(cached_display_name))                   # -- Print separator
                    print(cached_display_name)                              # -- Print message
                    print("-" * len(cached_display_name))                   # -- Print separator
                    display(process_PMTCT_EID_PCR_Test_Result_gap.cached_style)  # -- Display cached styled DataFrame
                    return                                                  # -- Exit function after displaying cache

        # -- Step 11: Filter and validate gaps
        df_main_gap = filter_gap_and_check_empty_df(                        # -- Filter rows with non-zero gaps
            df=df_main,                                                     # -- Input DataFrame
            msg=No_gap_msg,                                                 # -- Message if no gaps found
            opNonZero=gap_columns_wrap,                                     # -- Filter for non-zero gaps
            opNeg=None,                                                     # -- No negative value filter
            opPos=None,                                                     # -- No positive value filter
            opZero=None,                                                    # -- No zero value filter
            opLT100=None                                                    # -- No less-than-100 filter
        )                                                                   # -- Store filtered DataFrame with gaps
        if df_main_gap is None:                                             # -- Check if no gaps were found
            if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_style'):  # -- Clear cache if it exists
                del process_PMTCT_EID_PCR_Test_Result_gap.cached_style         # -- Remove cached style
            if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_shape'):  # -- Clear cached shape
                del process_PMTCT_EID_PCR_Test_Result_gap.cached_shape         # -- Remove cached shape
            return                                                          # -- Exit function

        # -- Step 12: Style the filtered DataFrame
        df_main_gap_style = (                                               # -- Apply styling to filtered DataFrame
            df_main_gap.style                                               # -- Create style object
            .hide(axis='index')                                             # -- Hide row index for cleaner output
            .map(outlier_yellow, subset=gap_columns_wrap)                      # -- Highlight non-zero gaps in red
        )                                                                   # -- Store styled DataFrame

        # -- Step 13: Cache styled DataFrame and shape
        process_PMTCT_EID_PCR_Test_Result_gap.cached_style = df_main_gap_style  # -- Store styled DataFrame
        process_PMTCT_EID_PCR_Test_Result_gap.cached_shape = df_main.shape    # -- Store DataFrame shape

        # -- Step 14: Prepare export variables
        report_month = df_main_gap['ReportPeriod'].iloc[0]                  # -- Extract report period
        report_image_name = f"{report_month}_{report_name}.png"             # -- Define image file name with period
        report_image_path = f"{sub_folder2_image_file_msf_outlier}"         # -- Define path for image export
        report_sheet_name = report_name                                     # -- Define Excel sheet name

        # -- Step 15: Create descriptions for Word document
        if (df_main_gap[gap_columns_wrap[0]] != 0).any():                   # -- Check for PMTCT Facility Delivery by PPW gaps
            report_description = (                                          # -- Define description for gap
                f"Report Name: {gap_columns[0]}\n"
                f"{df_columns[0]}\nplus {df_columns[1]}\n"
                f"should be equal to {df_columns[2]}\nplus {df_columns[3]}\n"  
                f"REPORT ONLY EID PCR TEST RESULT FOR 2025 LIVE BIRTHS BY PPW."         
                f"Note: Where this {name} is true, please ignore the outlier."
            )                                                               # -- Close description

        # -- Step 16: Export results to multiple formats
        export_df_to_doc_image_excel(                                       # -- Export styled DataFrame and descriptions
            report_name=report_name,                                        # -- Report identifier
            df_style=df_main_gap_style,                                     # -- Styled DataFrame for export
            img_file_name=report_image_name,                                # -- Image file name
            img_file_path=report_image_path,                                # -- Image file path
            doc_description=report_description,                             # -- Description for Word document
            doc_indicators_to_italicize=df_columns,                         # -- Italicize specified columns in Word
            doc_indicators_to_underline=gap_columns,                        # -- Underline gap columns in Word
            xlm_file_path=doc_file_msf_outlier_xlsx,                        # -- Excel file path
            xlm_sheet_name=report_sheet_name                                # -- Excel sheet name
        )                                                                   # -- Execute export to multiple formats

        # -- Step 17: Display styled DataFrame if requested
        if display_output:                                                  # -- Check if display is requested
            widget_display_df(df_main_gap_style)                            # -- Display styled DataFrame in widget

    except Exception as e:                                                  # -- Handle any errors during execution
        print(f"⦸ Error processing {report_name}: {str(e)}")                # -- Print error details
        if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_style'):    # -- Clear cache if it exists
            del process_PMTCT_EID_PCR_Test_Result_gap.cached_style             # -- Remove cached style
        if hasattr(process_PMTCT_EID_PCR_Test_Result_gap, 'cached_shape'):    # -- Clear cached shape
            del process_PMTCT_EID_PCR_Test_Result_gap.cached_shape             # -- Remove cached shape
        return                                                              # -- Exit function on error
    # -- End of function                                                    # -- End of function

## Console

In [ ]:
# -- List of function descriptions for display
function_description_name = [
    "Get Data",                                # -- Description for LGA report rate gap
    "Generate Report",                         # -- Description for report generating
    "ANSO Report Rate",                        # -- Description for LGA report rate gap
    "LGA Report Rate",                         # -- Description for facility report rate gap
    "AGYW HTS",                                # -- Description for AGYW HTS gap
    "AGYW Pos",                                # -- Description for AGYW positive gap
    "AGYW Pos Linkage",                        # -- Description for AGYW positive linkage gap 
    "AGYW TB Screening",                       # -- Description for AGYW TB screening gap
    "ART PosEnrolment",                        # -- Description for ART positive enrolment gap 
    "ART-3 subsets",                           # -- Description for ART regimen line, MMD and DSD
    "ART Tx_New TB Screening",                 # -- Description for ART Tx_New TB screening gap
    "ART TB Presumptive Test",                 # -- Description for ART TB presumptive test gap
    "ART TB Treatment",                        # -- Description for ART TB treatment gap
    "ART Viral Load Suppression",              # -- Description for ART viral load suppression gap
    "HTS New Positive",                        # -- Description for HTS new positive gap
    "HTS TB Screening",                        # -- Description for HTS TB screning gap
    "HTS Enrolment",                           # -- Description for HTS enrolment 
    "HTS Discordant Couples Test",             # -- Description for HTS discordant couple testing gap
    "HTS CD4 Test",                            # -- Description for HTS CD4 test result gap
    "HIVST Mode",                              # -- Description for HIVST distribution mode gap
    "HIVST Test Freq.",                        # -- Description for HIVST distribution mode gap
    "HIVST Result",                            # -- Description for HIVST result gap 
    "HIVST Reactive Link",                     # -- Description for HIVST reactive, confirmation and linkage gap
    "HIVST Prevention Service",                # -- Description for HIVST prevention service and linkage gap
    "HIVST Partner Screening",                 # -- Description for HIVST partner screening gap 
    "ICT Index Acceptance",                    # -- Description for ICT index acceptance gap
    "ICT Contact",                             # -- Description for ICT contact gap
    "ICT HTS",                                 # -- Description for ICT HTS gap 
    "ICT Positive Linkage",                    # -- Description for ICT poisitive linkage gap
    "PMTCT New ANC HTS",                       # -- Description for PMTCT new ANC gap
    "PMTCT Positive",                          # -- Description for PMTCT positive gap
    "PMTCT Previously Known",                  # -- Description for PMTCT previously known gap 
    "PMTCT Positive Linkage",                  # -- Description for PMTCT positive linkage
    "PMTCT Seroconversion",                    # -- Description for PMTCT seroconversion gap 
    "PMTCT Syphilis Test",                     # -- Description for PMTCT syphilis test gap 
    "PMTCT Hepatitis Test",                    # -- Description for PMTCT syphilis test gap 
    "PMTCT Labour and Delivery",               # -- Description for PMTCT labour and delivery gap 
    "PMTCT Facility HEI ARVs",                 # -- Description for PMTCT facility HEI ARVs gap 
    "PMTCT EID PCR Test"                       # -- Description for PMTCT EID PCR test gap  
]

# -- Define constants for UI elements
ui_title = "ANSO IHVN DHIS2 MSF Console"       # -- Report title to be displayed in bold
author = "Reuben Edidiong"                     # -- Author name, kept plain due to terminal italic limitation
version = "msf.vlr v1.0"                       # -- Version identifier for the report
ui_sepperator_line = 150                       # -- Length of separator lines (adjustable; 80 for cleaner look)
bold = "\033[1m"                               # -- ANSI code for bold text
reset = "\033[0m"                              # -- ANSI code to reset formatting

# -- Core components for separators
header = f"{bold}{ui_title}{reset} {f'© {author} {version}':>122}"  # -- Header with bold title, right-aligned copyright
top_line = f"{'-' * ui_sepperator_line}\n"       # -- Top separator line
bottom_line = f"\n{'-' * ui_sepperator_line}"  # -- Bottom separator line
spacing = "\n" * 15                            # -- Empty line gap in ui_separator_clear

# -- Separator definitions
ui_separator_top = f"{header}\n{top_line}"     # -- Top separator: header followed by a single line
ui_separator_bottom = f"{bottom_line}"         # -- Bottom separator: just a single line
ui_separator_clear = (                         # -- Full clear separator: header, top line, spacing, bottom line
    f"{header}\n"
    f"{top_line}"
    f"{spacing}"
    f"{bottom_line}"
)

def run_jupyter_mode():
    """
    Runs an interactive Jupyter interface with buttons to execute report processing functions.
    Group names are displayed horizontally with a dropdown arrow, bold text, and font size increased by 2 points.
    Sub-buttons appear when a group name is clicked.
    
    Args:
        None
    
    Returns:
        None 
    """

    # -- Step 1: Create buttons with descriptive labels
    botton0 = widgets.Button(description=f"{function_description_name[0]}")
    botton1 = widgets.Button(description=f"{function_description_name[1]}")
    botton2 = widgets.Button(description=f"{function_description_name[2]}")
    botton3 = widgets.Button(description=f"{function_description_name[3]}")
    botton4 = widgets.Button(description=f"{function_description_name[4]}")
    botton5 = widgets.Button(description=f"{function_description_name[5]}")
    botton6 = widgets.Button(description=f"{function_description_name[6]}")
    botton7 = widgets.Button(description=f"{function_description_name[7]}")
    botton8 = widgets.Button(description=f"{function_description_name[8]}")
    botton9 = widgets.Button(description=f"{function_description_name[9]}")
    botton10 = widgets.Button(description=f"{function_description_name[10]}")
    botton11 = widgets.Button(description=f"{function_description_name[11]}")
    botton12 = widgets.Button(description=f"{function_description_name[12]}")
    botton13 = widgets.Button(description=f"{function_description_name[13]}")
    botton14 = widgets.Button(description=f"{function_description_name[14]}")
    botton15 = widgets.Button(description=f"{function_description_name[15]}")
    botton16 = widgets.Button(description=f"{function_description_name[16]}")
    botton17 = widgets.Button(description=f"{function_description_name[17]}")
    botton18 = widgets.Button(description=f"{function_description_name[18]}")
    botton19 = widgets.Button(description=f"{function_description_name[19]}")
    botton20 = widgets.Button(description=f"{function_description_name[20]}")
    botton21 = widgets.Button(description=f"{function_description_name[21]}")
    botton22 = widgets.Button(description=f"{function_description_name[22]}")
    botton23 = widgets.Button(description=f"{function_description_name[23]}")
    botton24 = widgets.Button(description=f"{function_description_name[24]}")
    botton25 = widgets.Button(description=f"{function_description_name[25]}")
    botton26 = widgets.Button(description=f"{function_description_name[26]}")
    botton27 = widgets.Button(description=f"{function_description_name[27]}")
    botton28 = widgets.Button(description=f"{function_description_name[28]}")
    botton29 = widgets.Button(description=f"{function_description_name[29]}")
    botton30 = widgets.Button(description=f"{function_description_name[30]}")
    botton31 = widgets.Button(description=f"{function_description_name[31]}")
    botton32 = widgets.Button(description=f"{function_description_name[32]}")
    botton33 = widgets.Button(description=f"{function_description_name[33]}") 
    botton34 = widgets.Button(description=f"{function_description_name[34]}")
    botton35 = widgets.Button(description=f"{function_description_name[35]}") 
    botton36 = widgets.Button(description=f"{function_description_name[36]}")
    botton37 = widgets.Button(description=f"{function_description_name[37]}")
    botton38 = widgets.Button(description=f"{function_description_name[38]}")
    clear_button = widgets.Button(description="Clear screen")
    output = widgets.Output()

    # -- Step 2: Define button handlers 
    def on_botton0_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            fetch_dhis2_data_interactive_jupyter_mode()
            print(ui_separator_bottom)
            
    def on_botton2_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_lga_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton3_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_facility_report_rate_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton4_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_HTS_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton5_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton6_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton7_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_AGYW_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton8_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_PosEnrolment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton9_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_RegimentLine_MMD_DSD_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton10_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton11_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Presumptive_Test_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton12_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_TB_Treatment_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton13_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ART_Viral_Load_Suppression_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton14_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_New_Positive_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton15_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_TB_Screening_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton16_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_Enrolment_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton17_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_Couple_Counselling_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton18_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HTS_CD4_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton19_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Distr_Mode_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton20_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Test_Freq_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton21_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Result_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton22_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Reactive_Link_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton23_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Prevention_Serv_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton24_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_HIVST_Partner_Screening_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton25_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Index_Acceptance_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton26_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Contact_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton27_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_HTS_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton28_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_ICT_Positive_Link_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton29_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_ANC_Optmz_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton30_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Positive_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton31_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_PK_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton32_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Positive_Linkage_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton33_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Seroconversion_gap(display_output=True)
            print(ui_separator_bottom) 
    
    def on_botton34_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Syphilis_Test_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton35_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Hepatitis_Test_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton36_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Labour_Delivery_gap(display_output=True)
            print(ui_separator_bottom)

    def on_botton37_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_Facility_HEI_ARVs_gap(display_output=True)
            print(ui_separator_bottom)
    
    def on_botton38_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_PMTCT_EID_PCR_Test_gap(display_output=True)
            print(ui_separator_bottom)

    def on_clear_button_click(b):
        with output:
            clear_output()
            print(ui_separator_clear)

    def on_botton1_click(b):
        with output:
            clear_output()
            print(ui_separator_top)
            process_lga_report_rate_gap(display_output=False)
            process_facility_report_rate_gap(display_output=False)
            process_AGYW_HTS_gap(display_output=False)
            process_AGYW_Positive_gap(display_output=False)
            process_AGYW_Positive_Linkage_gap(display_output=False)
            process_AGYW_TB_Screening_gap(display_output=False)
            process_ART_PosEnrolment_gap(display_output=False)
            process_ART_RegimentLine_MMD_DSD_gap(display_output=False)
            process_ART_TB_Screening_gap(display_output=False)
            process_ART_TB_Presumptive_Test_gap(display_output=False)
            process_ART_TB_Treatment_gap(display_output=False)
            process_ART_Viral_Load_Suppression_gap(display_output=False)
            process_HTS_New_Positive_gap(display_output=False)
            process_HTS_TB_Screening_gap(display_output=False)
            process_HTS_Couple_Counselling_gap(display_output=False)
            process_HTS_Enrolment_gap(display_output=False)
            process_HTS_CD4_gap(display_output=False)
            process_HIVST_Distr_Mode_gap(display_output=False)
            process_HIVST_Test_Freq_gap(display_output=False)
            process_HIVST_Result_gap(display_output=False)
            process_HIVST_Reactive_Link_gap(display_output=False)
            process_HIVST_Prevention_Serv_gap(display_output=False)
            process_HIVST_Partner_Screening_gap(display_output=False)
            process_ICT_Index_Acceptance_gap(display_output=False)
            process_ICT_Contact_gap(display_output=False)
            process_ICT_HTS_gap(display_output=False)
            process_ICT_Positive_Link_gap(display_output=False)
            process_PMTCT_ANC_Optmz_gap(display_output=False)
            process_PMTCT_Positive_gap(display_output=False) 
            process_PMTCT_PK_gap(display_output=False)
            process_PMTCT_Positive_Linkage_gap(display_output=False)
            process_PMTCT_Seroconversion_gap(display_output=False)
            process_PMTCT_Syphilis_Test_gap(display_output=False)
            process_PMTCT_Hepatitis_Test_gap(display_output=False)
            process_PMTCT_Labour_Delivery_gap(display_output=False)
            process_PMTCT_Facility_HEI_ARVs_gap(display_output=False)
            process_PMTCT_EID_PCR_Test_gap(display_output=False)
            print(ui_separator_bottom)

    # -- Step 3: Link buttons to their handlers
    botton0.on_click(on_botton0_click)
    botton1.on_click(on_botton1_click)
    botton2.on_click(on_botton2_click)
    botton3.on_click(on_botton3_click)
    botton4.on_click(on_botton4_click)
    botton5.on_click(on_botton5_click)
    botton6.on_click(on_botton6_click)
    botton7.on_click(on_botton7_click)
    botton8.on_click(on_botton8_click)
    botton9.on_click(on_botton9_click)
    botton10.on_click(on_botton10_click)
    botton11.on_click(on_botton11_click)
    botton12.on_click(on_botton12_click)
    botton13.on_click(on_botton13_click)
    botton14.on_click(on_botton14_click)
    botton15.on_click(on_botton15_click)
    botton16.on_click(on_botton16_click)
    botton17.on_click(on_botton17_click)
    botton18.on_click(on_botton18_click)
    botton19.on_click(on_botton19_click)
    botton20.on_click(on_botton20_click)
    botton21.on_click(on_botton21_click)
    botton22.on_click(on_botton22_click)
    botton23.on_click(on_botton23_click)
    botton24.on_click(on_botton24_click)
    botton25.on_click(on_botton25_click)
    botton26.on_click(on_botton26_click)
    botton27.on_click(on_botton27_click)
    botton28.on_click(on_botton28_click)
    botton29.on_click(on_botton29_click)
    botton30.on_click(on_botton30_click)
    botton31.on_click(on_botton31_click)
    botton32.on_click(on_botton32_click)
    botton33.on_click(on_botton33_click)
    botton34.on_click(on_botton34_click)
    botton35.on_click(on_botton35_click)
    botton36.on_click(on_botton36_click)
    botton37.on_click(on_botton37_click)
    botton38.on_click(on_botton38_click)
    clear_button.on_click(on_clear_button_click)

    # -- Step 4: Create group buttons with dropdown arrow, bold text, and larger font
    # -- Define a consistent layout for button width
    group_button_layout = widgets.Layout(width='150px')  # You can adjust the width as needed

    # -- Define style for button text
    group_button_style = {'font_weight': 'bold', 'font_size': '12px'}
    
    general_button = widgets.Button(
        description="General Actions ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    report_rate_botton = widgets.Button(
        description="Report Rates ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    agyw_button = widgets.Button(
        description="AGYW Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    art_button = widgets.Button(
        description="ART Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    hts_button = widgets.Button(
        description="HTS Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    hivst_button = widgets.Button(
        description="HIVST Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    ict_button = widgets.Button(
        description="ICT Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )
    pmtct_button = widgets.Button(
        description="PMTCT Reports ▼",
        layout=group_button_layout,
        style=group_button_style
    )


    # -- Define sub-button containers
    general_sub_buttons = widgets.HBox([botton0, botton1, clear_button])
    report_rate_sub_botton = widgets.HBox([botton2, botton3, clear_button])
    agyw_sub_buttons = widgets.HBox([botton4, botton5, botton6, botton7, clear_button])
    art_sub_buttons = widgets.HBox([botton8, botton9, botton10, botton11, botton12, botton13, clear_button])
    hts_sub_buttons = widgets.HBox([botton14, botton15, botton16, botton17, botton18,  clear_button])
    hivst_sub_buttons = widgets.HBox([botton19, botton20, botton21, botton22, botton23, botton24, clear_button])
    ict_sub_buttons = widgets.HBox([botton25, botton26, botton27, botton28, clear_button])
    pmtct_sub_buttons = widgets.HBox([botton29, botton30, botton31, botton32, botton33, botton34, 
                                      botton35, botton36, botton37, botton38, clear_button])

    # -- Track the currently open group
    current_open = [None]
    sub_button_area = widgets.VBox([])

    # -- Step 5: Define group button handlers to toggle sub-buttons
    def update_button_descriptions(closed_button, opened_button):
        for btn in [general_button, report_rate_botton, agyw_button, art_button, 
                    hts_button, hivst_button, ict_button, pmtct_button]:
            if btn != opened_button and btn.description.endswith("▲"):
                btn.description = btn.description.replace("▲", "▼")
        if closed_button and closed_button.description.endswith("▲"):
            closed_button.description = closed_button.description.replace("▲", "▼")
        if opened_button and not opened_button.description.endswith("▲"):
            opened_button.description = opened_button.description.replace("▼", "▲")

    def on_general_button_click(b):
        if current_open[0] == general_button:
            sub_button_area.children = []
            update_button_descriptions(general_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [general_sub_buttons]
            update_button_descriptions(current_open[0], general_button)
            current_open[0] = general_button

    def on_report_rate_botton_click(b):
        if current_open[0] == report_rate_botton:
            sub_button_area.children = []
            update_button_descriptions(report_rate_botton, None)
            current_open[0] = None
        else:
            sub_button_area.children = [report_rate_sub_botton]
            update_button_descriptions(current_open[0], report_rate_botton)
            current_open[0] = report_rate_botton

    def on_agyw_button_click(b):
        if current_open[0] == agyw_button:
            sub_button_area.children = []
            update_button_descriptions(agyw_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [agyw_sub_buttons]
            update_button_descriptions(current_open[0], agyw_button)
            current_open[0] = agyw_button

    def on_art_button_click(b):
        if current_open[0] == art_button:
            sub_button_area.children = []
            update_button_descriptions(art_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [art_sub_buttons]
            update_button_descriptions(current_open[0], art_button)
            current_open[0] = art_button
    
    def on_hts_button_click(b):
        if current_open[0] == hts_button:
            sub_button_area.children = []
            update_button_descriptions(hts_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [hts_sub_buttons]
            update_button_descriptions(current_open[0], hts_button)
            current_open[0] = hts_button

    def on_hivst_button_click(b):
        if current_open[0] == hivst_button:
            sub_button_area.children = []
            update_button_descriptions(hivst_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [hivst_sub_buttons]
            update_button_descriptions(current_open[0], hivst_button)
            current_open[0] = hivst_button
    
    def on_ict_button_click(b):
        if current_open[0] == ict_button:
            sub_button_area.children = []
            update_button_descriptions(ict_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [ict_sub_buttons]
            update_button_descriptions(current_open[0], ict_button)
            current_open[0] = ict_button
    
    def on_pmtct_button_click(b):
        if current_open[0] == pmtct_button:
            sub_button_area.children = []
            update_button_descriptions(pmtct_button, None)
            current_open[0] = None
        else:
            sub_button_area.children = [pmtct_sub_buttons]
            update_button_descriptions(current_open[0], pmtct_button)
            current_open[0] = pmtct_button



    # -- Step 6: Link group buttons to their handlers
    general_button.on_click(on_general_button_click)
    report_rate_botton.on_click(on_report_rate_botton_click)
    agyw_button.on_click(on_agyw_button_click)
    art_button.on_click(on_art_button_click)
    hts_button.on_click(on_hts_button_click)
    hivst_button.on_click(on_hivst_button_click)
    ict_button.on_click(on_ict_button_click)
    pmtct_button.on_click(on_pmtct_button_click)
    # -- Step 7: Create a horizontal layout for group buttons
    group_buttons = widgets.HBox([
        general_button,
        report_rate_botton,
        agyw_button,
        art_button,
        hts_button,
        hivst_button,
        ict_button,
        pmtct_button
    ], layout=widgets.Layout(
        align_items="flex-start",
        padding="10px"
    ))

    # -- Step 8: Create the main layout
    layout = widgets.VBox([
        group_buttons,
        sub_button_area,
        output

    ], layout=widgets.Layout(
        align_items="flex-start",
        padding="10px"
    ))

    # -- Step 9: Display the interface
    display(layout)

# -- Ensure this is the last cell in your notebook
run_jupyter_mode()